# SEC EDGAR - Filings' Data Extraction Part 1

## Import Libraries

In [1]:
# import necessary libraries
import pandas as pd
import datetime as dt
import requests
from bs4 import BeautifulSoup
import time

# note: the requests library and the BeautifulSoup library will be used for web scraping

## Read Dataset to Pandas Dataframe

In [2]:
# read fundamentals_fscore_copy.csv file into a pandas dataframe
funda_df = pd.read_csv('fundamentals_fscore_copy.csv')
# display funda_df
display(funda_df.head())

,CIK,Ticker_Symbol,Company_Name,Sector,Data_Date,Market_Capitalization,Total_Assets,Total_Assets_PY1,Total_Assets_PY2,Total_Liabilities,...,F_CLEVER,CLIQUID,F_CLIQUID,EQ_OFFER,F_EQ_OFFER,CMARGIN,F_CMARGIN,CTURN,F_CTURN,F_SCORE
0,1750,AIR,AAR CORP,Capital Goods,2011-05-31,1049.8206,1703.727,1501.042,1377.511,868.438,...,1,-0.455190,0,0.297,0,-0.004629,0,0.201443,1,6
1,1750,AIR,AAR CORP,Capital Goods,2012-05-31,485.2897,2195.653,1703.727,1501.042,1329.631,...,0,0.049832,1,0.492,0,-0.008424,0,0.034591,1,5
2,1750,AIR,AAR CORP,Capital Goods,2013-05-31,790.0029,2136.900,2195.653,1703.727,1217.400,...,1,0.410468,1,-0.891,1,0.010205,1,-0.230628,0,7
3,1750,AIR,AAR CORP,Capital Goods,2014-05-31,961.3080,2199.500,2136.900,2195.653,1198.800,...,1,0.120341,1,0.178,0,0.014049,1,-0.034682,0,7
4,1750,AIR,AAR CORP,Capital Goods,2015-05-31,1046.3954,1515.000,2199.500,2136.900,669.900,...,1,-0.461891,0,-4.137,1,-0.065087,0,-0.227468,0,3


In [3]:
# convert values in the Data_Date column of funda_df to datetime 
funda_df['Data_Date'] = pd.to_datetime(funda_df['Data_Date'])

## Filter High Book-to-Market Companies

In [4]:
# note: piotroski's study focuses on companies with high b/m ratio values
# note: i.e. the companies that are contained in the highest b/m quintile
# note: thus, it makes sense to conduct exploratory data analysis on these companies only

# filter high b/m valued companies and reset index of funda_df
funda_df = funda_df[funda_df['BM_Quintile']=='Very High'].reset_index(drop=True)
# row count of funda_df after filtering
print('funda_df row count:', funda_df.shape[0])

funda_df row count: 1459


## Retrieve Filing Page URLs

In [5]:
# note: the SEC's EDGAR database contains filings from (nearly) every publicly listed company in the US
# note: filings can be obtained through web scraping EDGAR
# note: obtain the filing page url for the observations in funda_df

# create dataframe to store filing page url
filing_page_df = pd.DataFrame(columns=['CIK', 'Ticker_Symbol', 'Company_Name', 'Data_Date', 'Filing_Date',
                                       'Filing_Type', 'Filing_Url', 'MDA_Placement'])
# set the CIK column in filing_page_df to that of funda_df
filing_page_df['CIK'] = funda_df['CIK'] 
# set the Ticker_Symbol column in filing_page_df to that of funda_df
filing_page_df['Ticker_Symbol'] = funda_df['Ticker_Symbol']
# set the Company_Name column in filing_page_df to that of funda_df
filing_page_df['Company_Name'] = funda_df['Company_Name']
# set the Data_Date column in filing_page_df to the Data_Date column in funda_df
filing_page_df['Data_Date'] = funda_df['Data_Date']

In [6]:
# SEC EDGAR database search function
# note: use a company's cik number to search the SEC EDGAR database
def edgar_search(cik, filing_type):
    # define base url for searching EDGAR
    edgar_search_base_url = r'https://www.sec.gov/cgi-bin/browse-edgar'
    # define parameters that needed to search EDGAR for specific companies and filing types (e.g. 10-k)
    edgar_search_params = {'action':'getcompany',
                            'CIK':cik,
                            'type':filing_type,
                            'dateb':'',
                            'owner':'exclude',
                            'start':'',
                            'output':'',
                            'count':''}
    # request eadgar search results
    edgar_search_request = requests.get(url=edgar_search_base_url, params=edgar_search_params)
    # parse edgar_search_request response html
    edgar_search_soup = BeautifulSoup(edgar_search_request.content, 'html.parser')
    # return soup
    return edgar_search_soup

In [7]:
# SEC EDGAR filing date function
def edgar_filing_date(date, soup, filing_type):
    # locate table containing listed filings
    listed_filings = soup.find_all('table', class_='tableFile2')
    # create dataframe to store filing dates
    filing_date_df = pd.DataFrame(columns=['Filing_Date'])
    # set return variable to N/A
    filing_date = 'N/A'
    # loop over listings in listed_filings to source filing date
    for idx, row in enumerate(listed_filings[0].find_all('tr')):
        row_column = row.find_all('td')
        if (len(row_column)!=0 and row_column[0].text.strip() in filing_type and row_column[3].text.strip()[:4]==str(date.to_pydatetime().year)):
            filing_date_df.loc[len(filing_date_df), 'Filing_Date'] = row_column[3].text.strip()
            filing_date_df = pd.to_datetime(filing_date_df['Filing_Date'])
            if len(filing_date_df)>1:
                if date.to_pydatetime().month > filing_date_df.loc[(len(filing_date_df)-1)].to_pydatetime().month and (date.to_pydatetime().year + 1)==filing_date_df.loc[(len(filing_date_df)-2)].to_pydatetime().year:
                    filing_date = filing_date_df.loc[(len(filing_date_df)-2)].strftime('%Y-%m-%d')
                    break
                elif date.to_pydatetime().month < filing_date_df.loc[(len(filing_date_df)-1)].to_pydatetime().month:
                    filing_date = filing_date_df.loc[(len(filing_date_df)-1)].strftime('%Y-%m-%d')
                    break
                else:
                    break
            else:
                if date.to_pydatetime().month < filing_date_df.loc[(len(filing_date_df)-1)].to_pydatetime().month:
                    filing_date = filing_date_df.loc[(len(filing_date_df)-1)].strftime('%Y-%m-%d')
                    break
                else:
                    break
        elif len(row_column)!=0 and row_column[0].text.strip() in filing_type:
            filing_date_df.loc[len(filing_date_df), 'Filing_Date'] = row_column[3].text.strip()
            continue
        elif len(row_column)!=0:
            continue
        elif idx==0:
            continue
        else:
            break
    # return filing_date    
    return filing_date

In [8]:
# SEC EDGAR filing page function
def edgar_filing_page(date, soup, filing_type):
    # define base url for filing page url
    edgar_filing_page_base_url = r'https://www.sec.gov'
    # locate table containing listed filings
    listed_filings = soup.find_all('table', class_='tableFile2')
    # loop over listings in listed_filings table to source filing page url for given year
    for idx, row in enumerate(listed_filings[0].find_all('tr')):
        # find all columns in listed_filings
        row_column = row.find_all('td')
        # retrieve filing page url for given date
        if (len(row_column)!=0 and row_column[0].text.strip() in filing_type and row_column[3].text.strip()==date):
            filing_page_url = edgar_filing_page_base_url + \
            row_column[1].find('a', {'href':True, 'id':'documentsbutton'})['href']
            break
        elif len(row_column)!=0:
            continue
        else:
            filing_page_url = 'N/A'
    
    # return filing_page_url
    return filing_page_url

In [9]:
# time 
start = time.time()

# retrieve the filing date and filing page url for each observation in filing_page_df
for idx, row in filing_page_df.iterrows():
    filing_date = edgar_filing_date(row['Data_Date'], edgar_search(row['CIK'], '10-K'), ['10-K'])
    filing_url = edgar_filing_page(filing_date, edgar_search(row['CIK'], '10-K'), ['10-K'])
    filing_page_df.loc[idx, 'Filing_Date'] = filing_date
    filing_page_df.loc[idx, 'Filing_Url'] = filing_url
    # set the Filing_Type column and MDA_Placement in filing_page_df
    if filing_url != 'N/A':
        filing_page_df.loc[idx, 'Filing_Type'] = '10-K'
        filing_page_df.loc[idx, 'MDA_Placement'] = '10-K'
    else:
        filing_page_df.loc[idx, 'Filing_Type'] = 'N/A'
        filing_page_df.loc[idx, 'MDA_Placement'] = 'N/A'
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(row['Ticker_Symbol'])
    print('period of report:', row['Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_date)
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_url)
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

end = time.time()

print('total time:', end - start)

-------------------------------------------------------------------------------------
AIR
period of report: 2012-05-31
filing date: 2012-07-19
filing type: 10-K
filing url: https://www.sec.gov/Archives/edgar/data/1750/000104746912007300/0001047469-12-007300-index.htm
management discussion and analysis placement: 10-K


-------------------------------------------------------------------------------------
AIR
period of report: 2013-05-31
filing date: 2013-07-26
filing type: 10-K
filing url: https://www.sec.gov/Archives/edgar/data/1750/000104746913007797/0001047469-13-007797-index.htm
management discussion and analysis placement: 10-K


-------------------------------------------------------------------------------------
AIR
period of report: 2014-05-31
filing date: 2014-07-17
filing type: 10-K
filing url: https://www.sec.gov/Archives/edgar/data/1750/000104746914006243/0001047469-14-006243-index.htm
management discussion and analysis placement: 10-K


-------------------------------------

-------------------------------------------------------------------------------------
SSI
period of report: 2016-01-31
filing date: 2016-03-30
filing type: 10-K
filing url: https://www.sec.gov/Archives/edgar/data/6885/000000688516000411/0000006885-16-000411-index.htm
management discussion and analysis placement: 10-K


-------------------------------------------------------------------------------------
SSI
period of report: 2017-01-31
filing date: 2017-04-04
filing type: 10-K
filing url: https://www.sec.gov/Archives/edgar/data/6885/000000688517000079/0000006885-17-000079-index.htm
management discussion and analysis placement: 10-K


-------------------------------------------------------------------------------------
SSI
period of report: 2018-01-31
filing date: 2018-04-10
filing type: 10-K
filing url: https://www.sec.gov/Archives/edgar/data/6885/000000688518000135/0000006885-18-000135-index.htm
management discussion and analysis placement: 10-K


-------------------------------------

-------------------------------------------------------------------------------------
LUB
period of report: 2012-08-31
filing date: 2012-11-13
filing type: 10-K
filing url: https://www.sec.gov/Archives/edgar/data/16099/000119312512465459/0001193125-12-465459-index.htm
management discussion and analysis placement: 10-K


-------------------------------------------------------------------------------------
LUB
period of report: 2013-08-31
filing date: 2013-11-12
filing type: 10-K
filing url: https://www.sec.gov/Archives/edgar/data/16099/000143774913014511/0001437749-13-014511-index.htm
management discussion and analysis placement: 10-K


-------------------------------------------------------------------------------------
LUB
period of report: 2014-08-31
filing date: 2014-11-12
filing type: 10-K
filing url: https://www.sec.gov/Archives/edgar/data/16099/000143774914020335/0001437749-14-020335-index.htm
management discussion and analysis placement: 10-K


----------------------------------

-------------------------------------------------------------------------------------
CMC
period of report: 2011-08-31
filing date: 2011-10-31
filing type: 10-K
filing url: https://www.sec.gov/Archives/edgar/data/22444/000095012311093536/0000950123-11-093536-index.htm
management discussion and analysis placement: 10-K


-------------------------------------------------------------------------------------
CMC
period of report: 2012-08-31
filing date: 2012-10-30
filing type: 10-K
filing url: https://www.sec.gov/Archives/edgar/data/22444/000002244412000006/0000022444-12-000006-index.htm
management discussion and analysis placement: 10-K


-------------------------------------------------------------------------------------
TAP
period of report: 2011-12-31
filing date: 2012-02-27
filing type: 10-K
filing url: https://www.sec.gov/Archives/edgar/data/24545/000144530512000454/0001445305-12-000454-index.htm
management discussion and analysis placement: 10-K


----------------------------------

-------------------------------------------------------------------------------------
EBF
period of report: 2012-02-29
filing date: 2012-05-11
filing type: 10-K
filing url: https://www.sec.gov/Archives/edgar/data/33002/000119312512227898/0001193125-12-227898-index.htm
management discussion and analysis placement: 10-K


-------------------------------------------------------------------------------------
EBF
period of report: 2013-02-28
filing date: 2013-05-10
filing type: 10-K
filing url: https://www.sec.gov/Archives/edgar/data/33002/000119312513214067/0001193125-13-214067-index.htm
management discussion and analysis placement: 10-K


-------------------------------------------------------------------------------------
EBF
period of report: 2014-02-28
filing date: 2014-05-09
filing type: 10-K
filing url: https://www.sec.gov/Archives/edgar/data/33002/000119312514192331/0001193125-14-192331-index.htm
management discussion and analysis placement: 10-K


----------------------------------

...

-------------------------------------------------------------------------------------
NEWM
period of report: 2016-12-31
filing date: 2017-02-21
filing type: 10-K
filing url: https://www.sec.gov/Archives/edgar/data/1579684/000157968417000005/0001579684-17-000005-index.htm
management discussion and analysis placement: 10-K


-------------------------------------------------------------------------------------
NEWM
period of report: 2018-12-31
filing date: 2019-02-28
filing type: 10-K
filing url: https://www.sec.gov/Archives/edgar/data/1579684/000157968419000003/0001579684-19-000003-index.htm
management discussion and analysis placement: 10-K


-------------------------------------------------------------------------------------
NVGS
period of report: 2015-12-31
filing date: N/A
filing type: N/A
filing url: N/A
management discussion and analysis placement: N/A


-------------------------------------------------------------------------------------
NVGS
period of report: 2016-12-31
filing d

-------------------------------------------------------------------------------------
PRTY
period of report: 2018-12-31
filing date: 2019-02-28
filing type: 10-K
filing url: https://www.sec.gov/Archives/edgar/data/1592058/000119312519057714/0001193125-19-057714-index.htm
management discussion and analysis placement: 10-K


-------------------------------------------------------------------------------------
PE
period of report: 2018-12-31
filing date: 2019-02-27
filing type: 10-K
filing url: https://www.sec.gov/Archives/edgar/data/1594466/000159446619000035/0001594466-19-000035-index.htm
management discussion and analysis placement: 10-K


-------------------------------------------------------------------------------------
RUBI
period of report: 2016-12-31
filing date: 2017-03-15
filing type: 10-K
filing url: https://www.sec.gov/Archives/edgar/data/1595974/000162828017002588/0001628280-17-002588-index.htm
management discussion and analysis placement: 10-K


---------------------------

-------------------------------------------------------------------------------------
NEP
period of report: 2018-12-31
filing date: 2019-02-19
filing type: 10-K
filing url: https://www.sec.gov/Archives/edgar/data/1603145/000160314519000014/0001603145-19-000014-index.htm
management discussion and analysis placement: 10-K


-------------------------------------------------------------------------------------
EURN
period of report: 2015-12-31
filing date: N/A
filing type: N/A
filing url: N/A
management discussion and analysis placement: N/A


-------------------------------------------------------------------------------------
EURN
period of report: 2016-12-31
filing date: N/A
filing type: N/A
filing url: N/A
management discussion and analysis placement: N/A


-------------------------------------------------------------------------------------
EURN
period of report: 2017-12-31
filing date: N/A
filing type: N/A
filing url: N/A
management discussion and analysis placement: N/A


----------

-------------------------------------------------------------------------------------
OR
period of report: 2017-12-31
filing date: N/A
filing type: N/A
filing url: N/A
management discussion and analysis placement: N/A


-------------------------------------------------------------------------------------
OR
period of report: 2018-12-31
filing date: N/A
filing type: N/A
filing url: N/A
management discussion and analysis placement: N/A


-------------------------------------------------------------------------------------
EVH
period of report: 2017-12-31
filing date: 2018-03-01
filing type: 10-K
filing url: https://www.sec.gov/Archives/edgar/data/1628908/000162890818000015/0001628908-18-000015-index.htm
management discussion and analysis placement: 10-K


-------------------------------------------------------------------------------------
TGE
period of report: 2017-12-31
filing date: 2018-02-13
filing type: 10-K
filing url: https://www.sec.gov/Archives/edgar/data/1633651/000163365118000

-------------------------------------------------------------------------------------
HPR
period of report: 2012-12-31
filing date: N/A
filing type: N/A
filing url: N/A
management discussion and analysis placement: N/A


-------------------------------------------------------------------------------------
HPR
period of report: 2014-12-31
filing date: N/A
filing type: N/A
filing url: N/A
management discussion and analysis placement: N/A


-------------------------------------------------------------------------------------
HPR
period of report: 2015-12-31
filing date: N/A
filing type: N/A
filing url: N/A
management discussion and analysis placement: N/A


-------------------------------------------------------------------------------------
HPR
period of report: 2016-12-31
filing date: N/A
filing type: N/A
filing url: N/A
management discussion and analysis placement: N/A


-------------------------------------------------------------------------------------
HPR
period of report: 2017-12-

## Investigate Observations with N/A Filing Page URLs

In [10]:
# note: to maximize filing data, an investigation and recovery operation is necessary
# note: each 'N/A' observation will be examined in turn

# row count of 'N/A' observations
print('filing_page_df - \'N/A\' row count:', 
      filing_page_df.loc[filing_page_df['Filing_Url']=='N/A'].shape[0])

filing_page_df - 'N/A' row count: 392


In [11]:
# display all 'N/A' observations of filing_page_df
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(filing_page_df.loc[filing_page_df['Filing_Url']=='N/A'])

,CIK,Ticker_Symbol,Company_Name,Data_Date,Filing_Date,Filing_Type,Filing_Url,MDA_Placement
215,98222,TDW,TIDEWATER INC,2017-12-31,N/A,N/A,N/A,N/A
216,98222,TDW,TIDEWATER INC,2018-12-31,N/A,N/A,N/A,N/A
265,311337,SU,SUNCOR ENERGY INC,2011-12-31,N/A,N/A,N/A,N/A
292,701818,KGC,KINROSS GOLD CORP,2011-12-31,N/A,N/A,N/A,N/A
293,701818,KGC,KINROSS GOLD CORP,2012-12-31,N/A,N/A,N/A,N/A
294,701818,KGC,KINROSS GOLD CORP,2013-12-31,N/A,N/A,N/A,N/A
295,701818,KGC,KINROSS GOLD CORP,2014-12-31,N/A,N/A,N/A,N/A
296,701818,KGC,KINROSS GOLD CORP,2015-12-31,N/A,N/A,N/A,N/A
297,701818,KGC,KINROSS GOLD CORP,2016-12-31,N/A,N/A,N/A,N/A
298,701818,KGC,KINROSS GOLD CORP,2017-12-31,N/A,N/A,N/A,N/A


In [12]:
# notes on 'N/A' observations for TDW
'''

Data_Date: 2017-12-31 - report is of type '10-KT'
Dtae_Date: 2018-12-31 - report is of type '10-K'
solution - run edgar_filing_date function and edgar_filing_page function again with '10-KT' and '10-K' parameter 

'''
# run functions again for 'N/A' observations for TDW
# note: 'N/A' observations for TDW are located in index 215 and 216  of filing_page_df
# retrieve the filing date for TDW observatiosn
for idx in range(215, 217):
    filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '10-K'),
                                                           ['10-KT', '10-K'])
    # set the filing type for TDW observations
    if idx==215:
        filing_page_df.loc[idx, 'Filing_Type'] = '10-KT'
    else:
        filing_page_df.loc[idx, 'Filing_Type'] = '10-K'
    # retrieve the filing url for TDW observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '10-K'),
                                                          ['10-KT', '10-K'])
    # set the filing mda placement for TDW observations to Filing_Type value
    filing_page_df.loc[idx, 'MDA_Placement'] = filing_page_df.loc[idx, 'Filing_Type']
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

-------------------------------------------------------------------------------------
TDW
period of report: 2017-12-31
filing date: 2018-03-15
filing type: 10-KT
filing url: https://www.sec.gov/Archives/edgar/data/98222/000156459018005788/0001564590-18-005788-index.htm
management discussion and analysis placement: 10-KT


-------------------------------------------------------------------------------------
TDW
period of report: 2018-12-31
filing date: 2019-02-28
filing type: 10-K
filing url: https://www.sec.gov/Archives/edgar/data/98222/000156459019005545/0001564590-19-005545-index.htm
management discussion and analysis placement: 10-K




In [13]:
# notes on 'N/A' observations for SU
'''

SUNCOR ENERGY INC (SU) is domiciled in Canada; therefore, its annual filings are form 40-F filings and NOT form 10-K filings
Data_Date: 2011-12-31 - report is of type '40-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '40-F' parameter

'''
# run functions again for 'N/A' observation for SU
# note: 'N/A' observation for SU is located in index 265 of filing_page_df
# retrieve the filing date for SU observation
filing_page_df.loc[265, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[265, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[265, 'CIK'], '40-F'),
                                                           ['40-F'])
# set the filing type for SU observation to '40-F'
filing_page_df.loc[265, 'Filing_Type'] = '40-F'
# retrieve the filing url for SU observation
filing_page_df.loc[265, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[265, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[265, 'CIK'], '40-F'),
                                                          ['40-F'])
# set the filing mda placement for SU observation to 'EXHIBIT 99.2' (as the mda section is contained in a separate exhibit)
filing_page_df.loc[265, 'MDA_Placement'] = 'EX-99.2'
# provide user feedback
print('-------------------------------------------------------------------------------------')
print(filing_page_df.loc[265, 'Ticker_Symbol'])
print('period of report:', filing_page_df.loc[265, 'Data_Date'].strftime('%Y-%m-%d'))
print('filing date:', filing_page_df.loc[265, 'Filing_Date'])
print('filing type:', filing_page_df.loc[265, 'Filing_Type'])
print('filing url:', filing_page_df.loc[265, 'Filing_Url'])
print('management discussion and analysis placement:', filing_page_df.loc[265, 'MDA_Placement'])

-------------------------------------------------------------------------------------
SU
period of report: 2011-12-31
filing date: 2012-03-01
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/311337/000104746912001967/0001047469-12-001967-index.htm
management discussion and analysis placement: EX-99.2


In [14]:
# notes on 'N/A' observations for KGC
'''

KINROSS GOLD CORP (KGC) is domiciled in Canada; therefore, its annual filings are form 40-F filings and NOT form 10-K filings
Data_Date: 2011-12-31 - report is of type '40-F'
Data_Date: 2012-12-31 - report is of type '40-F'
Data_Date: 2013-12-31 - report is of type '40-F'
Data_Date: 2014-12-31 - report is of type '40-F'
Data_Date: 2015-12-31 - report is of type '40-F'
Data_Date: 2016-12-31 - report is of type '40-F'
Data_Date: 2017-12-31 - report is of type '40-F'
Data_Date: 2018-12-31 - report is of type '40-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '40-F' parameter

'''
# run functions again for 'N/A' observations for KGC
# note: 'N/A' observations for KGC are located in indexes 292 through to 299 of filing_page_df
for idx in range(292, 300):
    # retrieve the filing date for KGC observations
    filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '40-F'),
                                                           ['40-F'])
    # set the filing type for KGC observations to '40-F'
    filing_page_df.loc[idx, 'Filing_Type'] = '40-F'
    # retrieve the filing url for KGC observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '40-F'),
                                                          ['40-F'])
    # set the filing mda placement for KGC observations to 'EX-99.2' (as the mda section is contained in a separate exhibit)
    filing_page_df.loc[idx, 'MDA_Placement'] = 'EX-99.2'
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

-------------------------------------------------------------------------------------
KGC
period of report: 2011-12-31
filing date: 2012-03-30
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/701818/000104746912003703/0001047469-12-003703-index.htm
management discussion and analysis placement: EX-99.2


-------------------------------------------------------------------------------------
KGC
period of report: 2012-12-31
filing date: 2013-04-01
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/701818/000110465913026203/0001104659-13-026203-index.htm
management discussion and analysis placement: EX-99.2


-------------------------------------------------------------------------------------
KGC
period of report: 2013-12-31
filing date: 2014-03-31
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/701818/000110465914024565/0001104659-14-024565-index.htm
management discussion and analysis placement: EX-99.2


----------------------

In [15]:
%%capture
# notes on 'N/A' observations for MGA
'''

MAGNA INTERNATIONAL INC is domiciled in Canada; therefore, its annual filings are form 40-F filings and NOT form 10-K filings
Data_Date: 2011-12-31 - report is of type '40-F'
note that the management discussion and analysis placement is not contained in filing under separate exhibit
solution - UNSOLVED

'''

In [16]:
# notes on 'N/A' observations for GOLD
'''

BARRICK GOLD CORP is domiciled in Canada; therefore, its annual filings are form 40-F filings and not form 10-K filings
Data_Date: 2014-12-31 - report is of type '40-F'
Data_Date: 2015-12-31 - report is of type '40-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '40-F' parameter

'''
# run functions again for 'N/A' observations for GOLD
# note: 'N/A' observations for GOLD are located in indexes 347 and 348 of filing_page_df
for idx in range(347, 349):
    # retrieve the filing date for GOLD observations
    filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '40-F'),
                                                           ['40-F'])
    # set the filing type for GOLD observations to '40-F'
    filing_page_df.loc[idx, 'Filing_Type'] = '40-F'
    # retrieve the filing url for GOLD observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '40-F'),
                                                          ['40-F'])
    # set the filing mda placement for GOLD observations to 'EX-99.4' (as the mda section is contained in a separate exhibit)
    filing_page_df.loc[idx, 'MDA_Placement'] = 'EX-99.4'
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

-------------------------------------------------------------------------------------
GOLD
period of report: 2014-12-31
filing date: 2015-03-30
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/756894/000119312515109743/0001193125-15-109743-index.htm
management discussion and analysis placement: EX-99.4


-------------------------------------------------------------------------------------
GOLD
period of report: 2015-12-31
filing date: 2016-03-28
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/756894/000119312516518731/0001193125-16-518731-index.htm
management discussion and analysis placement: EX-99.4




In [17]:
# notes on 'N/A' observations for TECK
'''

TECK RESOURCES LTD is domiciled in Canada; therefore, its annual filings are form 40-F filings and not form 10-K filings
Data_Date: 2011-12-31 - report is of type '40-F'
Data_Date: 2012-12-31 - report is of type '40-F'
Data_Date: 2013-12-31 - report is of type '40-F'
Data_Date: 2014-12-31 - report is of type '40-F'
Data_Date: 2015-12-31 - report is of type '40-F'
Data_Date: 2016-12-31 - report is of type '40-F'
Data_Date: 2017-12-31 - report is of type '40-F'
Data_Date: 2018-12-31 - report is of type '40-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '40-F' parameter

'''
# run functions again for 'N/A' observations for TECK
# note: 'N/A' observations for TECK are located in indexes 463 through to 470 of filing_page_df
for idx in range(463, 471):
    # retrieve the filing date for TECK observations
    filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '40-F'),
                                                           ['40-F'])
    # set the filing type for TECK observations to '40-F'
    filing_page_df.loc[idx, 'Filing_Type'] = '40-F'
    # retrieve the filing url for TECK observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '40-F'),
                                                          ['40-F'])
    # set the filing mda placement for TECK observations to 'EX-99.3' (as the mda section is contained in a separate exhibit)
    filing_page_df.loc[idx, 'MDA_Placement'] = 'EX-99.3'
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

-------------------------------------------------------------------------------------
TECK
period of report: 2011-12-31
filing date: 2012-03-14
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/886986/000095014212000605/0000950142-12-000605-index.htm
management discussion and analysis placement: EX-99.3


-------------------------------------------------------------------------------------
TECK
period of report: 2012-12-31
filing date: 2013-03-11
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/886986/000095014213000574/0000950142-13-000574-index.htm
management discussion and analysis placement: EX-99.3


-------------------------------------------------------------------------------------
TECK
period of report: 2013-12-31
filing date: 2014-03-11
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/886986/000095014214000616/0000950142-14-000616-index.htm
management discussion and analysis placement: EX-99.3


-------------------

In [18]:
# notes on 'N/A' observation for ENB
'''

ENBRIDGE INC is domiciled in Canada; therefore, its annual filings are form 40-F filings and not form 10-K filings
ENBRIDGE INC merged with SPECTRA ENERGY CORP (which is domiciled in the US) in September 2016; therefore, ENBRIDGE INC was required to issue form 10-K filings from 2018 onwards
Data_Date: 2017-12-31 - report is of type '10-K'
solution - add filing date to filing_page_df manually for ENB observation (as edgar_search function can only accept one filing type), and run edgar_filing_page function again with '10-K' parameter

'''
# run functions again for 'N/A' observation for ENB
# note: 'N/A' observation for ENB is located in index 486 of filing_page_df
# retrieve the filing date for ENB observation
filing_page_df.loc[486, 'Filing_Date'] = '2018-02-16'
# set the filing type for ENB observation to '10-K'
filing_page_df.loc[486, 'Filing_Type'] = '10-K'
# retrieve the filing url for ENB observation
filing_page_df.loc[486, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[486, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[486, 'CIK'], '10-K'),
                                                          ['10-K'])
# set the filing mda placement for ENB observations to Filing_Type value
filing_page_df.loc[486, 'MDA_Placement'] = filing_page_df.loc[486, 'Filing_Type']
# provide user feedback
print('-------------------------------------------------------------------------------------')
print(filing_page_df.loc[486, 'Ticker_Symbol'])
print('period of report:', filing_page_df.loc[486, 'Data_Date'].strftime('%Y-%m-%d'))
print('filing date:', filing_page_df.loc[486, 'Filing_Date'])
print('filing type:', filing_page_df.loc[486, 'Filing_Type'])
print('filing url:', filing_page_df.loc[486, 'Filing_Url'])
print('management discussion and analysis placement:', filing_page_df.loc[486, 'MDA_Placement'])

-------------------------------------------------------------------------------------
ENB
period of report: 2017-12-31
filing date: 2018-02-16
filing type: 10-K
filing url: https://www.sec.gov/Archives/edgar/data/895728/000089572818000008/0000895728-18-000008-index.htm
management discussion and analysis placement: 10-K


In [19]:
# notes on 'N/A' observations for TK
'''

TEEKAY CORP is classed as a 'foreign private issuer'; therefore, its annual filings are form 20-F filings and form 10-K filings
Data_Date: 2011-12-31 - report is of type '20-F'
Data_Date: 2012-12-31 - report is of type '20-F'
Data_Date: 2013-12-31 - report is of type '20-F'
Data_Date: 2014-12-31 - report is of type '20-F'
Data_Date: 2015-12-31 - report is of type '20-F'
Data_Date: 2016-12-31 - report is of type '20-F'
Data_Date: 2017-12-31 - report is of type '20-F'
Data_Date: 2018-12-31 - report is of type '20-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '20-F' parameter

'''
# run functions again for 'N/A' observations for TK
# note: 'N/A' observations for TK are located in indexes 495 through to 502 of filing_page_df
for idx in range(495, 503):
    # retrieve the filing date for TK observations
    filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                           ['20-F'])
    # set the filing type for TK observations to '20-F'
    filing_page_df.loc[idx, 'Filing_Type'] = '20-F'
    # retrieve the filing url for TK observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                          ['20-F'])
    # set the filing mda placement for TK observations to Filing_Type value
    filing_page_df.loc[idx, 'MDA_Placement'] = filing_page_df.loc[idx, 'Filing_Type']
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

-------------------------------------------------------------------------------------
TK
period of report: 2011-12-31
filing date: 2012-04-25
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/911971/000119312512182144/0001193125-12-182144-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
TK
period of report: 2012-12-31
filing date: 2013-04-29
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/911971/000119312513181831/0001193125-13-181831-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
TK
period of report: 2013-12-31
filing date: 2014-04-28
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/911971/000119312514163511/0001193125-14-163511-index.htm
management discussion and analysis placement: 20-F


----------------------------------

In [20]:
# notes on 'N/A' observations for FRO
'''

FRONTLINE LTD is classed as a 'foreign private issuer'; therefore, its annual filings are form 20-F filings and form 10-K filings
Data_Date: 2017-12-31 - report is of type '20-F'
Data_Date: 2018-12-31 - report is of type '20-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '20-F' parameter

'''
# run functions again for 'N/A' observations for FRO
# note: 'N/A' observations for FRO are located in indexes 505 and 506 of filing_page_df
for idx in range(505, 507):
    # retrieve the filing date for FRO observations
    filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                           ['20-F'])
    # set the filing type for FRO observations to '20-F'
    filing_page_df.loc[idx, 'Filing_Type'] = '20-F'
    # retrieve the filing url for FRO observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                          ['20-F'])
    # set the filing mda placement for FRO observations to Filing_Type value
    filing_page_df.loc[idx, 'MDA_Placement'] = filing_page_df.loc[idx, 'Filing_Type']
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

-------------------------------------------------------------------------------------
FRO
period of report: 2017-12-31
filing date: 2018-03-19
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/913290/000091329018000003/0000913290-18-000003-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
FRO
period of report: 2018-12-31
filing date: 2019-03-28
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/913290/000091329019000004/0000913290-19-000004-index.htm
management discussion and analysis placement: 20-F




In [21]:
# notes on 'N/A' observations for EGO
'''

ELDORADO GOLD CORP is domiciled in Canada; therefore, its annual filings are form 40-F filings and not form 10-K filings
Data_Date: 2013-12-31 - report is of type '40-F'
Data_Date: 2014-12-31 - report is of type '40-F'
Data_Date: 2015-12-31 - report is of type '40-F'
Data_Date: 2016-12-31 - report is of type '40-F'
Data_Date: 2017-12-31 - report is of type '40-F'
Data_Date: 2018-12-31 - report is of type '40-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '40-F' parameter

'''
# run functions again for 'N/A' observations for EGO
# note: 'N/A' observations for EGO are located in indexes 509 through to 514 of filing_page_df
for idx in range(509, 515):
    # retrieve the filing date for EGO observations
    filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '40-F'),
                                                           ['40-F'])
    # set the filing type for EGO observations to '40-F'
    filing_page_df.loc[idx, 'Filing_Type'] = '40-F'
    # retrieve the filing url for EGO observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '40-F'),
                                                          ['40-F'])
    # set the filing mda placement for EGO observations to 'EX-99.3' (as the mda section is contained in a separate exhibit)
    filing_page_df.loc[idx, 'MDA_Placement'] = 'EX-99.3'
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

-------------------------------------------------------------------------------------
EGO
period of report: 2013-12-31
filing date: 2014-03-28
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/918608/000113717114000057/0001137171-14-000057-index.htm
management discussion and analysis placement: EX-99.3


-------------------------------------------------------------------------------------
EGO
period of report: 2014-12-31
filing date: 2015-03-27
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/918608/000091228215000109/0000912282-15-000109-index.htm
management discussion and analysis placement: EX-99.3


-------------------------------------------------------------------------------------
EGO
period of report: 2015-12-31
filing date: 2016-03-30
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/918608/000091228216000667/0000912282-16-000667-index.htm
management discussion and analysis placement: EX-99.3


----------------------

In [22]:
# notes on 'N/A' observations for CYD
'''

CHINA YUCHAI INTERNATIONAL is classed as a 'foreign private issuer'; therefore, its annual filings are form 20-F filings and form 10-K filings
Data_Date: 2011-12-31 - report is of type '20-F'
Data_Date: 2012-12-31 - report is of type '20-F'
Data_Date: 2013-12-31 - report is of type '20-F'
Data_Date: 2014-12-31 - report is of type '20-F'
Data_Date: 2015-12-31 - report is of type '20-F'
Data_Date: 2016-12-31 - report is of type '20-F'
Data_Date: 2017-12-31 - report is of type '20-F'
Data_Date: 2018-12-31 - report is of type '20-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '20-F' parameter

'''
# run functions again for 'N/A' observations for CYD
# note: 'N/A' observations for CYD are located in indexes 530 through to 537 of filing_page_df
for idx in range(530, 538):
    # retrieve the filing date for CYD observations
    filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                           ['20-F'])
    # set the filing type for CYD observations to '20-F'
    filing_page_df.loc[idx, 'Filing_Type'] = '20-F'
    # retrieve the filing url for CYD observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                          ['20-F'])
    # set the filing mda placement for CYD observations to Filing_Type value
    filing_page_df.loc[idx, 'MDA_Placement'] = filing_page_df.loc[idx, 'Filing_Type']
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

-------------------------------------------------------------------------------------
CYD
period of report: 2011-12-31
filing date: 2012-04-19
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/932695/000119312512169443/0001193125-12-169443-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
CYD
period of report: 2012-12-31
filing date: 2013-04-22
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/932695/000119312513163830/0001193125-13-163830-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
CYD
period of report: 2013-12-31
filing date: 2014-04-22
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/932695/000119312514151627/0001193125-14-151627-index.htm
management discussion and analysis placement: 20-F


-------------------------------

In [23]:
# notes on 'N/A' observations for NAT
'''

NORDIC AMERICAN TANKERS LTD is classed as a 'foreign private issuer'; therefore, its annual filings are form 20-F filings and form 10-K filings
Data_Date: 2011-12-31 - report is of type '20-F'
Data_Date: 2012-12-31 - report is of type '20-F'
Data_Date: 2013-12-31 - report is of type '20-F'
Data_Date: 2014-12-31 - report is of type '20-F'
Data_Date: 2016-12-31 - report is of type '20-F'
Data_Date: 2017-12-31 - report is of type '20-F'
Data_Date: 2018-12-31 - report is of type '20-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '20-F' parameter

'''
# run functions again for 'N/A' observations for NAT
# note: 'N/A' observations for NAT are located in indexes 573 through to 579 of filing_page_df
for idx in range(573, 580):
    # retrieve the filing date for NAT observations
    filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                           ['20-F'])
    # set the filing type for NAT observations to '20-F'
    filing_page_df.loc[idx, 'Filing_Type'] = '20-F'
    # retrieve the filing url for NAT observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                          ['20-F'])
    # set the filing mda placement for NAT observations to Filing_Type value
    filing_page_df.loc[idx, 'MDA_Placement'] = filing_page_df.loc[idx, 'Filing_Type']
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

-------------------------------------------------------------------------------------
NAT
period of report: 2011-12-31
filing date: 2012-04-17
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1000177/000091957412002820/0000919574-12-002820-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
NAT
period of report: 2012-12-31
filing date: 2013-03-19
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1000177/000119312513113445/0001193125-13-113445-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
NAT
period of report: 2013-12-31
filing date: 2014-04-04
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1000177/000091957414002516/0000919574-14-002516-index.htm
management discussion and analysis placement: 20-F


----------------------------

In [24]:
# notes on 'N/A' observations for CCJ
'''

CAMECO CORP is domiciled in Canada; therefore, its annual filings are form 40-F filings and not form 10-K filings
Data_Date: 2015-12-31 - report is of type '40-F'
Data_Date: 2016-12-31 - report is of type '40-F'
Data_Date: 2017-12-31 - report is of type '40-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '40-F' parameter

'''
# run functions again for 'N/A' observations for CCJ
# note: 'N/A' observations for CCJ are located in indexes 589 through to 591 of filing_page_df
for idx in range(589, 592):
    # retrieve the filing date for CCJ observations
    filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '40-F'),
                                                           ['40-F'])
    # set the filing type for CCJ observations to '40-F'
    filing_page_df.loc[idx, 'Filing_Type'] = '40-F'
    # retrieve the filing url for CCJ observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '40-F'),
                                                          ['40-F'])
    # set the filing mda placement for CCJ observations to 'EX-99.3' (as the mda section is contained in a separate exhibit)
    filing_page_df.loc[idx, 'MDA_Placement'] = 'EX-99.3'
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

-------------------------------------------------------------------------------------
CCJ
period of report: 2015-12-31
filing date: 2016-03-30
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/1009001/000119312516521653/0001193125-16-521653-index.htm
management discussion and analysis placement: EX-99.3


-------------------------------------------------------------------------------------
CCJ
period of report: 2016-12-31
filing date: 2017-03-23
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/1009001/000119312517093880/0001193125-17-093880-index.htm
management discussion and analysis placement: EX-99.3


-------------------------------------------------------------------------------------
CCJ
period of report: 2017-12-31
filing date: 2018-03-28
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/1009001/000119312518099445/0001193125-18-099445-index.htm
management discussion and analysis placement: EX-99.3




In [25]:
# notes on 'N/A' observations for PDS
'''

PRECISION DRILLING CORP is domiciled in Canada; therefore, its annual filings are form 40-F filings and not form 10-K filings
Data_Date: 2012-12-31 - report is of type '40-F'
Data_Date: 2013-12-31 - report is of type '40-F'
Data_Date: 2014-12-31 - report is of type '40-F'
Data_Date: 2015-12-31 - report is of type '40-F'
Data_Date: 2016-12-31 - report is of type '40-F'
Data_Date: 2017-12-31 - report is of type '40-F'
Data_Date: 2018-12-31 - report is of type '40-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '40-F' parameter

'''
# run functions again for 'N/A' observations for PDS
# note: 'N/A' observations for PDS are located in indexes 599 through to 605 of filing_page_df
for idx in range(599, 606):
    # retrieve the filing date for PDS observations
    filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '40-F'),
                                                           ['40-F'])
    # set the filing type for PDS observations to '40-F'
    filing_page_df.loc[idx, 'Filing_Type'] = '40-F'
    # retrieve the filing url for PDS observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '40-F'),
                                                          ['40-F'])
    # set the filing mda placement for PDS observations to 'EX-99.2' (as the mda section is contained in a separate exhibit)
    filing_page_df.loc[idx, 'MDA_Placement'] = 'EX-99.2'
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

-------------------------------------------------------------------------------------
PDS
period of report: 2012-12-31
filing date: 2013-03-19
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/1013605/000119312513114892/0001193125-13-114892-index.htm
management discussion and analysis placement: EX-99.2


-------------------------------------------------------------------------------------
PDS
period of report: 2013-12-31
filing date: 2014-03-14
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/1013605/000119312514100267/0001193125-14-100267-index.htm
management discussion and analysis placement: EX-99.2


-------------------------------------------------------------------------------------
PDS
period of report: 2014-12-31
filing date: 2015-03-13
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/1013605/000119312515090604/0001193125-15-090604-index.htm
management discussion and analysis placement: EX-99.2


-------------------

In [26]:
# notes on 'N/A' observation for CNQ
'''

CANADIAN NATURAL RESOURCES is domiciled in Canada; therefore, its annual filings are form 40-F filings and not form 10-K filings
Data_Date: 2015-12-31 - report is of type '40-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '40-F' parameter

'''
# run functions again for 'N/A' observation for CNQ
# note: 'N/A' observation for CNQ is located in index 613 of filing_page_df
# retrieve the filing date for CNQ observation
filing_page_df.loc[613, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[613, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[613, 'CIK'], '40-F'),
                                                           ['40-F'])
# set the filing type for CNQ observation to '40-F'
filing_page_df.loc[613, 'Filing_Type'] = '40-F'
# retrieve the filing url for CNQ observation
filing_page_df.loc[613, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[613, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[613, 'CIK'], '40-F'),
                                                          ['40-F'])
# set the filing mda placement for CNQ observations to Filing_Type value
filing_page_df.loc[613, 'MDA_Placement'] = filing_page_df.loc[613, 'Filing_Type']
# provide user feedback
print('-------------------------------------------------------------------------------------')
print(filing_page_df.loc[613, 'Ticker_Symbol'])
print('period of report:', filing_page_df.loc[613, 'Data_Date'].strftime('%Y-%m-%d'))
print('filing date:', filing_page_df.loc[613, 'Filing_Date'])
print('filing type:', filing_page_df.loc[613, 'Filing_Type'])
print('filing url:', filing_page_df.loc[613, 'Filing_Url'])
print('management discussion and analysis placement:', filing_page_df.loc[613, 'MDA_Placement'])

-------------------------------------------------------------------------------------
CNQ
period of report: 2015-12-31
filing date: 2016-03-24
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/1017413/000095014216003370/0000950142-16-003370-index.htm
management discussion and analysis placement: 40-F


In [27]:
# notes on 'N/A' observations for CEA
'''

CHINA EASTERN AIRLINES CORP is classed as a 'foreign private issuer'; therefore, its annual filings are form 20-F filings and form 10-K filings
Data_Date: 2011-12-31 - report is of type '20-F'
Data_Date: 2012-12-31 - report is of type '20-F'
Data_Date: 2013-12-31 - report is of type '20-F'
Data_Date: 2014-12-31 - report is of type '20-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '20-F' parameter

'''
# run functions again for 'N/A' observations for CEA
# note: 'N/A' observations for CEA are located in indexes 634 through to 637 of filing_page_df
for idx in range(634, 638):
    # retrieve the filing date for CEA observations
    filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                           ['20-F'])
    # set the filing type for CEA observations to '20-F'
    filing_page_df.loc[idx, 'Filing_Type'] = '20-F'
    # retrieve the filing url for CEA observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                          ['20-F'])
    # set the filing mda placement for CEA observations to Filing_Type value
    filing_page_df.loc[idx, 'MDA_Placement'] = filing_page_df.loc[idx, 'Filing_Type']
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

-------------------------------------------------------------------------------------
CEA
period of report: 2011-12-31
filing date: 2012-04-24
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1030475/000114420412023290/0001144204-12-023290-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
CEA
period of report: 2012-12-31
filing date: 2013-04-24
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1030475/000114420413023527/0001144204-13-023527-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
CEA
period of report: 2013-12-31
filing date: 2014-04-25
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1030475/000114420414024778/0001144204-14-024778-index.htm
management discussion and analysis placement: 20-F


----------------------------

In [28]:
# notes on 'N/A' observations for CLS
'''

CELESTICA INC is classed as a 'foreign private issuer'; therefore, its annual filings are form 20-F filings and form 10-K filings
Data_Date: 2011-12-31 - report is of type '20-F'
Data_Date: 2012-12-31 - report is of type '20-F'
Data_Date: 2017-12-31 - report is of type '20-F'
Data_Date: 2018-12-31 - report is of type '20-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '20-F' parameter

'''
# run functions again for 'N/A' observations for CLS
# note: 'N/A' observations for CLS are located in indexes 638 through to 641 of filing_page_df
for idx in range(638, 642):
    # retrieve the filing date for CLS observations
    filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                           ['20-F'])
    # set the filing type for CLS observations to '20-F'
    filing_page_df.loc[idx, 'Filing_Type'] = '20-F'
    # retrieve the filing url for CLS observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                          ['20-F'])
    # set the filing mda placement for CLS observations to Filing_Type value
    filing_page_df.loc[idx, 'MDA_Placement'] = filing_page_df.loc[idx, 'Filing_Type']
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

-------------------------------------------------------------------------------------
CLS
period of report: 2011-12-31
filing date: 2012-03-22
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1030894/000104746912003092/0001047469-12-003092-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
CLS
period of report: 2012-12-31
filing date: 2013-03-15
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1030894/000104746913002816/0001047469-13-002816-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
CLS
period of report: 2017-12-31
filing date: 2018-03-12
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1030894/000162828018003056/0001628280-18-003056-index.htm
management discussion and analysis placement: 20-F


----------------------------

In [29]:
# notes on 'N/A' observations for BB
'''

BLACKBERRY LTD is domiciled in Canada; therefore, its annual filings are form 40-F filings and not form 10-K filings
Data_Date: 2012-02-29 - report is of type '40-F'
Data_Date: 2013-02-29 - report is of type '40-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '40-F' parameter

'''
# run functions again for 'N/A' observations for BB
# note: 'N/A' observations for BB are located in indexes 698 and 699 of filing_page_df
for idx in range(698, 700):
    # retrieve the filing date for BB observations
    filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '40-F'),
                                                           ['40-F'])
    # set the filing type for BB observations to '40-F'
    filing_page_df.loc[idx, 'Filing_Type'] = '40-F'
    # retrieve the filing url for BB observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '40-F'),
                                                          ['40-F'])
    # set the filing mda placement for BB observations to 'EX-1.3' (as the mda section is contained in a separate exhibit)
    filing_page_df.loc[idx, 'MDA_Placement'] = 'EX-1.3'
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

-------------------------------------------------------------------------------------
BB
period of report: 2012-02-29
filing date: 2012-04-09
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/1070235/000119312512155342/0001193125-12-155342-index.htm
management discussion and analysis placement: EX-1.3


-------------------------------------------------------------------------------------
BB
period of report: 2013-02-28
filing date: 2013-03-28
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/1070235/000119312513132586/0001193125-13-132586-index.htm
management discussion and analysis placement: EX-1.3




In [30]:
# notes on 'N/A' observations for AGI
'''

cik number corresponds to AURICO GOLD INC, not to ALAMOS GOLD INC
AURICO GOLD INC is domiciled in Canada; therefore, its annual filings are form 40-F filings and not form 10-K filings
AURICO GOLD INC merged with ALAMOS GOLD INC (which is domiciled in Canada too) in April 2015, therefore, all filings post April 2015 correspond to ALAMOS GOLD INC
Data_Date: 2011-12-31 - report is of type '40-F'; corresponds to AURICO GOLD INC
Data_Date: 2012-12-31 - report is of type '40-F'; corresponds to AURICO GOLD INC
Data_Date: 2013-12-31 - report is of type '40-F'; corresponds to AURICO GOLD INC
Data_Date: 2014-12-31 - report is of type '40-F'; corresponds to AURICO GOLD INC
Data_Date: 2015-12-31 - report is of type '40-F'; corresponds to ALAMOS GOLD INC
Data_Date: 2016-12-31 - report is of type '40-F'; corresponds to ALAMOS GOLD INC
Data_Date: 2017-12-31 - report is of type '40-F'; corresponds to ALAMOS GOLD INC
Data_Date: 2018-12-31 - report is of type '40-F'; corresponds to ALAMOS GOLD INC
solution - change necessary information to correspond to each company and run edgar_filing_date function and edgar_filing_page function again with '40-F' parameter

'''
# change Ticker_Symbol for indexes 706 through to 709 to AUQ (AURICO GOLD INC) in filing_page_df
# change Company_Name for indexes 706 through to 709 to AURICO GOLD INC in filing_page_df
# run functions again for 'N/A' observations for AUQ
for idx in range(706, 710):
    filing_page_df.loc[idx, 'Ticker_Symbol'] = 'AUQ'
    filing_page_df.loc[idx, 'Company_Name'] = 'AURICO GOLD INC'
    # retrieve the filing date for AUQ observations
    filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '40-F'),
                                                           ['40-F'])
    # set the filing type for AUQ observations to '40-F'
    filing_page_df.loc[idx, 'Filing_Type'] = '40-F'
    # retrieve the filing url for AUQ observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '40-F'),
                                                          ['40-F'])
    # set the filing mda placement for AUQ observations to 'EX-99.2' (as the mda section is contained in a separate exhibit)
    filing_page_df.loc[idx, 'MDA_Placement'] = 'EX-99.2'
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

# change CIK for indexes 710 through to 713 to 1178819 (ALAMOS GOLD INC) in filing_page_df
# run functions again for 'N/A' observations for AGI
for idx in range(710, 714):
    filing_page_df.loc[idx, 'CIK'] = 1178819
    # retrieve the filing date for AGI observations
    filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '40-F'),
                                                           ['40-F'])
    # set the filing type for AGI observations to '40-F'
    filing_page_df.loc[idx, 'Filing_Type'] = '40-F'
    # retrieve the filing url for AGI observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '40-F'),
                                                          ['40-F'])
    # set the filing mda placement for AGI observations to 'EX-99.2' (as the mda section is contained in a separate exhibit)
    filing_page_df.loc[idx, 'MDA_Placement'] = 'EX-99.2'
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

-------------------------------------------------------------------------------------
AUQ
period of report: 2011-12-31
filing date: 2012-03-29
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/1078217/000120445912000634/0001204459-12-000634-index.htm
management discussion and analysis placement: EX-99.2


-------------------------------------------------------------------------------------
AUQ
period of report: 2012-12-31
filing date: 2013-04-01
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/1078217/000106299313001596/0001062993-13-001596-index.htm
management discussion and analysis placement: EX-99.2


-------------------------------------------------------------------------------------
AUQ
period of report: 2013-12-31
filing date: 2014-03-04
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/1078217/000106299314001093/0001062993-14-001093-index.htm
management discussion and analysis placement: EX-99.2


-------------------

In [31]:
# notes on 'N/A' observation for MAXR
'''

MAXAR TECHNOLOGIES INC is domiciled in the US; filing for Data_Date: 2018-12-31 was not found as no filings exist for year 2018
Data_Date: 2018-12-31 - report is of type '10-K'
solution - add filing date to filing_page_df manually for MAXR observation, and run edgar_filing_page function again with '10-K' parameter

'''
# run functions again for 'N/A' observation for MAXR
# note: 'N/A' observation for MAXR is located in index 752 of filing_page_df
# retrieve the filing date for MAXR observation
filing_page_df.loc[752, 'Filing_Date'] = '2019-03-01'
# set the filing type for MAXR observation to '10-K'
filing_page_df.loc[752, 'Filing_Type'] = '10-K'
# retrieve the filing url for MAXR observation
filing_page_df.loc[752, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[752, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[752, 'CIK'], '10-K'),
                                                          ['10-K'])
# set the filing mda placement for MAXR observations to Filing_Type value
filing_page_df.loc[752, 'MDA_Placement'] = filing_page_df.loc[752, 'Filing_Type']
# provide user feedback
print('-------------------------------------------------------------------------------------')
print(filing_page_df.loc[752, 'Ticker_Symbol'])
print('period of report:', filing_page_df.loc[752, 'Data_Date'].strftime('%Y-%m-%d'))
print('filing date:', filing_page_df.loc[752, 'Filing_Date'])
print('filing type:', filing_page_df.loc[752, 'Filing_Type'])
print('filing url:', filing_page_df.loc[752, 'Filing_Url'])
print('management discussion and analysis placement:', filing_page_df.loc[752, 'MDA_Placement'])

-------------------------------------------------------------------------------------
MAXR
period of report: 2018-12-31
filing date: 2019-03-01
filing type: 10-K
filing url: https://www.sec.gov/Archives/edgar/data/1121142/000155837019001372/0001558370-19-001372-index.htm
management discussion and analysis placement: 10-K


In [32]:
# notes on 'N/A' observations for ERF
'''

ENERPLUS CORP is domiciled in Canada; therefore, its annual filings are form 40-F filings and not form 10-K filings
Data_Date: 2012-12-31 - report is of type '40-F'
Data_Date: 2014-12-31 - report is of type '40-F'
Data_Date: 2015-12-31 - report is of type '40-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '40-F' parameter

'''
# run functions again for 'N/A' observations for ERF
# note: 'N/A' observations for ERF are located in indexes 755 through to 757 of filing_page_df
for idx in range(755, 758):
    # retrieve the filing date for ERF observations
    filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '40-F'),
                                                           ['40-F'])
    # set the filing type for ERF observations to '40-F'
    filing_page_df.loc[idx, 'Filing_Type'] = '40-F'
    # retrieve the filing url for ERF observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '40-F'),
                                                          ['40-F'])
    # set the filing mda placement for ERF observations to 'EX-99.3' (as the mda section is contained in a separate exhibit)
    filing_page_df.loc[idx, 'MDA_Placement'] = 'EX-99.3'
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

-------------------------------------------------------------------------------------
ERF
period of report: 2012-12-31
filing date: 2013-02-22
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/1126874/000104746913001431/0001047469-13-001431-index.htm
management discussion and analysis placement: EX-99.3


-------------------------------------------------------------------------------------
ERF
period of report: 2014-12-31
filing date: 2015-02-20
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/1126874/000104746915000977/0001047469-15-000977-index.htm
management discussion and analysis placement: EX-99.3


-------------------------------------------------------------------------------------
ERF
period of report: 2015-12-31
filing date: 2016-02-19
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/1126874/000104746916010227/0001047469-16-010227-index.htm
management discussion and analysis placement: EX-99.3




In [33]:
# notes on 'N/A' observation for CEQP
'''

CRESTWOOD EQUITY PARTNERS LP is domiciled in the US; filing for Data_Date: 2013-12-31 was not found as no filings exist for year 2013
Data_Date: 2013-12-31 - report is of type '10-K'
solution - add filing date to filing_page_df manually for CEQP observation, and run edgar_filing_page function again with '10-K' parameter

'''
# run functions again for 'N/A' observation for CEQP
# note: 'N/A' observation for CEQP is located in index 766 of filing_page_df
# retrieve the filing date for CEQP observation
filing_page_df.loc[766, 'Filing_Date'] = '2014-02-28'
# set the filing type for CEQP observation to '10-K'
filing_page_df.loc[766, 'Filing_Type'] = '10-K'
# retrieve the filing url for CEQP observation
filing_page_df.loc[766, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[766, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[766, 'CIK'], '10-K'),
                                                          ['10-K'])
# set the filing mda placement for CEQP observations to Filing_Type value
filing_page_df.loc[766, 'MDA_Placement'] = filing_page_df.loc[766, 'Filing_Type']
# provide user feedback
print('-------------------------------------------------------------------------------------')
print(filing_page_df.loc[766, 'Ticker_Symbol'])
print('period of report:', filing_page_df.loc[766, 'Data_Date'].strftime('%Y-%m-%d'))
print('filing date:', filing_page_df.loc[766, 'Filing_Date'])
print('filing type:', filing_page_df.loc[766, 'Filing_Type'])
print('filing url:', filing_page_df.loc[766, 'Filing_Url'])
print('management discussion and analysis placement:', filing_page_df.loc[766, 'MDA_Placement'])

-------------------------------------------------------------------------------------
CEQP
period of report: 2013-12-31
filing date: 2014-02-28
filing type: 10-K
filing url: https://www.sec.gov/Archives/edgar/data/1136352/000144530514000767/0001445305-14-000767-index.htm
management discussion and analysis placement: 10-K


In [34]:
# notes on 'N/A' observations for TAC
'''

TRANSALTA CORP is domiciled in Canada; therefore, its annual filings are form 40-F filings and not form 10-K filings
Data_Date: 2012-12-31 - report is of type '40-F'
Data_Date: 2013-12-31 - report is of type '40-F'
Data_Date: 2014-12-31 - report is of type '40-F'
Data_Date: 2015-12-31 - report is of type '40-F'
Data_Date: 2016-12-31 - report is of type '40-F'
Data_Date: 2017-12-31 - report is of type '40-F'
Data_Date: 2018-12-31 - report is of type '40-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '40-F' parameter

'''
# run functions again for 'N/A' observations for TAC
# note: 'N/A' observations for TAC are located in indexes 778 through to 784 of filing_page_df
for idx in range(778, 785):
    # retrieve the filing date for TAC observations
    filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '40-F'),
                                                           ['40-F'])
    # set the filing type for TAC observations to '40-F'
    filing_page_df.loc[idx, 'Filing_Type'] = '40-F'
    # retrieve the filing url for TAC observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '40-F'),
                                                          ['40-F'])
    # set the filing mda placement for TAC observations to 'EX-13.2' (as the mda section is contained in a separate exhibit)
    filing_page_df.loc[idx, 'MDA_Placement'] = 'EX-13.2'
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

-------------------------------------------------------------------------------------
TAC
period of report: 2012-12-31
filing date: 2013-02-27
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/1144800/000110465913014735/0001104659-13-014735-index.htm
management discussion and analysis placement: EX-13.2


-------------------------------------------------------------------------------------
TAC
period of report: 2013-12-31
filing date: 2014-02-20
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/1144800/000110465914011632/0001104659-14-011632-index.htm
management discussion and analysis placement: EX-13.2


-------------------------------------------------------------------------------------
TAC
period of report: 2014-12-31
filing date: 2015-02-19
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/1144800/000110465915011941/0001104659-15-011941-index.htm
management discussion and analysis placement: EX-13.2


-------------------

In [35]:
# notes on 'N/A' observations for ECA
'''

ENCANA CORP is domiciled in Canada; therefore, its annual filings are form 40-F filings and not form 10-K filings (pre-2016)
Data_Date: 2011-12-31 - report is of type '40-F'
Data_Date: 2014-12-31 - report is of type '40-F'
Data_Date: 2015-12-31 - report is of type '40-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '40-F' parameter

'''
# run functions again for 'N/A' observations for ECA
# note: 'N/A' observations for ECA are located in indexes 791 through to 793 of filing_page_df
for idx in range(791, 794):
    # retrieve the filing date for ECA observations
    filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '40-F'),
                                                           ['40-F'])
    # set the filing type for ECA observations to '40-F'
    filing_page_df.loc[idx, 'Filing_Type'] = '40-F'
    # retrieve the filing url for ECA observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '40-F'),
                                                          ['40-F'])
    # set the filing mda placement for ECA observations to Filing_Type value
    filing_page_df.loc[idx, 'MDA_Placement'] = filing_page_df.loc[idx, 'Filing_Type']
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

-------------------------------------------------------------------------------------
ECA
period of report: 2011-12-31
filing date: 2012-02-23
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/1157806/000110465912012005/0001104659-12-012005-index.htm
management discussion and analysis placement: 40-F


-------------------------------------------------------------------------------------
ECA
period of report: 2014-12-31
filing date: 2015-03-03
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/1157806/000110465915016292/0001104659-15-016292-index.htm
management discussion and analysis placement: 40-F


-------------------------------------------------------------------------------------
ECA
period of report: 2015-12-31
filing date: 2016-02-29
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/1157806/000110465916100966/0001104659-16-100966-index.htm
management discussion and analysis placement: 40-F




In [36]:
# notes on 'N/A' observations for TRQ
'''

TURQUOISE HILL RESOURCES LTD is domiciled in Canada; therefore, its annual filings are form 40-F filings and not form 10-K filings
Data_Date: 2013-12-31 - report is of type '40-F'
Data_Date: 2014-12-31 - report is of type '40-F'
Data_Date: 2015-12-31 - report is of type '40-F'
Data_Date: 2016-12-31 - report is of type '40-F'
Data_Date: 2017-12-31 - report is of type '40-F'
Data_Date: 2018-12-31 - report is of type '40-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '40-F' parameter

'''
# run functions again for 'N/A' observations for TRQ
# note: 'N/A' observations for TRQ are located in indexes 795 through to 800 of filing_page_df
for idx in range(795, 801):
    # retrieve the filing date for TRQ observations
    filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '40-F'),
                                                           ['40-F'])
    # set the filing type for TRQ observations to '40-F'
    filing_page_df.loc[idx, 'Filing_Type'] = '40-F'
    # retrieve the filing url for TRQ observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '40-F'),
                                                          ['40-F'])
    # set the filing mda placement for TRQ observations to 'EX-99.3' (as the mda section is contained in a separate exhibit)
    filing_page_df.loc[idx, 'MDA_Placement'] = 'EX-99.3'
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

-------------------------------------------------------------------------------------
TRQ
period of report: 2013-12-31
filing date: 2014-03-26
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/1158041/000119312514115526/0001193125-14-115526-index.htm
management discussion and analysis placement: EX-99.3


-------------------------------------------------------------------------------------
TRQ
period of report: 2014-12-31
filing date: 2015-03-24
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/1158041/000119312515103329/0001193125-15-103329-index.htm
management discussion and analysis placement: EX-99.3


-------------------------------------------------------------------------------------
TRQ
period of report: 2015-12-31
filing date: 2016-03-17
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/1158041/000119312516508742/0001193125-16-508742-index.htm
management discussion and analysis placement: EX-99.3


-------------------

In [37]:
# notes on 'N/A' observations for TNP
'''

TSAKOS ENERGY NAVIGATION LTD is classed as a 'foreign private issuer'; therefore, its annual filings are form 20-F filings and form 10-K filings
Data_Date: 2011-12-31 - report is of type '20-F'
Data_Date: 2012-12-31 - report is of type '20-F'
Data_Date: 2013-12-31 - report is of type '20-F'
Data_Date: 2014-12-31 - report is of type '20-F'
Data_Date: 2015-12-31 - report is of type '20-F'
Data_Date: 2016-12-31 - report is of type '20-F'
Data_Date: 2017-12-31 - report is of type '20-F'
Data_Date: 2018-12-31 - report is of type '20-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '20-F' parameter

'''
# run functions again for 'N/A' observations for TNP
# note: 'N/A' observations for TNP are located in indexes 822 through to 829 of filing_page_df
for idx in range(822, 830):
    # retrieve the filing date for TNP observations
    filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                           ['20-F'])
    # set the filing type for TNP observations to '20-F'
    filing_page_df.loc[idx, 'Filing_Type'] = '20-F'
    # retrieve the filing url for TNP observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                          ['20-F'])
    # set the filing mda placement for TNP observations to Filing_Type value
    filing_page_df.loc[idx, 'MDA_Placement'] = filing_page_df.loc[idx, 'Filing_Type']
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

-------------------------------------------------------------------------------------
TNP
period of report: 2011-12-31
filing date: 2012-04-17
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1166663/000119312512165081/0001193125-12-165081-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
TNP
period of report: 2012-12-31
filing date: 2013-04-29
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1166663/000119312513179738/0001193125-13-179738-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
TNP
period of report: 2013-12-31
filing date: 2014-04-11
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1166663/000119312514140547/0001193125-14-140547-index.htm
management discussion and analysis placement: 20-F


----------------------------

In [38]:
# notes on 'N/A' observations for AQN
'''

ALGONQUIN POWER & UTIL CORP is domiciled in Canada; therefore, its annual filings are form 40-F filings and not form 10-K filings
Data_Date: 2012-12-31 - report is of type '40-F'
Data_Date: 2013-12-31 - report is of type '40-F'
Data_Date: 2015-12-31 - report is of type '40-F'
Data_Date: 2018-12-31 - report is of type '40-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '40-F' parameter

'''
# run functions again for 'N/A' observations for AQN
# note: 'N/A' observations for AQN are located in indexes 832 through to 835 of filing_page_df
for idx in range(832, 836):
    # retrieve the filing date for AQN observations
    filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '40-F'),
                                                           ['40-F'])
    # set the filing type for AQN observations to '40-F'
    filing_page_df.loc[idx, 'Filing_Type'] = '40-F'
    # retrieve the filing url for AQN observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '40-F'),
                                                          ['40-F'])
    # set the filing mda placement for AQN observations to 'EX-99.3' (as the mda section is contained in a separate exhibit)
    filing_page_df.loc[idx, 'MDA_Placement'] = 'EX-99.3'
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

-------------------------------------------------------------------------------------
AQN
period of report: 2012-12-31
filing date: 2013-03-29
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/1174169/000119312513133176/0001193125-13-133176-index.htm
management discussion and analysis placement: EX-99.3


-------------------------------------------------------------------------------------
AQN
period of report: 2013-12-31
filing date: 2014-03-28
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/1174169/000144530514001201/0001445305-14-001201-index.htm
management discussion and analysis placement: EX-99.3


-------------------------------------------------------------------------------------
AQN
period of report: 2015-12-31
filing date: 2016-03-15
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/1174169/000162828016012691/0001628280-16-012691-index.htm
management discussion and analysis placement: EX-99.3


-------------------

In [39]:
# notes on 'N/A' observations for IAG
'''

IAMGOLD CORP is domiciled in Canada; therefore, its annual filings are form 40-F filings and not form 10-K filings
Data_Date: 2012-12-31 - report is of type '40-F'
Data_Date: 2013-12-31 - report is of type '40-F'
Data_Date: 2014-12-31 - report is of type '40-F'
Data_Date: 2015-12-31 - report is of type '40-F'
Data_Date: 2016-12-31 - report is of type '40-F'
Data_Date: 2017-12-31 - report is of type '40-F'
Data_Date: 2018-12-31 - report is of type '40-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '40-F' parameter

'''
# run functions again for 'N/A' observations for IAG
# note: 'N/A' observations for IAG are located in indexes 839 through to 845 of filing_page_df
for idx in range(839, 846):
    # retrieve the filing date for IAG observations
    filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '40-F'),
                                                           ['40-F'])
    # set the filing type for IAG observations to '40-F'
    filing_page_df.loc[idx, 'Filing_Type'] = '40-F'
    # retrieve the filing url for IAG observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '40-F'),
                                                          ['40-F'])
    # set the filing mda placement for IAG observations to 'EX-99.2' (as the mda section is contained in a separate exhibit)
    filing_page_df.loc[idx, 'MDA_Placement'] = 'EX-99.2'
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

-------------------------------------------------------------------------------------
IAG
period of report: 2012-12-31
filing date: 2013-03-26
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/1203464/000119312513124666/0001193125-13-124666-index.htm
management discussion and analysis placement: EX-99.2


-------------------------------------------------------------------------------------
IAG
period of report: 2013-12-31
filing date: 2014-03-21
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/1203464/000119312514110217/0001193125-14-110217-index.htm
management discussion and analysis placement: EX-99.2


-------------------------------------------------------------------------------------
IAG
period of report: 2014-12-31
filing date: 2015-03-25
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/1203464/000119312515105051/0001193125-15-105051-index.htm
management discussion and analysis placement: EX-99.2


-------------------

In [40]:
# notes on 'N/A' observations for AUY
'''

YAMANA GOLD INC is domiciled in Canada; therefore, its annual filings are form 40-F filings and not form 10-K filings
Data_Date: 2013-12-31 - report is of type '40-F'
Data_Date: 2014-12-31 - report is of type '40-F'
Data_Date: 2015-12-31 - report is of type '40-F'
Data_Date: 2016-12-31 - report is of type '40-F'
Data_Date: 2017-12-31 - report is of type '40-F'
Data_Date: 2018-12-31 - report is of type '40-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '40-F' parameter

'''
# run functions again for 'N/A' observations for AUY
# note: 'N/A' observations for AUY are located in indexes 856 through to 861 of filing_page_df
for idx in range(856, 862):
    # retrieve the filing date for AUY observations
    filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '40-F'),
                                                           ['40-F'])
    # set the filing type for AUY observations to '40-F'
    filing_page_df.loc[idx, 'Filing_Type'] = '40-F'
    # retrieve the filing url for AUY observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '40-F'),
                                                          ['40-F'])
    # set the filing mda placement for AUY observations to 'EX-99.2' (as the mda section is contained in a separate exhibit)
    filing_page_df.loc[idx, 'MDA_Placement'] = 'EX-99.2'
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

-------------------------------------------------------------------------------------
AUY
period of report: 2013-12-31
filing date: 2014-03-28
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/1264089/000126408914000007/0001264089-14-000007-index.htm
management discussion and analysis placement: EX-99.2


-------------------------------------------------------------------------------------
AUY
period of report: 2014-12-31
filing date: 2015-03-27
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/1264089/000126408915000005/0001264089-15-000005-index.htm
management discussion and analysis placement: EX-99.2


-------------------------------------------------------------------------------------
AUY
period of report: 2015-12-31
filing date: 2016-03-30
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/1264089/000126408916000019/0001264089-16-000019-index.htm
management discussion and analysis placement: EX-99.2


-------------------

In [41]:
# notes on 'N/A' observation for EXK
'''

ENDEAVOUR SILVER CORP is domiciled in Canada; therefore, its annual filings are form 40-F filings and not form 10-K filings
Data_Date: 2014-12-31 - report is of type '40-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '40-F' parameter

'''
# run functions again for 'N/A' observation for EXK
# note: 'N/A' observation for EXK is located in index 868 of filing_page_df
# retrieve the filing date for EXK observation
filing_page_df.loc[868, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[868, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[868, 'CIK'], '40-F'),
                                                           ['40-F'])
# set the filing type for EXK observation to '40-F'
filing_page_df.loc[868, 'Filing_Type'] = '40-F'
# retrieve the filing url for EXK observation
filing_page_df.loc[868, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[868, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[868, 'CIK'], '40-F'),
                                                          ['40-F'])
# set the filing mda placement for EXK observations to 'EX-99.4' (as the mda section is contained in a separate exhibit)
filing_page_df.loc[868, 'MDA_Placement'] = 'EX-99.4'
# provide user feedback
print('-------------------------------------------------------------------------------------')
print(filing_page_df.loc[868, 'Ticker_Symbol'])
print('period of report:', filing_page_df.loc[868, 'Data_Date'].strftime('%Y-%m-%d'))
print('filing date:', filing_page_df.loc[868, 'Filing_Date'])
print('filing type:', filing_page_df.loc[868, 'Filing_Type'])
print('filing url:', filing_page_df.loc[868, 'Filing_Url'])
print('management discussion and analysis placement:', filing_page_df.loc[868, 'MDA_Placement'])

-------------------------------------------------------------------------------------
EXK
period of report: 2014-12-31
filing date: 2015-03-05
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/1277866/000106299315001172/0001062993-15-001172-index.htm
management discussion and analysis placement: EX-99.4


In [42]:
%%capture
# notes on 'N/A' observations for BTE
'''

BAYTEX ENERGY CORP is domiciled in Canada; therefore, its annual filings are form 40-F filings and not form 10-K filings
Data_Date: 2015-12-31 - report is of type '40-F'
Data_Date: 2016-12-31 - report is of type '40-F'
Data_Date: 2017-12-31 - report is of type '40-F'
Data_Date: 2018-12-31 - report is of type '40-F'
note that the management discussion and analysis placement is not contained in filing under separate exhibit
solution - UNSOLVED

'''

In [43]:
# notes on 'N/A' observation for SFL
'''

SHIP FINANCE INTL LTD is classed as a 'foreign private issuer'; therefore, its annual filings are form 20-F filings and form 10-K filings
Data_Date: 2018-12-31 - report is of type '20-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '20-F' parameter

'''
# run functions again for 'N/A' observation for SFL
# note: 'N/A' observation for SFL is located in index 883 of filing_page_df
# retrieve the filing date for SFL observation
filing_page_df.loc[883, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[883, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[883, 'CIK'], '20-F'),
                                                           ['20-F'])
# set the filing type for SFL observation to '20-F'
filing_page_df.loc[883, 'Filing_Type'] = '20-F'
# retrieve the filing url for SFL observation
filing_page_df.loc[883, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[883, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[883, 'CIK'], '20-F'),
                                                          ['20-F'])
# set the filing mda placement for SFL observations to Filing_Type value
filing_page_df.loc[883, 'MDA_Placement'] = filing_page_df.loc[883, 'Filing_Type']
# provide user feedback
print('-------------------------------------------------------------------------------------')
print(filing_page_df.loc[883, 'Ticker_Symbol'])
print('period of report:', filing_page_df.loc[883, 'Data_Date'].strftime('%Y-%m-%d'))
print('filing date:', filing_page_df.loc[883, 'Filing_Date'])
print('filing type:', filing_page_df.loc[883, 'Filing_Type'])
print('filing url:', filing_page_df.loc[883, 'Filing_Url'])
print('management discussion and analysis placement:', filing_page_df.loc[883, 'MDA_Placement'])

-------------------------------------------------------------------------------------
SFL
period of report: 2018-12-31
filing date: 2019-03-26
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1289877/000128987719000009/0001289877-19-000009-index.htm
management discussion and analysis placement: 20-F


In [44]:
# notes on 'N/A' observations for TGP
'''

TEEKAY LNG PARTNERS LP is classed as a 'foreign private issuer'; therefore, its annual filings are form 20-F filings and form 10-K filings
Data_Date: 2015-12-31 - report is of type '20-F'
Data_Date: 2016-12-31 - report is of type '20-F'
Data_Date: 2017-12-31 - report is of type '20-F'
Data_Date: 2018-12-31 - report is of type '20-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '20-F' parameter

'''
# run functions again for 'N/A' observations for TGP
# note: 'N/A' observations for TGP are located in indexes 891 through to 894 of filing_page_df
for idx in range(891, 895):
    # retrieve the filing date for TGP observations
    filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                           ['20-F'])
    # set the filing type for TGP observations to '20-F'
    filing_page_df.loc[idx, 'Filing_Type'] = '20-F'
    # retrieve the filing url for TGP observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                          ['20-F'])
    # set the filing mda placement for TGP observations to Filing_Type value
    filing_page_df.loc[idx, 'MDA_Placement'] = filing_page_df.loc[idx, 'Filing_Type']
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

-------------------------------------------------------------------------------------
TGP
period of report: 2015-12-31
filing date: 2016-04-27
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1308106/000119312516559694/0001193125-16-559694-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
TGP
period of report: 2016-12-31
filing date: 2017-04-26
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1308106/000162828017004290/0001628280-17-004290-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
TGP
period of report: 2017-12-31
filing date: 2018-04-16
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1308106/000162828018004282/0001628280-18-004282-index.htm
management discussion and analysis placement: 20-F


----------------------------

In [45]:
# notes on 'N/A' observations for AG
'''

FIRST MAJESTIC SILVER CORP is domiciled in Canada; therefore, its annual filings are form 40-F filings and not form 10-K filings
Data_Date: 2014-12-31 - report is of type '40-F'
Data_Date: 2015-12-31 - report is of type '40-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '40-F' parameter

'''
# run functions again for 'N/A' observations for AG
# note: 'N/A' observations for AG are located in indexes 895 and 896 of filing_page_df
for idx in range(895, 897):
    # retrieve the filing date for AG observations
    filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '40-F'),
                                                           ['40-F'])
    # set the filing type for AG observations to '40-F'
    filing_page_df.loc[idx, 'Filing_Type'] = '40-F'
    # retrieve the filing url for AG observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '40-F'),
                                                          ['40-F'])
    # set the filing mda placement for AG observations to 'EX-99.3' (as the mda section is contained in a separate exhibit)
    filing_page_df.loc[idx, 'MDA_Placement'] = 'EX-99.3'
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

-------------------------------------------------------------------------------------
AG
period of report: 2014-12-31
filing date: 2015-03-31
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/1308648/000127956915000616/0001279569-15-000616-index.htm
management discussion and analysis placement: EX-99.3


-------------------------------------------------------------------------------------
AG
period of report: 2015-12-31
filing date: 2016-03-30
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/1308648/000127956916003049/0001279569-16-003049-index.htm
management discussion and analysis placement: EX-99.3




In [46]:
# notes on 'N/A' observations for DSX
'''

DIANA SHIPPING INC LP is classed as a 'foreign private issuer'; therefore, its annual filings are form 20-F filings and form 10-K filings
Data_Date: 2011-12-31 - report is of type '20-F'
Data_Date: 2012-12-31 - report is of type '20-F'
Data_Date: 2013-12-31 - report is of type '20-F'
Data_Date: 2014-12-31 - report is of type '20-F'
Data_Date: 2015-12-31 - report is of type '20-F'
Data_Date: 2016-12-31 - report is of type '20-F'
Data_Date: 2017-12-31 - report is of type '20-F'
Data_Date: 2018-12-31 - report is of type '20-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '20-F' parameter

'''
# run functions again for 'N/A' observations for DSX
# note: 'N/A' observations for DSX are located in indexes 898 through to 905 of filing_page_df
for idx in range(898, 906):
    # retrieve the filing date for DSX observations
    filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                           ['20-F'])
    # set the filing type for DSX observations to '20-F'
    filing_page_df.loc[idx, 'Filing_Type'] = '20-F'
    # retrieve the filing url for DSX observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                          ['20-F'])
    # set the filing mda placement for DSX observations to Filing_Type value
    filing_page_df.loc[idx, 'MDA_Placement'] = filing_page_df.loc[idx, 'Filing_Type']
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

-------------------------------------------------------------------------------------
DSX
period of report: 2011-12-31
filing date: 2012-04-20
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1318885/000091957412002867/0000919574-12-002867-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
DSX
period of report: 2012-12-31
filing date: 2013-03-22
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1318885/000091957413002517/0000919574-13-002517-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
DSX
period of report: 2013-12-31
filing date: 2014-03-27
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1318885/000091957414002437/0000919574-14-002437-index.htm
management discussion and analysis placement: 20-F


----------------------------

In [47]:
# notes on 'N/A' observations for HBM
'''

HUDBAY MINERALS INC is domiciled in Canada; therefore, its annual filings are form 40-F filings and not form 10-K filings
Data_Date: 2011-12-31 - report is of type '40-F'
Data_Date: 2012-12-31 - report is of type '40-F'
Data_Date: 2013-12-31 - report is of type '40-F'
Data_Date: 2014-12-31 - report is of type '40-F'
Data_Date: 2015-12-31 - report is of type '40-F'
Data_Date: 2016-12-31 - report is of type '40-F'
Data_Date: 2017-12-31 - report is of type '40-F'
Data_Date: 2018-12-31 - report is of type '40-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '40-F' parameter

'''
# run functions again for 'N/A' observations for HBM
# note: 'N/A' observations for HBM are located in indexes 909 through to 916 of filing_page_df
for idx in range(909, 917):
    # retrieve the filing date for HBM observations
    filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '40-F'),
                                                           ['40-F'])
    # set the filing type for HBM observations to '40-F'
    filing_page_df.loc[idx, 'Filing_Type'] = '40-F'
    # retrieve the filing url for HBM observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '40-F'),
                                                          ['40-F'])
    # set the filing mda placement for HBM observations to 'EX-99.3' (as the mda section is contained in a separate exhibit)
    filing_page_df.loc[idx, 'MDA_Placement'] = 'EX-99.3'
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

-------------------------------------------------------------------------------------
HBM
period of report: 2011-12-31
filing date: 2012-04-02
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/1322422/000110465912023253/0001104659-12-023253-index.htm
management discussion and analysis placement: EX-99.3


-------------------------------------------------------------------------------------
HBM
period of report: 2012-12-31
filing date: 2013-03-28
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/1322422/000110465913025425/0001104659-13-025425-index.htm
management discussion and analysis placement: EX-99.3


-------------------------------------------------------------------------------------
HBM
period of report: 2013-12-31
filing date: 2014-03-31
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/1322422/000110465914024440/0001104659-14-024440-index.htm
management discussion and analysis placement: EX-99.3


-------------------

In [48]:
# notes on 'N/A' observation for WPM
'''

WHEATON PRECIOUS METALS CORP is domiciled in Canada; therefore, its annual filings are form 40-F filings and not form 10-K filings
Data_Date: 2015-12-31 - report is of type '40-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '40-F' parameter

'''
# run functions again for 'N/A' observation for WPM
# note: 'N/A' observation for WPM is located in index 917 of filing_page_df
# retrieve the filing date for WPM observation
filing_page_df.loc[917, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[917, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[917, 'CIK'], '40-F'),
                                                           ['40-F'])
# set the filing type for WPM observation to '40-F'
filing_page_df.loc[917, 'Filing_Type'] = '40-F'
# retrieve the filing url for WPM observation
filing_page_df.loc[917, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[917, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[917, 'CIK'], '40-F'),
                                                          ['40-F'])
# set the filing mda placement for WPM observations to 'EX-99.2' (as the mda section is contained in a separate exhibit)
filing_page_df.loc[917, 'MDA_Placement'] = 'EX-99.2'
# provide user feedback
print('-------------------------------------------------------------------------------------')
print(filing_page_df.loc[917, 'Ticker_Symbol'])
print('period of report:', filing_page_df.loc[917, 'Data_Date'].strftime('%Y-%m-%d'))
print('filing date:', filing_page_df.loc[917, 'Filing_Date'])
print('filing type:', filing_page_df.loc[917, 'Filing_Type'])
print('filing url:', filing_page_df.loc[917, 'Filing_Url'])
print('management discussion and analysis placement:', filing_page_df.loc[917, 'MDA_Placement'])

-------------------------------------------------------------------------------------
WPM
period of report: 2015-12-31
filing date: 2016-03-30
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/1323404/000106299316008586/0001062993-16-008586-index.htm
management discussion and analysis placement: EX-99.2


In [49]:
# notes on 'N/A' observations for DHT
'''

DHT HOLDINGS INC is classed as a 'foreign private issuer'; therefore, its annual filings are form 20-F filings and form 10-K filings
Data_Date: 2011-12-31 - report is of type '20-F'
Data_Date: 2012-12-31 - report is of type '20-F'
Data_Date: 2013-12-31 - report is of type '20-F'
Data_Date: 2014-12-31 - report is of type '20-F'
Data_Date: 2015-12-31 - report is of type '20-F'
Data_Date: 2016-12-31 - report is of type '20-F'
Data_Date: 2017-12-31 - report is of type '20-F'
Data_Date: 2018-12-31 - report is of type '20-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '20-F' parameter

'''
# run functions again for 'N/A' observations for DHT
# note: 'N/A' observations for DHT are located in indexes 937 through to 944 of filing_page_df
for idx in range(937, 945):
    # retrieve the filing date for DHT observations
    filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                           ['20-F'])
    # set the filing type for DHT observations to '20-F'
    filing_page_df.loc[idx, 'Filing_Type'] = '20-F'
    # retrieve the filing url for DHT observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                          ['20-F'])
    # set the filing mda placement for DHT observations to Filing_Type value
    filing_page_df.loc[idx, 'MDA_Placement'] = filing_page_df.loc[idx, 'Filing_Type']
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

-------------------------------------------------------------------------------------
DHT
period of report: 2011-12-31
filing date: 2012-03-19
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1331284/000095015712000123/0000950157-12-000123-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
DHT
period of report: 2012-12-31
filing date: 2013-04-29
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1331284/000095015713000168/0000950157-13-000168-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
DHT
period of report: 2013-12-31
filing date: 2014-03-03
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1331284/000095015714000189/0000950157-14-000189-index.htm
management discussion and analysis placement: 20-F


----------------------------

In [50]:
# notes on 'N/A' observations for SSW
'''

SEASPAN CORP is classed as a 'foreign private issuer'; therefore, its annual filings are form 20-F filings and form 10-K filings
Data_Date: 2011-12-31 - report is of type '20-F'
Data_Date: 2012-12-31 - report is of type '20-F'
Data_Date: 2013-12-31 - report is of type '20-F'
Data_Date: 2014-12-31 - report is of type '20-F'
Data_Date: 2015-12-31 - report is of type '20-F'
Data_Date: 2016-12-31 - report is of type '20-F'
Data_Date: 2017-12-31 - report is of type '20-F'
Data_Date: 2018-12-31 - report is of type '20-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '20-F' parameter

'''
# run functions again for 'N/A' observations for SSW
# note: 'N/A' observations for SSW are located in indexes 947 through to 954 of filing_page_df
for idx in range(947, 955):
    # retrieve the filing date for SSW observations
    filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                           ['20-F'])
    # set the filing type for SSW observations to '20-F'
    filing_page_df.loc[idx, 'Filing_Type'] = '20-F'
    # retrieve the filing url for SSW observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                          ['20-F'])
    # set the filing mda placement for SSW observations to Filing_Type value
    filing_page_df.loc[idx, 'MDA_Placement'] = filing_page_df.loc[idx, 'Filing_Type']
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

-------------------------------------------------------------------------------------
SSW
period of report: 2011-12-31
filing date: 2012-03-26
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1332639/000119312512130108/0001193125-12-130108-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
SSW
period of report: 2012-12-31
filing date: 2013-03-19
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1332639/000119312513113506/0001193125-13-113506-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
SSW
period of report: 2013-12-31
filing date: 2014-03-11
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1332639/000119312514093840/0001193125-14-093840-index.htm
management discussion and analysis placement: 20-F


----------------------------

In [51]:
# notes on 'N/A' observations for NM
'''

NAVIOS MARITIME HOLDINGS INC is classed as a 'foreign private issuer'; therefore, its annual filings are form 20-F filings and form 10-K filings
Data_Date: 2011-12-31 - report is of type '20-F'
Data_Date: 2012-12-31 - report is of type '20-F'
Data_Date: 2013-12-31 - report is of type '20-F'
Data_Date: 2014-12-31 - report is of type '20-F'
Data_Date: 2015-12-31 - report is of type '20-F'
Data_Date: 2016-12-31 - report is of type '20-F'
Data_Date: 2017-12-31 - report is of type '20-F'
Data_Date: 2018-12-31 - report is of type '20-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '20-F' parameter

'''
# run functions again for 'N/A' observations for NM
# note: 'N/A' observations for NM are located in indexes 955 through to 962 of filing_page_df
for idx in range(955, 963):
    # retrieve the filing date for NM observations
    filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                           ['20-F'])
    # set the filing type for NM observations to '20-F'
    filing_page_df.loc[idx, 'Filing_Type'] = '20-F'
    # retrieve the filing url for NM observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                          ['20-F'])
    # set the filing mda placement for NM observations to Filing_Type value
    filing_page_df.loc[idx, 'MDA_Placement'] = filing_page_df.loc[idx, 'Filing_Type']
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

-------------------------------------------------------------------------------------
NM
period of report: 2011-12-31
filing date: 2012-03-28
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1333172/000119312512137497/0001193125-12-137497-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
NM
period of report: 2012-12-31
filing date: 2013-04-24
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1333172/000119312513168990/0001193125-13-168990-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
NM
period of report: 2013-12-31
filing date: 2014-04-29
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1333172/000119312514164924/0001193125-14-164924-index.htm
management discussion and analysis placement: 20-F


-------------------------------

In [52]:
# notes on 'N/A' observations for OBE
'''

OBSIDIAN ENERGY LTD is domiciled in Canada; therefore, its annual filings are form 40-F filings and not form 10-K filings
Data_Date: 2011-12-31 - report is of type '40-F'
Data_Date: 2012-12-31 - report is of type '40-F'
Data_Date: 2013-12-31 - report is of type '40-F'
Data_Date: 2014-12-31 - report is of type '40-F'
Data_Date: 2015-12-31 - report is of type '40-F'
Data_Date: 2016-12-31 - report is of type '40-F'
Data_Date: 2017-12-31 - report is of type '40-F'
Data_Date: 2018-12-31 - report is of type '40-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '40-F' parameter

'''
# run functions again for 'N/A' observations for OBE
# note: 'N/A' observations for OBE are located in indexes 963 through to 970 of filing_page_df
for idx in range(963, 971):
    # retrieve the filing date for OBE observations
    filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '40-F'),
                                                           ['40-F'])
    # set the filing type for OBE observations to '40-F'
    filing_page_df.loc[idx, 'Filing_Type'] = '40-F'
    # retrieve the filing url for OBE observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '40-F'),
                                                          ['40-F'])
    # set the filing mda placement for OBE observations to 'EX-99.2' (as the mda section is contained in a separate exhibit)
    filing_page_df.loc[idx, 'MDA_Placement'] = 'EX-99.2'
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

-------------------------------------------------------------------------------------
OBE
period of report: 2011-12-31
filing date: 2012-03-16
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/1334388/000119312512119598/0001193125-12-119598-index.htm
management discussion and analysis placement: EX-99.2


-------------------------------------------------------------------------------------
OBE
period of report: 2012-12-31
filing date: 2013-03-15
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/1334388/000119312513107905/0001193125-13-107905-index.htm
management discussion and analysis placement: EX-99.2


-------------------------------------------------------------------------------------
OBE
period of report: 2013-12-31
filing date: 2014-03-07
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/1334388/000119312514089189/0001193125-14-089189-index.htm
management discussion and analysis placement: EX-99.2


-------------------

In [53]:
# notes on 'N/A' observations for FSM
'''

FORTUNA SILVER MINES INC is domiciled in Canada; therefore, its annual filings are form 40-F filings and not form 10-K filings
Data_Date: 2015-12-31 - report is of type '40-F'
Data_Date: 2018-12-31 - report is of type '40-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '40-F' parameter

'''
# run functions again for 'N/A' observations for FSM
# note: 'N/A' observations for FSM are located in indexes 976 and 977 of filing_page_df
for idx in range(976, 978):
    # retrieve the filing date for FSM observations
    filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '40-F'),
                                                           ['40-F'])
    # set the filing type for FSM observations to '40-F'
    filing_page_df.loc[idx, 'Filing_Type'] = '40-F'
    # retrieve the filing url for FSM observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '40-F'),
                                                          ['40-F'])
    # set the filing mda placement for FSM observations to 'EX-99.3' (as the mda section is contained in a separate exhibit)
    filing_page_df.loc[idx, 'MDA_Placement'] = 'EX-99.3'
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

-------------------------------------------------------------------------------------
FSM
period of report: 2015-12-31
filing date: 2016-03-29
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/1341335/000121716016000460/0001217160-16-000460-index.htm
management discussion and analysis placement: EX-99.3


-------------------------------------------------------------------------------------
FSM
period of report: 2018-12-31
filing date: 2019-03-29
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/1341335/000127956919000763/0001279569-19-000763-index.htm
management discussion and analysis placement: EX-99.3




In [54]:
# notes on 'N/A' observation for NOA
'''

NORTH AMERICAN CONST GRP LTD is domiciled in Canada; therefore, its annual filings are form 40-F filings and not form 10-K filings
NORTH AMERICAN CONST GRP LTD annual filing for Data_Date: 2015-12-31 is a form 20-F filing
Data_Date: 2013-12-31 - report is of type '40-F'
Data_Date: 2014-12-31 - report is of type '40-F'
Data_Date: 2015-12-31 - report is of type '20-F'
Data_Date: 2016-12-31 - report is of type '40-F'
Data_Date: 2017-12-31 - report is of type '40-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '40-F' parameter

'''
# run functions again for 'N/A' observations for NOA
# note: 'N/A' observations for NOA are located in indexes 991 through to 995 of filing_page_df
for idx in range(991, 996):
    if idx==993:
        # retrieve the filing date for NOA observation
        # set Filing_Date manually
        filing_page_df.loc[idx, 'Filing_Date'] = '2016-02-16'
        # set the filing type for NOA observation to '20-F'
        filing_page_df.loc[idx, 'Filing_Type'] = '20-F'
        # retrieve the filing url for NOA observation
        filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                              edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                              ['20-F'])
        # set the filing mda placement for NM observation to Filing_Type value
        filing_page_df.loc[idx, 'MDA_Placement'] = filing_page_df.loc[idx, 'Filing_Type'] = '20-F'
        # provide user feedback
        print('-------------------------------------------------------------------------------------')
        print(filing_page_df.loc[idx, 'Ticker_Symbol'])
        print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
        print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
        print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
        print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
        print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
        print('\n')
        # note: the SEC strictly states that developers should not exceed 10 requests per second 
        # delay next request to EDGAR by 0.5 seconds 
        time.sleep(0.5)
    else:
        # retrieve the filing date for NOA observations
        if idx==994:
            # set Filing_Date manually
            filing_page_df.loc[idx, 'Filing_Date'] = '2017-02-14'
        else:
            filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                               edgar_search(filing_page_df.loc[idx, 'CIK'], '40-F'),
                                                               ['40-F'])
        # set the filing type for NOA observations to '40-F'
        filing_page_df.loc[idx, 'Filing_Type'] = '40-F'
        # retrieve the filing url for NOA observations
        filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                              edgar_search(filing_page_df.loc[idx, 'CIK'], '40-F'),
                                                              ['40-F'])
        # set the filing mda placement for NOA observations to 'EX-99.3' (as the mda section is contained in a separate exhibit)
        filing_page_df.loc[idx, 'MDA_Placement'] = 'EX-99.3'
        # provide user feedback
        print('-------------------------------------------------------------------------------------')
        print(filing_page_df.loc[idx, 'Ticker_Symbol'])
        print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
        print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
        print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
        print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
        print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
        print('\n')
        # note: the SEC strictly states that developers should not exceed 10 requests per second 
        # delay next request to EDGAR by 0.5 seconds 
        time.sleep(0.5)

-------------------------------------------------------------------------------------
NOA
period of report: 2013-12-31
filing date: 2014-02-19
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/1368519/000136851914000004/0001368519-14-000004-index.htm
management discussion and analysis placement: EX-99.3


-------------------------------------------------------------------------------------
NOA
period of report: 2014-12-31
filing date: 2015-02-17
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/1368519/000136851915000004/0001368519-15-000004-index.htm
management discussion and analysis placement: EX-99.3


-------------------------------------------------------------------------------------
NOA
period of report: 2015-12-31
filing date: 2016-02-16
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1368519/000136851916000024/0001368519-16-000024-index.htm
management discussion and analysis placement: 20-F


----------------------

In [55]:
# notes on 'N/A' observations for DAC
'''

DANAOS CORP is classed as a 'foreign private issuer'; therefore, its annual filings are form 20-F filings and form 10-K filings
Data_Date: 2011-12-31 - report is of type '20-F'
Data_Date: 2012-12-31 - report is of type '20-F'
Data_Date: 2013-12-31 - report is of type '20-F'
Data_Date: 2014-12-31 - report is of type '20-F'
Data_Date: 2015-12-31 - report is of type '20-F'
Data_Date: 2016-12-31 - report is of type '20-F'
Data_Date: 2017-12-31 - report is of type '20-F'
Data_Date: 2018-12-31 - report is of type '20-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '20-F' parameter

'''
# run functions again for 'N/A' observations for DAC
# note: 'N/A' observations for DAC are located in indexes 996 through to 1003 of filing_page_df
for idx in range(996, 1004):
    # retrieve the filing date for DAC observations
    filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                           ['20-F'])
    # set the filing type for DAC observations to '20-F'
    filing_page_df.loc[idx, 'Filing_Type'] = '20-F'
    # retrieve the filing url for DAC observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                          ['20-F'])
    # set the filing mda placement for DAC observations to Filing_Type value
    filing_page_df.loc[idx, 'MDA_Placement'] = filing_page_df.loc[idx, 'Filing_Type']
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

-------------------------------------------------------------------------------------
DAC
period of report: 2011-12-31
filing date: 2012-03-30
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1369241/000104746912003699/0001047469-12-003699-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
DAC
period of report: 2012-12-31
filing date: 2013-03-01
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1369241/000104746913002051/0001047469-13-002051-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
DAC
period of report: 2013-12-31
filing date: 2014-02-28
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1369241/000104746914001551/0001047469-14-001551-index.htm
management discussion and analysis placement: 20-F


----------------------------

In [56]:
# notes on 'N/A' observations for TOO
'''

TEEKAY OFFSHORE PARTNERS LP is classed as a 'foreign private issuer'; therefore, its annual filings are form 20-F filings and form 10-K filings
Data_Date: 2015-12-31 - report is of type '20-F'
Data_Date: 2016-12-31 - report is of type '20-F'
Data_Date: 2017-12-31 - report is of type '20-F'
Data_Date: 2018-12-31 - report is of type '20-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '20-F' parameter

'''
# run functions again for 'N/A' observations for TOO
# note: 'N/A' observations for TOO are located in indexes 1015 through to 1018 of filing_page_df
for idx in range(1015, 1019):
    # retrieve the filing date for TOO observations
    filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                           ['20-F'])
    # set the filing type for TOO observations to '20-F'
    filing_page_df.loc[idx, 'Filing_Type'] = '20-F'
    # retrieve the filing url for TOO observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                          ['20-F'])
    # set the filing mda placement for TOO observations to Filing_Type value
    filing_page_df.loc[idx, 'MDA_Placement'] = filing_page_df.loc[idx, 'Filing_Type']
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

-------------------------------------------------------------------------------------
TOO
period of report: 2015-12-31
filing date: 2016-04-18
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1382298/000119312516543486/0001193125-16-543486-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
TOO
period of report: 2016-12-31
filing date: 2017-04-12
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1382298/000162828017003551/0001628280-17-003551-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
TOO
period of report: 2017-12-31
filing date: 2018-03-21
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1382298/000162828018003423/0001628280-18-003423-index.htm
management discussion and analysis placement: 20-F


----------------------------

In [57]:
# notes on 'N/A' observations for CZZ
'''

COSAN LTD is classed as a 'foreign private issuer'; therefore, its annual filings are form 20-F filings and form 10-K filings
Data_Date: 2011-03-31 - report is of type '20-F'
Data_Date: 2012-03-31 - report is of type '20-F'
Data_Date: 2013-03-31 - report is of type '20-F'
Data_Date: 2013-12-31 - report is of type '20-F'
Data_Date: 2014-12-31 - report is of type '20-F'
Data_Date: 2015-12-31 - report is of type '20-F'
Data_Date: 2016-12-31 - report is of type '20-F'
Data_Date: 2017-12-31 - report is of type '20-F'
Data_Date: 2018-12-31 - report is of type '20-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '20-F' parameter

'''
# run functions again for 'N/A' observations for CZZ
# note: 'N/A' observations for CZZ are located in indexes 1051 through to 1059 of filing_page_df
for idx in range(1051, 1060):
    # retrieve the filing date for CZZ observations
    filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                           ['20-F'])
    # set the filing type for CZZ observations to '20-F'
    filing_page_df.loc[idx, 'Filing_Type'] = '20-F'
    # retrieve the filing url for CZZ observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                          ['20-F'])
    # set the filing mda placement for CZZ observations to Filing_Type value
    filing_page_df.loc[idx, 'MDA_Placement'] = filing_page_df.loc[idx, 'Filing_Type']
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

-------------------------------------------------------------------------------------
CZZ
period of report: 2011-03-31
filing date: 2011-09-30
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1402902/000095010311004101/0000950103-11-004101-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
CZZ
period of report: 2012-03-31
filing date: 2012-07-31
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1402902/000095010312003853/0000950103-12-003853-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
CZZ
period of report: 2013-03-31
filing date: 2013-07-31
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1402902/000095010313004606/0000950103-13-004606-index.htm
management discussion and analysis placement: 20-F


----------------------------

In [58]:
# notes on 'N/A' observations for BIP
'''

BROOKFIELD INFRS PTRS LP is classed as a 'foreign private issuer'; therefore, its annual filings are form 20-F filings and form 10-K filings
Data_Date: 2011-12-31 - report is of type '20-F'
Data_Date: 2012-12-31 - report is of type '20-F'
Data_Date: 2013-12-31 - report is of type '20-F'
Data_Date: 2014-12-31 - report is of type '20-F'
Data_Date: 2015-12-31 - report is of type '20-F'
Data_Date: 2016-12-31 - report is of type '20-F'
Data_Date: 2017-12-31 - report is of type '20-F'
Data_Date: 2018-12-31 - report is of type '20-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '20-F' parameter

'''
# run functions again for 'N/A' observations for BIP
# note: 'N/A' observations for BIP are located in indexes 1060 through to 1067 of filing_page_df
for idx in range(1060, 1068):
    # retrieve the filing date for BIP observations
    filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                           ['20-F'])
    # set the filing type for BIP observations to '20-F'
    filing_page_df.loc[idx, 'Filing_Type'] = '20-F'
    # retrieve the filing url for BIP observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                          ['20-F'])
    # set the filing mda placement for BIP observations to Filing_Type value
    filing_page_df.loc[idx, 'MDA_Placement'] = filing_page_df.loc[idx, 'Filing_Type']
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

-------------------------------------------------------------------------------------
BIP
period of report: 2011-12-31
filing date: 2012-04-27
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1406234/000119312512191496/0001193125-12-191496-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
BIP
period of report: 2012-12-31
filing date: 2013-04-30
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1406234/000104746913005193/0001047469-13-005193-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
BIP
period of report: 2013-12-31
filing date: 2014-03-28
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1406234/000104746914003182/0001047469-14-003182-index.htm
management discussion and analysis placement: 20-F


----------------------------

In [59]:
# notes on 'N/A' observations for TGH
'''

TEXTAINER GROUP HOLDINGS LTD is classed as a 'foreign private issuer'; therefore, its annual filings are form 20-F filings and form 10-K filings
Data_Date: 2015-12-31 - report is of type '20-F'
Data_Date: 2016-12-31 - report is of type '20-F'
Data_Date: 2017-12-31 - report is of type '20-F'
Data_Date: 2018-12-31 - report is of type '20-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '20-F' parameter

'''
# run functions again for 'N/A' observations for TGH
# note: 'N/A' observations for TGH are located in indexes 1072 through to 1075 of filing_page_df
for idx in range(1072, 1076):
    # retrieve the filing date for TGH observations
    filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                           ['20-F'])
    # set the filing type for TGH observations to '20-F'
    filing_page_df.loc[idx, 'Filing_Type'] = '20-F'
    # retrieve the filing url for TGH observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                          ['20-F'])
    # set the filing mda placement for TGH observations to Filing_Type value
    filing_page_df.loc[idx, 'MDA_Placement'] = filing_page_df.loc[idx, 'Filing_Type']
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

-------------------------------------------------------------------------------------
TGH
period of report: 2015-12-31
filing date: 2016-03-11
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1413159/000119312516501644/0001193125-16-501644-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
TGH
period of report: 2016-12-31
filing date: 2017-03-27
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1413159/000156459017005310/0001564590-17-005310-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
TGH
period of report: 2017-12-31
filing date: 2018-03-14
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1413159/000156459018005639/0001564590-18-005639-index.htm
management discussion and analysis placement: 20-F


----------------------------

In [60]:
# notes on 'N/A' observations for NMM
'''

NAVIOS MARITIME PARTNERS LP is classed as a 'foreign private issuer'; therefore, its annual filings are form 20-F filings and form 10-K filings
Data_Date: 2012-12-31 - report is of type '20-F'
Data_Date: 2014-12-31 - report is of type '20-F'
Data_Date: 2015-12-31 - report is of type '20-F'
Data_Date: 2016-12-31 - report is of type '20-F'
Data_Date: 2017-12-31 - report is of type '20-F'
Data_Date: 2018-12-31 - report is of type '20-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '20-F' parameter

'''
# run functions again for 'N/A' observations for NMM
# note: 'N/A' observations for NMM are located in indexes 1082 through to 1087 of filing_page_df
for idx in range(1082, 1088):
    # retrieve the filing date for NMM observations
    filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                           ['20-F'])
    # set the filing type for NMM observations to '20-F'
    filing_page_df.loc[idx, 'Filing_Type'] = '20-F'
    # retrieve the filing url for NMM observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                          ['20-F'])
    # set the filing mda placement for NMM observations to Filing_Type value
    filing_page_df.loc[idx, 'MDA_Placement'] = filing_page_df.loc[idx, 'Filing_Type']
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

-------------------------------------------------------------------------------------
NMM
period of report: 2012-12-31
filing date: 2013-03-15
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1415921/000119312513110353/0001193125-13-110353-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
NMM
period of report: 2014-12-31
filing date: 2015-03-13
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1415921/000119312515091409/0001193125-15-091409-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
NMM
period of report: 2015-12-31
filing date: 2016-03-23
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1415921/000119312516515161/0001193125-16-515161-index.htm
management discussion and analysis placement: 20-F


----------------------------

In [61]:
# notes on 'N/A' observations for TNK
'''

TEEKAY TANKERS LTD is classed as a 'foreign private issuer'; therefore, its annual filings are form 20-F filings and form 10-K filings
Data_Date: 2011-12-31 - report is of type '20-F'
Data_Date: 2012-12-31 - report is of type '20-F'
Data_Date: 2013-12-31 - report is of type '20-F'
Data_Date: 2015-12-31 - report is of type '20-F'
Data_Date: 2016-12-31 - report is of type '20-F'
Data_Date: 2017-12-31 - report is of type '20-F'
Data_Date: 2018-12-31 - report is of type '20-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '20-F' parameter

'''
# run functions again for 'N/A' observations for TNK
# note: 'N/A' observations for TNK are located in indexes 1094 through to 1100 of filing_page_df
for idx in range(1094, 1101):
    # retrieve the filing date for TNK observations
    filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                           ['20-F'])
    # set the filing type for TNK observations to '20-F'
    filing_page_df.loc[idx, 'Filing_Type'] = '20-F'
    # retrieve the filing url for TNK observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                          ['20-F'])
    # set the filing mda placement for TNK observations to Filing_Type value
    filing_page_df.loc[idx, 'MDA_Placement'] = filing_page_df.loc[idx, 'Filing_Type']
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

-------------------------------------------------------------------------------------
TNK
period of report: 2011-12-31
filing date: 2012-04-13
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1419945/000119312512162738/0001193125-12-162738-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
TNK
period of report: 2012-12-31
filing date: 2013-04-30
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1419945/000119312513186733/0001193125-13-186733-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
TNK
period of report: 2013-12-31
filing date: 2014-04-30
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1419945/000119312514173022/0001193125-14-173022-index.htm
management discussion and analysis placement: 20-F


----------------------------

In [62]:
# notes on 'N/A' observations for GSL
'''

GLOBAL SHIP LEASE INC is classed as a 'foreign private issuer'; therefore, its annual filings are form 20-F filings and form 10-K filings
Data_Date: 2011-12-31 - report is of type '20-F'
Data_Date: 2012-12-31 - report is of type '20-F'
Data_Date: 2013-12-31 - report is of type '20-F'
Data_Date: 2014-12-31 - report is of type '20-F'
Data_Date: 2015-12-31 - report is of type '20-F'
Data_Date: 2016-12-31 - report is of type '20-F'
Data_Date: 2017-12-31 - report is of type '20-F'
Data_Date: 2018-12-31 - report is of type '20-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '20-F' parameter

'''
# run functions again for 'N/A' observations for GSL
# note: 'N/A' observations for GSL are located in indexes 1112 through to 1119 of filing_page_df
for idx in range(1112, 1120):
    # retrieve the filing date for GSL observations
    filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                           ['20-F'])
    # set the filing type for GSL observations to '20-F'
    filing_page_df.loc[idx, 'Filing_Type'] = '20-F'
    # retrieve the filing url for GSL observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                          ['20-F'])
    # set the filing mda placement for GSL observations to Filing_Type value
    filing_page_df.loc[idx, 'MDA_Placement'] = filing_page_df.loc[idx, 'Filing_Type']
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

-------------------------------------------------------------------------------------
GSL
period of report: 2011-12-31
filing date: 2012-04-13
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1430725/000119312512162876/0001193125-12-162876-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
GSL
period of report: 2012-12-31
filing date: 2013-04-12
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1430725/000119312513153007/0001193125-13-153007-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
GSL
period of report: 2013-12-31
filing date: 2014-04-22
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1430725/000119312514152688/0001193125-14-152688-index.htm
management discussion and analysis placement: 20-F


----------------------------

In [63]:
# notes on 'N/A' observations for SB
'''

SAFE BULKERS INC is classed as a 'foreign private issuer'; therefore, its annual filings are form 20-F filings and form 10-K filings
Data_Date: 2012-12-31 - report is of type '20-F'
Data_Date: 2014-12-31 - report is of type '20-F'
Data_Date: 2015-12-31 - report is of type '20-F'
Data_Date: 2016-12-31 - report is of type '20-F'
Data_Date: 2017-12-31 - report is of type '20-F'
Data_Date: 2018-12-31 - report is of type '20-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '20-F' parameter

'''
# run functions again for 'N/A' observations for SB
# note: 'N/A' observations for SB are located in indexes 1124 through to 1129 of filing_page_df
for idx in range(1124, 1130):
    # retrieve the filing date for SB observations
    filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                           ['20-F'])
    # set the filing type for SB observations to '20-F'
    filing_page_df.loc[idx, 'Filing_Type'] = '20-F'
    # retrieve the filing url for SB observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                          ['20-F'])
    # set the filing mda placement for SB observations to Filing_Type value
    filing_page_df.loc[idx, 'MDA_Placement'] = filing_page_df.loc[idx, 'Filing_Type']
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

-------------------------------------------------------------------------------------
SB
period of report: 2012-12-31
filing date: 2013-02-28
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1434754/000093041313001278/0000930413-13-001278-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
SB
period of report: 2014-12-31
filing date: 2015-03-10
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1434754/000093041315001149/0000930413-15-001149-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
SB
period of report: 2015-12-31
filing date: 2016-03-04
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1434754/000093041316005861/0000930413-16-005861-index.htm
management discussion and analysis placement: 20-F


-------------------------------

In [64]:
# notes on 'N/A' observations for NNA
'''

NAVIOS MARITIME ACQUISITION is classed as a 'foreign private issuer'; therefore, its annual filings are form 20-F filings and form 10-K filings
Data_Date: 2011-12-31 - report is of type '20-F'
Data_Date: 2012-12-31 - report is of type '20-F'
Data_Date: 2014-12-31 - report is of type '20-F'
Data_Date: 2015-12-31 - report is of type '20-F'
Data_Date: 2016-12-31 - report is of type '20-F'
Data_Date: 2017-12-31 - report is of type '20-F'
Data_Date: 2018-12-31 - report is of type '20-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '20-F' parameter

'''
# run functions again for 'N/A' observations for NNA
# note: 'N/A' observations for NNA are located in indexes 1130 through to 1136 of filing_page_df
for idx in range(1130, 1137):
    # retrieve the filing date for NNA observations
    filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                           ['20-F'])
    # set the filing type for NNA observations to '20-F'
    filing_page_df.loc[idx, 'Filing_Type'] = '20-F'
    # retrieve the filing url for NNA observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                          ['20-F'])
    # set the filing mda placement for NNA observations to Filing_Type value
    filing_page_df.loc[idx, 'MDA_Placement'] = filing_page_df.loc[idx, 'Filing_Type']
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

-------------------------------------------------------------------------------------
NNA
period of report: 2011-12-31
filing date: 2012-03-16
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1437260/000119312512117959/0001193125-12-117959-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
NNA
period of report: 2012-12-31
filing date: 2013-03-25
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1437260/000119312513124386/0001193125-13-124386-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
NNA
period of report: 2014-12-31
filing date: 2015-03-30
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1437260/000119312515111951/0001193125-15-111951-index.htm
management discussion and analysis placement: 20-F


----------------------------

In [65]:
# notes on 'N/A' observation for GPRK
'''

GEOPARK LTD is classed as a 'foreign private issuer'; therefore, its annual filings are form 20-F filings and form 10-K filings
Data_Date: 2015-12-31 - report is of type '20-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '20-F' parameter

'''
# run functions again for 'N/A' observation for GPRK
# note: 'N/A' observation for GPRK is located in index 1164 of filing_page_df
# retrieve the filing date for GPRK observation
filing_page_df.loc[1164, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[1164, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[1164, 'CIK'], '20-F'),
                                                           ['20-F'])
# set the filing type for GPRK observation to '20-F'
filing_page_df.loc[1164, 'Filing_Type'] = '20-F'
# retrieve the filing url for GPRK observation
filing_page_df.loc[1164, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[1164, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[1164, 'CIK'], '20-F'),
                                                          ['20-F'])
# set the filing mda placement for GPRK observations to Filing_Type value
filing_page_df.loc[1164, 'MDA_Placement'] = filing_page_df.loc[1164, 'Filing_Type']
# provide user feedback
print('-------------------------------------------------------------------------------------')
print(filing_page_df.loc[1164, 'Ticker_Symbol'])
print('period of report:', filing_page_df.loc[1164, 'Data_Date'].strftime('%Y-%m-%d'))
print('filing date:', filing_page_df.loc[1164, 'Filing_Date'])
print('filing type:', filing_page_df.loc[1164, 'Filing_Type'])
print('filing url:', filing_page_df.loc[1164, 'Filing_Url'])
print('management discussion and analysis placement:', filing_page_df.loc[1164, 'MDA_Placement'])

-------------------------------------------------------------------------------------
GPRK
period of report: 2015-12-31
filing date: 2016-04-15
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1464591/000095010316012647/0000950103-16-012647-index.htm
management discussion and analysis placement: 20-F


In [66]:
# notes on 'N/A' observation for CVE
'''

CENOVUS ENERGY INC is domiciled in Canada; therefore, its annual filings are form 40-F filings and not form 10-K filings
Data_Date: 2015-12-31 - report is of type '40-F'
Data_Date: 2017-12-31 - report is of type '40-F'
Data_Date: 2018-12-31 - report is of type '40-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '40-F' parameter

'''
# run functions again for 'N/A' observations for CVE
# note: 'N/A' observations for CVE are located in indexes 1167 through to 1169 of filing_page_df
for idx in range(1167, 1170):
    # retrieve the filing date for CVE observations
    filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '40-F'),
                                                           ['40-F'])
    # set the filing type for CVE observations to '40-F'
    filing_page_df.loc[idx, 'Filing_Type'] = '40-F'
    # retrieve the filing url for CVE observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '40-F'),
                                                          ['40-F'])
    if idx==1167:
        # set the filing mda placement for GPRK observations to Filing_Type value (the 40-F filing contains the MD&A section)
        filing_page_df.loc[idx, 'MDA_Placement'] = filing_page_df.loc[idx, 'Filing_Type']
    else:
        # set the filing mda placement for CVE observations to 'EX-99.2' (as the mda section is contained in a separate exhibit)
        filing_page_df.loc[idx, 'MDA_Placement'] = 'EX-99.2'
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

-------------------------------------------------------------------------------------
CVE
period of report: 2015-12-31
filing date: 2016-02-11
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/1475260/000119312516460090/0001193125-16-460090-index.htm
management discussion and analysis placement: 40-F


-------------------------------------------------------------------------------------
CVE
period of report: 2017-12-31
filing date: 2018-02-15
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/1475260/000119312518047312/0001193125-18-047312-index.htm
management discussion and analysis placement: EX-99.2


-------------------------------------------------------------------------------------
CVE
period of report: 2018-12-31
filing date: 2019-02-13
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/1475260/000156459019002871/0001564590-19-002871-index.htm
management discussion and analysis placement: EX-99.2




In [67]:
# notes on 'N/A' observations for STING
'''

SCORPIO TANKERS INC is classed as a 'foreign private issuer'; therefore, its annual filings are form 20-F filings and form 10-K filings
Data_Date: 2012-12-31 - report is of type '20-F'
Data_Date: 2015-12-31 - report is of type '20-F'
Data_Date: 2016-12-31 - report is of type '20-F'
Data_Date: 2017-12-31 - report is of type '20-F'
Data_Date: 2018-12-31 - report is of type '20-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '20-F' parameter

'''
# run functions again for 'N/A' observations for STING
# note: 'N/A' observations for STING are located in indexes 1176 through to 1180 of filing_page_df
for idx in range(1176, 1181):
    # retrieve the filing date for STING observations
    filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                           ['20-F'])
    # set the filing type for STING observations to '20-F'
    filing_page_df.loc[idx, 'Filing_Type'] = '20-F'
    # retrieve the filing url for STING observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                          ['20-F'])
    # set the filing mda placement for STING observations to Filing_Type value
    filing_page_df.loc[idx, 'MDA_Placement'] = filing_page_df.loc[idx, 'Filing_Type']
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

-------------------------------------------------------------------------------------
STNG
period of report: 2012-12-31
filing date: 2013-03-29
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1483934/000117120013000047/0001171200-13-000047-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
STNG
period of report: 2015-12-31
filing date: 2016-03-18
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1483934/000148393416000057/0001483934-16-000057-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
STNG
period of report: 2016-12-31
filing date: 2017-03-16
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1483934/000148393417000024/0001483934-17-000024-index.htm
management discussion and analysis placement: 20-F


-------------------------

In [68]:
# notes on 'N/A' observations for AGRO
'''

ADECOAGRO SA is classed as a 'foreign private issuer'; therefore, its annual filings are form 20-F filings and form 10-K filings
Data_Date: 2013-12-31 - report is of type '20-F'
Data_Date: 2018-12-31 - report is of type '20-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '20-F' parameter

'''
# run functions again for 'N/A' observations for  AGRO
# note: 'N/A' observations for AGRO are located in indexes 1197 and 1198 of filing_page_df
for idx in range(1197, 1199):
    # retrieve the filing date for AGRO observations
    filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                           ['20-F'])
    # set the filing type for AGRO observations to '20-F'
    filing_page_df.loc[idx, 'Filing_Type'] = '20-F'
    # retrieve the filing url for AGRO observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                          ['20-F'])
    # set the filing mda placement for AGRO observations to Filing_Type value
    filing_page_df.loc[idx, 'MDA_Placement'] = filing_page_df.loc[idx, 'Filing_Type']
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

-------------------------------------------------------------------------------------
AGRO
period of report: 2013-12-31
filing date: 2014-04-30
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1499505/000119312514171434/0001193125-14-171434-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
AGRO
period of report: 2018-12-31
filing date: 2019-04-29
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1499505/000162828019004963/0001628280-19-004963-index.htm
management discussion and analysis placement: 20-F




In [69]:
# notes on 'N/A' observations for CMRE
'''

COSTAMARE INC is classed as a 'foreign private issuer'; therefore, its annual filings are form 20-F filings and form 10-K filings
Data_Date: 2015-12-31 - report is of type '20-F'
Data_Date: 2016-12-31 - report is of type '20-F'
Data_Date: 2017-12-31 - report is of type '20-F'
Data_Date: 2018-12-31 - report is of type '20-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '20-F' parameter

'''
# run functions again for 'N/A' observations for CMRE
# note: 'N/A' observations for CMRE are located in indexes 1202 through to 1205 of filing_page_df
for idx in range(1202, 1206):
    # retrieve the filing date for CMRE observations
    filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                           ['20-F'])
    # set the filing type for CMRE observations to '20-F'
    filing_page_df.loc[idx, 'Filing_Type'] = '20-F'
    # retrieve the filing url for CMRE observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                          ['20-F'])
    # set the filing mda placement for CMRE observations to Filing_Type value
    filing_page_df.loc[idx, 'MDA_Placement'] = filing_page_df.loc[idx, 'Filing_Type']
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

-------------------------------------------------------------------------------------
CMRE
period of report: 2015-12-31
filing date: 2016-04-27
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1503584/000093041316006664/0000930413-16-006664-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
CMRE
period of report: 2016-12-31
filing date: 2017-03-14
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1503584/000093041317001032/0000930413-17-001032-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
CMRE
period of report: 2017-12-31
filing date: 2018-02-27
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1503584/000093041318000609/0000930413-18-000609-index.htm
management discussion and analysis placement: 20-F


-------------------------

In [70]:
# notes on 'N/A' observations for PACD
'''

PACIFIC DRILLING SA is classed as a 'foreign private issuer'; therefore, its annual filings are form 20-F filings and form 10-K filings
Data_Date: 2013-12-31 - report is of type '20-F'
Data_Date: 2014-12-31 - report is of type '20-F'
Data_Date: 2015-12-31 - report is of type '20-F'
Data_Date: 2016-12-31 - report is of type '20-F'
Data_Date: 2017-12-31 - report is of type '20-F'
Data_Date: 2018-12-31 - report is of type '20-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '20-F' parameter

'''
# run functions again for 'N/A' observations for PACD
# note: 'N/A' observations for PACD are located in indexes 1221 through to 1226 of filing_page_df
for idx in range(1221, 1227):
    # retrieve the filing date for PACD observations
    filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                           ['20-F'])
    # set the filing type for PACD observations to '20-F'
    filing_page_df.loc[idx, 'Filing_Type'] = '20-F'
    # retrieve the filing url for PACD observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                          ['20-F'])
    # set the filing mda placement for PACD observations to Filing_Type value
    filing_page_df.loc[idx, 'MDA_Placement'] = filing_page_df.loc[idx, 'Filing_Type']
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

-------------------------------------------------------------------------------------
PACD
period of report: 2013-12-31
filing date: 2014-03-04
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1517342/000119312514080924/0001193125-14-080924-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
PACD
period of report: 2014-12-31
filing date: 2015-02-26
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1517342/000151734215000009/0001517342-15-000009-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
PACD
period of report: 2015-12-31
filing date: 2016-03-01
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1517342/000151734216000062/0001517342-16-000062-index.htm
management discussion and analysis placement: 20-F


-------------------------

In [71]:
# notes on 'N/A' observations for EROS
'''

EROS INTERNATIONAL PLC is classed as a 'foreign private issuer'; therefore, its annual filings are form 20-F filings and form 10-K filings
Data_Date: 2016-03-31 - report is of type '20-F'
Data_Date: 2017-03-31 - report is of type '20-F'
Data_Date: 2018-03-31 - report is of type '20-F'
Data_Date: 2019-03-31 - report is of type '20-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '20-F' parameter

'''
# run functions again for 'N/A' observations for EROS
# note: 'N/A' observations for EROS are located in indexes 1247 through to 1250 of filing_page_df
for idx in range(1247, 1251):
    # retrieve the filing date for EROS observations
    filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                           ['20-F'])
    # set the filing type for EROS observations to '20-F'
    filing_page_df.loc[idx, 'Filing_Type'] = '20-F'
    # retrieve the filing url for EROS observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                          ['20-F'])
    # set the filing mda placement for EROS observations to Filing_Type value
    filing_page_df.loc[idx, 'MDA_Placement'] = filing_page_df.loc[idx, 'Filing_Type']
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

-------------------------------------------------------------------------------------
EROS
period of report: 2016-03-31
filing date: 2016-07-27
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1532981/000117152016000992/0001171520-16-000992-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
EROS
period of report: 2017-03-31
filing date: 2017-07-31
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1532981/000117152017000332/0001171520-17-000332-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
EROS
period of report: 2018-03-31
filing date: 2018-07-31
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1532981/000117152018000361/0001171520-18-000361-index.htm
management discussion and analysis placement: 20-F


-------------------------

In [72]:
# notes on 'N/A' observations for BEP
'''

BROOKFIELD RENEWABLE PRTS LP is classed as a 'foreign private issuer'; therefore, its annual filings are form 20-F filings and form 10-K filings
Data_Date: 2013-12-31 - report is of type '20-F'
Data_Date: 2014-12-31 - report is of type '20-F'
Data_Date: 2015-12-31 - report is of type '20-F'
Data_Date: 2016-12-31 - report is of type '20-F'
Data_Date: 2017-12-31 - report is of type '20-F'
Data_Date: 2018-12-31 - report is of type '20-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '20-F' parameter

'''
# run functions again for 'N/A' observations for BEP
# note: 'N/A' observations for BEP are located in indexes 1251 through to 1256 of filing_page_df
for idx in range(1251, 1257):
    # retrieve the filing date for BEP observations
    if idx==1251:
        filing_page_df.loc[idx, 'Filing_Date'] = '2014-03-18'
    else:
        filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                           ['20-F'])
    # set the filing type for BEP observations to '20-F'
    filing_page_df.loc[idx, 'Filing_Type'] = '20-F'
    # retrieve the filing url for BEP observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                          ['20-F'])
    # set the filing mda placement for BEP observations to Filing_Type value
    filing_page_df.loc[idx, 'MDA_Placement'] = filing_page_df.loc[idx, 'Filing_Type']
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

-------------------------------------------------------------------------------------
BEP
period of report: 2013-12-31
filing date: 2014-03-18
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1533232/000110262414000400/0001102624-14-000400-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
BEP
period of report: 2014-12-31
filing date: 2015-03-02
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1533232/000110262415000296/0001102624-15-000296-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
BEP
period of report: 2015-12-31
filing date: 2016-02-29
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1533232/000110262416002214/0001102624-16-002214-index.htm
management discussion and analysis placement: 20-F


----------------------------

In [73]:
# notes on 'N/A' observations for GLOG
'''

GASLOG LTD is classed as a 'foreign private issuer'; therefore, its annual filings are form 20-F filings and form 10-K filings
Data_Date: 2015-12-31 - report is of type '20-F'
Data_Date: 2016-12-31 - report is of type '20-F'
Data_Date: 2017-12-31 - report is of type '20-F'
Data_Date: 2018-12-31 - report is of type '20-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '20-F' parameter

'''
# run functions again for 'N/A' observations for GLOG
# note: 'N/A' observations for GLOG are located in indexes 1262 through to 1265 of filing_page_df
for idx in range(1262, 1266):
    # retrieve the filing date for GLOG observations
    filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                           ['20-F'])
    # set the filing type for GLOG observations to '20-F'
    filing_page_df.loc[idx, 'Filing_Type'] = '20-F'
    # retrieve the filing url for GLOG observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                          ['20-F'])
    # set the filing mda placement for GLOG observations to Filing_Type value
    filing_page_df.loc[idx, 'MDA_Placement'] = filing_page_df.loc[idx, 'Filing_Type']
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

-------------------------------------------------------------------------------------
GLOG
period of report: 2015-12-31
filing date: 2016-03-14
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1534126/000093041316006011/0000930413-16-006011-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
GLOG
period of report: 2016-12-31
filing date: 2017-03-01
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1534126/000093041317000810/0000930413-17-000810-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
GLOG
period of report: 2017-12-31
filing date: 2018-02-28
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1534126/000104746918001163/0001047469-18-001163-index.htm
management discussion and analysis placement: 20-F


-------------------------

In [74]:
# notes on 'N/A' observations for CPG
'''

CRESCENT POINT ENERGY CORP is domiciled in Canada; therefore, its annual filings are form 40-F filings and not form 10-K filings
Data_Date: 2014-12-31 - report is of type '40-F'
Data_Date: 2015-12-31 - report is of type '40-F'
Data_Date: 2016-12-31 - report is of type '40-F'
Data_Date: 2017-12-31 - report is of type '40-F'
Data_Date: 2018-12-31 - report is of type '40-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '40-F' parameter

'''
# run functions again for 'N/A' observations for CPG
# note: 'N/A' observations for CPG are located in indexes 1275 through to 1279 of filing_page_df
for idx in range(1275, 1280):
    # retrieve the filing date for CPG observations
    filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '40-F'),
                                                           ['40-F'])
    # set the filing type for CPG observations to '40-F'
    filing_page_df.loc[idx, 'Filing_Type'] = '40-F'
    # retrieve the filing url for CPG observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '40-F'),
                                                          ['40-F'])
    # set the filing mda placement for CPG observations to 'EX-99.3' (as the mda section is contained in a separate exhibit)
    filing_page_df.loc[idx, 'MDA_Placement'] = 'EX-99.3'
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

-------------------------------------------------------------------------------------
CPG
period of report: 2014-12-31
filing date: 2015-03-11
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/1545851/000110262415000363/0001102624-15-000363-index.htm
management discussion and analysis placement: EX-99.3


-------------------------------------------------------------------------------------
CPG
period of report: 2015-12-31
filing date: 2016-03-09
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/1545851/000110262416002276/0001102624-16-002276-index.htm
management discussion and analysis placement: EX-99.3


-------------------------------------------------------------------------------------
CPG
period of report: 2016-12-31
filing date: 2017-02-23
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/1545851/000119907317000144/0001199073-17-000144-index.htm
management discussion and analysis placement: EX-99.3


-------------------

In [75]:
# notes on 'N/A' observations for ANFI
'''

AMIRA NATURE FOODS LTD is classed as a 'foreign private issuer'; therefore, its annual filings are form 20-F filings and form 10-K filings
Data_Date: 2015-03-31 - report is of type '20-F'
Data_Date: 2016-03-31 - report is of type '20-F'
Data_Date: 2017-03-31 - report is of type '20-F'
Data_Date: 2018-03-31 - report is of type '20-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '20-F' parameter

'''
# run functions again for 'N/A' observations for ANFI
# note: 'N/A' observations for ANFI are located in indexes 1288 through to 1291 of filing_page_df
for idx in range(1288, 1292):
    # retrieve the filing date for ANFI observations
    if idx==1288:
        filing_page_df.loc[idx, 'Filing_Date'] = '2016-01-25'
    else:
        filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                           ['20-F'])
    # set the filing type for ANFI observations to '20-F'
    filing_page_df.loc[idx, 'Filing_Type'] = '20-F'
    # retrieve the filing url for ANFI observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                          ['20-F'])
    # set the filing mda placement for ANFI observations to Filing_Type value
    filing_page_df.loc[idx, 'MDA_Placement'] = filing_page_df.loc[idx, 'Filing_Type']
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

-------------------------------------------------------------------------------------
ANFI
period of report: 2015-03-31
filing date: 2016-01-25
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1552448/000114420416076738/0001144204-16-076738-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
ANFI
period of report: 2016-03-31
filing date: 2016-07-29
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1552448/000114420416115291/0001144204-16-115291-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
ANFI
period of report: 2017-03-31
filing date: 2017-07-31
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1552448/000114420417039118/0001144204-17-039118-index.htm
management discussion and analysis placement: 20-F


-------------------------

In [76]:
# notes on 'N/A' observations for KNOP
'''

KNOT OFFSHORE PRTNRS LP is classed as a 'foreign private issuer'; therefore, its annual filings are form 20-F filings and form 10-K filings
Data_Date: 2015-12-31 - report is of type '20-F'
Data_Date: 2017-12-31 - report is of type '20-F'
Data_Date: 2018-12-31 - report is of type '20-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '20-F' parameter

'''
# run functions again for 'N/A' observations for KNOP
# note: 'N/A' observations for KNOP are located in indexes 1292 through to 1294 of filing_page_df
for idx in range(1292, 1295):
    # retrieve the filing date for KNOP observations
    filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                           ['20-F'])
    # set the filing type for KNOP observations to '20-F'
    filing_page_df.loc[idx, 'Filing_Type'] = '20-F'
    # retrieve the filing url for KNOP observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                          ['20-F'])
    # set the filing mda placement for KNOP observations to Filing_Type value
    filing_page_df.loc[idx, 'MDA_Placement'] = filing_page_df.loc[idx, 'Filing_Type']
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

-------------------------------------------------------------------------------------
KNOP
period of report: 2015-12-31
filing date: 2016-03-18
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1564180/000119312516509485/0001193125-16-509485-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
KNOP
period of report: 2017-12-31
filing date: 2018-04-25
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1564180/000119312518131307/0001193125-18-131307-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
KNOP
period of report: 2018-12-31
filing date: 2019-04-10
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1564180/000104746919002126/0001047469-19-002126-index.htm
management discussion and analysis placement: 20-F




In [77]:
# notes on 'N/A' observation for MNK
'''

MALLINCKRODT PLC is domiciled in the US; filing for Data_Date: 2017-12-31 was not found as no filings exist for year 2017
Data_Date: 2017-12-31 - report is of type '10-K'
solution - add filing date to filing_page_df manually for MNK observation, and run edgar_filing_page function again with '10-K' parameter

'''
# run functions again for 'N/A' observation for MNK
# note: 'N/A' observation for MNK is located in index 1299 of filing_page_df
# retrieve the filing date for MNK observation
filing_page_df.loc[1299, 'Filing_Date'] = '2018-02-27'
# set the filing type for MNK observation to '10-K'
filing_page_df.loc[1299, 'Filing_Type'] = '10-K'
# retrieve the filing url for MNK observation
filing_page_df.loc[1299, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[1299, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[1299, 'CIK'], '10-K'),
                                                          ['10-K'])
# set the filing mda placement for MNK observations to Filing_Type value
filing_page_df.loc[1299, 'MDA_Placement'] = filing_page_df.loc[1299, 'Filing_Type']
# provide user feedback
print('-------------------------------------------------------------------------------------')
print(filing_page_df.loc[1299, 'Ticker_Symbol'])
print('period of report:', filing_page_df.loc[1299, 'Data_Date'].strftime('%Y-%m-%d'))
print('filing date:', filing_page_df.loc[1299, 'Filing_Date'])
print('filing type:', filing_page_df.loc[1299, 'Filing_Type'])
print('filing url:', filing_page_df.loc[1299, 'Filing_Url'])
print('management discussion and analysis placement:', filing_page_df.loc[1299, 'MDA_Placement'])

-------------------------------------------------------------------------------------
MNK
period of report: 2017-12-31
filing date: 2018-02-27
filing type: 10-K
filing url: https://www.sec.gov/Archives/edgar/data/1567892/000156789218000010/0001567892-18-000010-index.htm
management discussion and analysis placement: 10-K


In [78]:
# notes on 'N/A' observations for ASC
'''

ARDMORE SHIPPING CORP is classed as a 'foreign private issuer'; therefore, its annual filings are form 20-F filings and form 10-K filings
Data_Date: 2015-12-31 - report is of type '20-F'
Data_Date: 2016-12-31 - report is of type '20-F'
Data_Date: 2017-12-31 - report is of type '20-F'
Data_Date: 2018-12-31 - report is of type '20-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '20-F' parameter

'''
# run functions again for 'N/A' observations for ASC
# note: 'N/A' observations for ASC are located in indexes 1302 through to 1305 of filing_page_df
for idx in range(1302, 1306):
    # retrieve the filing date for ASC observations
    filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                           ['20-F'])
    # set the filing type for ASC observations to '20-F'
    filing_page_df.loc[idx, 'Filing_Type'] = '20-F'
    # retrieve the filing url for ASC observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                          ['20-F'])
    # set the filing mda placement for ASC observations to Filing_Type value
    filing_page_df.loc[idx, 'MDA_Placement'] = filing_page_df.loc[idx, 'Filing_Type']
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

-------------------------------------------------------------------------------------
ASC
period of report: 2015-12-31
filing date: 2016-04-06
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1577437/000114420416092895/0001144204-16-092895-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
ASC
period of report: 2016-12-31
filing date: 2017-03-13
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1577437/000114420417014251/0001144204-17-014251-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
ASC
period of report: 2017-12-31
filing date: 2018-03-29
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1577437/000114420418017953/0001144204-18-017953-index.htm
management discussion and analysis placement: 20-F


----------------------------

In [79]:
# notes on 'N/A' observations for DLING
'''

DYNAGAS LNG PARTNERS LP is classed as a 'foreign private issuer'; therefore, its annual filings are form 20-F filings and form 10-K filings
Data_Date: 2015-12-31 - report is of type '20-F'
Data_Date: 2016-12-31 - report is of type '20-F'
Data_Date: 2017-12-31 - report is of type '20-F'
Data_Date: 2018-12-31 - report is of type '20-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '20-F' parameter

'''
# run functions again for 'N/A' observations for DLING
# note: 'N/A' observations for DLING are located in indexes 1306 through to 1309 of filing_page_df
for idx in range(1306, 1310):
    # retrieve the filing date for DLING observations
    filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                           ['20-F'])
    # set the filing type for DLING observations to '20-F'
    filing_page_df.loc[idx, 'Filing_Type'] = '20-F'
    # retrieve the filing url for DLING observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                          ['20-F'])
    # set the filing mda placement for DLING observations to Filing_Type value
    filing_page_df.loc[idx, 'MDA_Placement'] = filing_page_df.loc[idx, 'Filing_Type']
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

-------------------------------------------------------------------------------------
DLNG
period of report: 2015-12-31
filing date: 2016-04-18
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1578453/000091957416012495/0000919574-16-012495-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
DLNG
period of report: 2016-12-31
filing date: 2017-03-20
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1578453/000091957417002894/0000919574-17-002894-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
DLNG
period of report: 2017-12-31
filing date: 2018-03-09
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1578453/000091957418002352/0000919574-18-002352-index.htm
management discussion and analysis placement: 20-F


-------------------------

In [80]:
# notes on 'N/A' observations for NVGS
'''

NAVIGATOR HOLDINGS LTD is classed as a 'foreign private issuer'; therefore, its annual filings are form 20-F filings and form 10-K filings
Data_Date: 2015-12-31 - report is of type '20-F'
Data_Date: 2016-12-31 - report is of type '20-F'
Data_Date: 2017-12-31 - report is of type '20-F'
Data_Date: 2018-12-31 - report is of type '20-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '20-F' parameter

'''
# run functions again for 'N/A' observations for NVGS
# note: 'N/A' observations for NVGS are located in indexes 1316 through to 1319 of filing_page_df
for idx in range(1316, 1320):
    # retrieve the filing date for NVGS observations
    filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                           ['20-F'])
    # set the filing type for NVGS observations to '20-F'
    filing_page_df.loc[idx, 'Filing_Type'] = '20-F'
    # retrieve the filing url for NVGS observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                          ['20-F'])
    # set the filing mda placement for NVGS observations to Filing_Type value
    filing_page_df.loc[idx, 'MDA_Placement'] = filing_page_df.loc[idx, 'Filing_Type']
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

-------------------------------------------------------------------------------------
NVGS
period of report: 2015-12-31
filing date: 2016-03-03
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1581804/000119312516491713/0001193125-16-491713-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
NVGS
period of report: 2016-12-31
filing date: 2017-03-01
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1581804/000119312517065882/0001193125-17-065882-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
NVGS
period of report: 2017-12-31
filing date: 2018-03-05
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1581804/000119312518070502/0001193125-18-070502-index.htm
management discussion and analysis placement: 20-F


-------------------------

In [81]:
# notes on 'N/A' observations for SALT
'''

SCORPIO BULKERS is classed as a 'foreign private issuer'; therefore, its annual filings are form 20-F filings and form 10-K filings
Data_Date: 2015-12-31 - report is of type '20-F'
Data_Date: 2016-12-31 - report is of type '20-F'
Data_Date: 2017-12-31 - report is of type '20-F'
Data_Date: 2018-12-31 - report is of type '20-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '20-F' parameter

'''
# run functions again for 'N/A' observations for SALT
# note: 'N/A' observations for SALT are located in indexes 1325 through to 1328 of filing_page_df
for idx in range(1325, 1329):
    # retrieve the filing date for SALT observations
    filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                           ['20-F'])
    # set the filing type for SALT observations to '20-F'
    filing_page_df.loc[idx, 'Filing_Type'] = '20-F'
    # retrieve the filing url for SALT observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                          ['20-F'])
    # set the filing mda placement for SALT observations to Filing_Type value
    filing_page_df.loc[idx, 'MDA_Placement'] = filing_page_df.loc[idx, 'Filing_Type']
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

-------------------------------------------------------------------------------------
SALT
period of report: 2015-12-31
filing date: 2016-03-01
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1587264/000158726416000039/0001587264-16-000039-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
SALT
period of report: 2016-12-31
filing date: 2017-02-28
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1587264/000158726417000007/0001587264-17-000007-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
SALT
period of report: 2017-12-31
filing date: 2018-03-02
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1587264/000158726418000007/0001587264-18-000007-index.htm
management discussion and analysis placement: 20-F


-------------------------

In [82]:
# notes on 'N/A' observations for PSV
'''

HERMITAGE OFFSHORE SERVICES is classed as a 'foreign private issuer'; therefore, its annual filings are form 20-F filings and form 10-K filings
Data_Date: 2016-12-31 - report is of type '20-F'
Data_Date: 2017-12-31 - report is of type '20-F'
Data_Date: 2018-12-31 - report is of type '20-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '20-F' parameter

'''
# run functions again for 'N/A' observations for PSV
# note: 'N/A' observations for PSV are located in indexes 1349 through to 1351 of filing_page_df
for idx in range(1349, 1352):
    # retrieve the filing date for PSV observations
    filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                           ['20-F'])
    # set the filing type for PSV observations to '20-F'
    filing_page_df.loc[idx, 'Filing_Type'] = '20-F'
    # retrieve the filing url for PSV observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                          ['20-F'])
    # set the filing mda placement for PSV observations to Filing_Type value
    filing_page_df.loc[idx, 'MDA_Placement'] = filing_page_df.loc[idx, 'Filing_Type']
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

-------------------------------------------------------------------------------------
PSV
period of report: 2016-12-31
filing date: 2017-04-24
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1597659/000091957417003568/0000919574-17-003568-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
PSV
period of report: 2017-12-31
filing date: 2018-05-03
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1597659/000091957418003272/0000919574-18-003272-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
PSV
period of report: 2018-12-31
filing date: 2019-05-15
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1597659/000091957419003731/0000919574-19-003731-index.htm
management discussion and analysis placement: 20-F




In [83]:
# notes on 'N/A' observations for GLOP
'''

GASLOG PARTNERS LP is classed as a 'foreign private issuer'; therefore, its annual filings are form 20-F filings and form 10-K filings
Data_Date: 2016-12-31 - report is of type '20-F'
Data_Date: 2017-12-31 - report is of type '20-F'
Data_Date: 2018-12-31 - report is of type '20-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '20-F' parameter

'''
# run functions again for 'N/A' observations for GLOP
# note: 'N/A' observations for GLOP are located in indexes 1356 through to 1358 of filing_page_df
for idx in range(1356, 1359):
    # retrieve the filing date for GLOP observations
    filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                           ['20-F'])
    # set the filing type for GLOP observations to '20-F'
    filing_page_df.loc[idx, 'Filing_Type'] = '20-F'
    # retrieve the filing url for GLOP observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                          ['20-F'])
    # set the filing mda placement for GLOP observations to Filing_Type value
    filing_page_df.loc[idx, 'MDA_Placement'] = filing_page_df.loc[idx, 'Filing_Type']
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

-------------------------------------------------------------------------------------
GLOP
period of report: 2016-12-31
filing date: 2017-02-13
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1598655/000093041317000414/0000930413-17-000414-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
GLOP
period of report: 2017-12-31
filing date: 2018-02-12
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1598655/000104746918000752/0001047469-18-000752-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
GLOP
period of report: 2018-12-31
filing date: 2019-02-26
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1598655/000104746919000676/0001047469-19-000676-index.htm
management discussion and analysis placement: 20-F




In [84]:
# notes on 'N/A' observation for HMLP
'''

HOEGH LNG PARTNERS LP is classed as a 'foreign private issuer'; therefore, its annual filings are form 20-F filings and form 10-K filings
Data_Date: 2018-12-31 - report is of type '20-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '20-F' parameter

'''
# run functions again for 'N/A' observation for HMLP
# note: 'N/A' observation for HMLP is located in index 1367 of filing_page_df
# retrieve the filing date for HMLP observation
filing_page_df.loc[1367, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[1367, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[1367, 'CIK'], '20-F'),
                                                           ['20-F'])
# set the filing type for HMLP observation to '20-F'
filing_page_df.loc[1367, 'Filing_Type'] = '20-F'
# retrieve the filing url for HMLP observation
filing_page_df.loc[1367, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[1367, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[1367, 'CIK'], '20-F'),
                                                          ['20-F'])
# set the filing mda placement for HMLP observations to Filing_Type value
filing_page_df.loc[1367, 'MDA_Placement'] = filing_page_df.loc[1367, 'Filing_Type']
# provide user feedback
print('-------------------------------------------------------------------------------------')
print(filing_page_df.loc[1367, 'Ticker_Symbol'])
print('period of report:', filing_page_df.loc[1367, 'Data_Date'].strftime('%Y-%m-%d'))
print('filing date:', filing_page_df.loc[1367, 'Filing_Date'])
print('filing type:', filing_page_df.loc[1367, 'Filing_Type'])
print('filing url:', filing_page_df.loc[1367, 'Filing_Url'])
print('management discussion and analysis placement:', filing_page_df.loc[1367, 'MDA_Placement'])

-------------------------------------------------------------------------------------
HMLP
period of report: 2018-12-31
filing date: 2019-04-10
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1603016/000114420419019053/0001144204-19-019053-index.htm
management discussion and analysis placement: 20-F


In [85]:
# notes on 'N/A' observations for EURN
'''

EURONAV is classed as a 'foreign private issuer'; therefore, its annual filings are form 20-F filings and form 10-K filings
Data_Date: 2015-12-31 - report is of type '20-F'
Data_Date: 2016-12-31 - report is of type '20-F'
Data_Date: 2017-12-31 - report is of type '20-F'
Data_Date: 2018-12-31 - report is of type '20-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '20-F' parameter

'''
# run functions again for 'N/A' observations for EURN
# note: 'N/A' observations for EURN are located in indexes 1371 through to 1374 of filing_page_df
for idx in range(1371, 1375):
    # retrieve the filing date for EURN observations
    filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                           ['20-F'])
    # set the filing type for EURN observations to '20-F'
    filing_page_df.loc[idx, 'Filing_Type'] = '20-F'
    # retrieve the filing url for EURN observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                          ['20-F'])
    # set the filing mda placement for EURN observations to Filing_Type value
    filing_page_df.loc[idx, 'MDA_Placement'] = filing_page_df.loc[idx, 'Filing_Type']
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

-------------------------------------------------------------------------------------
EURN
period of report: 2015-12-31
filing date: 2016-04-05
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1604481/000091957416012266/0000919574-16-012266-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
EURN
period of report: 2016-12-31
filing date: 2017-04-14
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1604481/000091957417003437/0000919574-17-003437-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
EURN
period of report: 2017-12-31
filing date: 2018-04-17
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1604481/000160448118000007/0001604481-18-000007-index.htm
management discussion and analysis placement: 20-F


-------------------------

In [86]:
# notes on 'N/A' observation for FCAU
'''

FIAT CHRYSLER AUTOMOBILES NV is classed as a 'foreign private issuer'; therefore, its annual filings are form 20-F filings and form 10-K filings
Data_Date: 2018-12-31 - report is of type '20-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '20-F' parameter

'''
# run functions again for 'N/A' observation for FCAU
# note: 'N/A' observation for FCAU is located in index 1378 of filing_page_df
# retrieve the filing date for FCAU observation
filing_page_df.loc[1378, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[1378, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[1378, 'CIK'], '20-F'),
                                                           ['20-F'])
# set the filing type for FCAU observation to '20-F'
filing_page_df.loc[1378, 'Filing_Type'] = '20-F'
# retrieve the filing url for FCAU observation
filing_page_df.loc[1378, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[1378, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[1378, 'CIK'], '20-F'),
                                                          ['20-F'])
# set the filing mda placement for FCAU observations to Filing_Type value
filing_page_df.loc[1378, 'MDA_Placement'] = filing_page_df.loc[1378, 'Filing_Type']
# provide user feedback
print('-------------------------------------------------------------------------------------')
print(filing_page_df.loc[1378, 'Ticker_Symbol'])
print('period of report:', filing_page_df.loc[1378, 'Data_Date'].strftime('%Y-%m-%d'))
print('filing date:', filing_page_df.loc[1378, 'Filing_Date'])
print('filing type:', filing_page_df.loc[1378, 'Filing_Type'])
print('filing url:', filing_page_df.loc[1378, 'Filing_Url'])
print('management discussion and analysis placement:', filing_page_df.loc[1378, 'MDA_Placement'])

-------------------------------------------------------------------------------------
FCAU
period of report: 2018-12-31
filing date: 2019-02-22
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1605484/000160548419000014/0001605484-19-000014-index.htm
management discussion and analysis placement: 20-F


In [87]:
# notes on 'N/A' observation for ATTO
'''

ATENTO SA is classed as a 'foreign private issuer'; therefore, its annual filings are form 20-F filings and form 10-K filings
Data_Date: 2018-12-31 - report is of type '20-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '20-F' parameter

'''
# run functions again for 'N/A' observation for ATTO
# note: 'N/A' observation for ATTO is located in index 1381 of filing_page_df
# retrieve the filing date for ATTO observation
filing_page_df.loc[1381, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[1381, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[1381, 'CIK'], '20-F'),
                                                           ['20-F'])
# set the filing type for ATTO observation to '20-F'
filing_page_df.loc[1381, 'Filing_Type'] = '20-F'
# retrieve the filing url for ATTO observation
filing_page_df.loc[1381, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[1381, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[1381, 'CIK'], '20-F'),
                                                          ['20-F'])
# set the filing mda placement for ATTO observations to Filing_Type value
filing_page_df.loc[1381, 'MDA_Placement'] = filing_page_df.loc[1381, 'Filing_Type']
# provide user feedback
print('-------------------------------------------------------------------------------------')
print(filing_page_df.loc[1381, 'Ticker_Symbol'])
print('period of report:', filing_page_df.loc[1381, 'Data_Date'].strftime('%Y-%m-%d'))
print('filing date:', filing_page_df.loc[1381, 'Filing_Date'])
print('filing type:', filing_page_df.loc[1381, 'Filing_Type'])
print('filing url:', filing_page_df.loc[1381, 'Filing_Url'])
print('management discussion and analysis placement:', filing_page_df.loc[1381, 'MDA_Placement'])

-------------------------------------------------------------------------------------
ATTO
period of report: 2018-12-31
filing date: 2019-03-20
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1606457/000129281419000802/0001292814-19-000802-index.htm
management discussion and analysis placement: 20-F


In [88]:
# notes on 'N/A' observations for KEN
'''

KENON HOLDINGS LTD is classed as a 'foreign private issuer'; therefore, its annual filings are form 20-F filings and form 10-K filings
Data_Date: 2017-12-31 - report is of type '20-F'
Data_Date: 2018-12-31 - report is of type '20-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '20-F' parameter

'''
# run functions again for 'N/A' observations for KEN
# note: 'N/A' observations for KEN are located in indexes 1385 and 1386 of filing_page_df
for idx in range(1385, 1387):
    # retrieve the filing date for KEN observations
    filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                           ['20-F'])
    # set the filing type for KEN observations to '20-F'
    filing_page_df.loc[idx, 'Filing_Type'] = '20-F'
    # retrieve the filing url for KEN observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                          ['20-F'])
    # set the filing mda placement for KEN observations to Filing_Type value
    filing_page_df.loc[idx, 'MDA_Placement'] = filing_page_df.loc[idx, 'Filing_Type']
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

-------------------------------------------------------------------------------------
KEN
period of report: 2017-12-31
filing date: 2018-04-09
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1611005/000117891318001140/0001178913-18-001140-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
KEN
period of report: 2018-12-31
filing date: 2019-04-08
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1611005/000117891319001062/0001178913-19-001062-index.htm
management discussion and analysis placement: 20-F




In [89]:
# notes on 'N/A' observation for CJ
'''

C&J ENERGY SERVICES INC is domiciled in the US; filing for Data_Date: 2014-12-31 was not found as no filings exist for year 2014
Data_Date: 2014-12-31 - report is of type '10-K'
solution - add filing date to filing_page_df manually for CJ observation, and run edgar_filing_page function again with '10-K' parameter

'''
# run functions again for 'N/A' observation for CJ
# note: 'N/A' observation for CJ is located in index 1387 of filing_page_df
# retrieve the filing date for CJ observation
filing_page_df.loc[1387, 'Filing_Date'] = '2015-03-23'
# set the filing type for CJ observation to '10-K'
filing_page_df.loc[1387, 'Filing_Type'] = '10-K'
# retrieve the filing url for CJ observation
filing_page_df.loc[1387, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[1387, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[1387, 'CIK'], '10-K'),
                                                          ['10-K'])
# set the filing mda placement for CJ observations to Filing_Type value
filing_page_df.loc[1387, 'MDA_Placement'] = filing_page_df.loc[1387, 'Filing_Type']
# provide user feedback
print('-------------------------------------------------------------------------------------')
print(filing_page_df.loc[1387, 'Ticker_Symbol'])
print('period of report:', filing_page_df.loc[1387, 'Data_Date'].strftime('%Y-%m-%d'))
print('filing date:', filing_page_df.loc[1387, 'Filing_Date'])
print('filing type:', filing_page_df.loc[1387, 'Filing_Type'])
print('filing url:', filing_page_df.loc[1387, 'Filing_Url'])
print('management discussion and analysis placement:', filing_page_df.loc[1387, 'MDA_Placement'])

-------------------------------------------------------------------------------------
CJ
period of report: 2014-12-31
filing date: 2015-03-23
filing type: 10-K
filing url: https://www.sec.gov/Archives/edgar/data/1615817/000110465915021779/0001104659-15-021779-index.htm
management discussion and analysis placement: 10-K


In [90]:
%%capture
# notes on 'N/A' observation for JAX
'''

J. ALEXANDER'S HOLDINGS INC is domiciled in US; therefore, its annual filings form 10-K filings
Data_Date: 2011-12-31 - report does not exist
solution - UNSOLVED

'''

In [91]:
# notes on 'N/A' observation for QSR
'''

RESTAURANT BRANDS INTL INC is domiciled in the US; filing for Data_Date: 2014-12-31 was not found as no filings exist for year 2014
Data_Date: 2014-12-31 - report is of type '10-K'
solution - add filing date to filing_page_df manually for QSR observation, and run edgar_filing_page function again with '10-K' parameter

'''
# run functions again for 'N/A' observation for QSR
# note: 'N/A' observation for QSR is located in index 1395 of filing_page_df
# retrieve the filing date for QSR observation
filing_page_df.loc[1395, 'Filing_Date'] = '2015-03-02'
# set the filing type for QSR observation to '10-K'
filing_page_df.loc[1395, 'Filing_Type'] = '10-K'
# retrieve the filing url for QSR observation
filing_page_df.loc[1395, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[1395, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[1395, 'CIK'], '10-K'),
                                                          ['10-K'])
# set the filing mda placement for QSR observations to Filing_Type value
filing_page_df.loc[1395, 'MDA_Placement'] = filing_page_df.loc[1395, 'Filing_Type']
# provide user feedback
print('-------------------------------------------------------------------------------------')
print(filing_page_df.loc[1395, 'Ticker_Symbol'])
print('period of report:', filing_page_df.loc[1395, 'Data_Date'].strftime('%Y-%m-%d'))
print('filing date:', filing_page_df.loc[1395, 'Filing_Date'])
print('filing type:', filing_page_df.loc[1395, 'Filing_Type'])
print('filing url:', filing_page_df.loc[1395, 'Filing_Url'])
print('management discussion and analysis placement:', filing_page_df.loc[1395, 'MDA_Placement'])

-------------------------------------------------------------------------------------
QSR
period of report: 2014-12-31
filing date: 2015-03-02
filing type: 10-K
filing url: https://www.sec.gov/Archives/edgar/data/1618756/000119312515072217/0001193125-15-072217-index.htm
management discussion and analysis placement: 10-K


In [92]:
# notes on 'N/A' observation for IGT
'''

INTL GAME TECHNOLOGY PLC is classed as a 'foreign private issuer'; therefore, its annual filings are form 20-F filings and form 10-K filings
Data_Date: 2018-12-31 - report is of type '20-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '20-F' parameter

'''
# run functions again for 'N/A' observation for IGT
# note: 'N/A' observation for IGT is located in index 1396 of filing_page_df
# retrieve the filing date for IGT observation
filing_page_df.loc[1396, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[1396, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[1396, 'CIK'], '20-F'),
                                                           ['20-F'])
# set the filing type for IGT observation to '20-F'
filing_page_df.loc[1396, 'Filing_Type'] = '20-F'
# retrieve the filing url for IGT observation
filing_page_df.loc[1396, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[1396, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[1396, 'CIK'], '20-F'),
                                                          ['20-F'])
# set the filing mda placement for IGT observations to Filing_Type value
filing_page_df.loc[1396, 'MDA_Placement'] = filing_page_df.loc[1396, 'Filing_Type']
# provide user feedback
print('-------------------------------------------------------------------------------------')
print(filing_page_df.loc[1396, 'Ticker_Symbol'])
print('period of report:', filing_page_df.loc[1396, 'Data_Date'].strftime('%Y-%m-%d'))
print('filing date:', filing_page_df.loc[1396, 'Filing_Date'])
print('filing type:', filing_page_df.loc[1396, 'Filing_Type'])
print('filing url:', filing_page_df.loc[1396, 'Filing_Url'])
print('management discussion and analysis placement:', filing_page_df.loc[1396, 'MDA_Placement'])

-------------------------------------------------------------------------------------
IGT
period of report: 2018-12-31
filing date: 2019-03-08
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1619762/000161976219000007/0001619762-19-000007-index.htm
management discussion and analysis placement: 20-F


In [93]:
# notes on 'N/A' observations for OR
'''

OSISKO GOLD ROYALTIES is domiciled in Canada; therefore, its annual filings are form 40-F filings and not form 10-K filings
Data_Date: 2013-12-31 - report does not exist
Data_Date: 2016-12-31 - report is of type '40-F'
Data_Date: 2017-12-31 - report is of type '40-F'
Data_Date: 2018-12-31 - report is of type '40-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '40-F' parameter (for obtainable filings)

'''
# run functions again for 'N/A' observations for OR
# note: 'N/A' observations for OR are located in indexes 1399 through to 1401 of filing_page_df
for idx in range(1399, 1402):
    # retrieve the filing date for OR observations
    if idx==1399:
        filing_page_df.loc[idx, 'Filing_Date'] = '2017-03-27'
    else:
        filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '40-F'),
                                                           ['40-F'])
    # set the filing type for OR observations to '40-F'
    filing_page_df.loc[idx, 'Filing_Type'] = '40-F'
    # retrieve the filing url for OR observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '40-F'),
                                                          ['40-F'])
    # set the filing mda placement for OR observations to 'EX-99.3' (as the mda section is contained in a separate exhibit)
    filing_page_df.loc[idx, 'MDA_Placement'] = 'EX-99.3'
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

-------------------------------------------------------------------------------------
OR
period of report: 2016-12-31
filing date: 2017-03-27
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/1627272/000106299317001498/0001062993-17-001498-index.htm
management discussion and analysis placement: EX-99.3


-------------------------------------------------------------------------------------
OR
period of report: 2017-12-31
filing date: 2018-03-29
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/1627272/000106299318001395/0001062993-18-001395-index.htm
management discussion and analysis placement: EX-99.3


-------------------------------------------------------------------------------------
OR
period of report: 2018-12-31
filing date: 2019-03-29
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/1627272/000106299319001469/0001062993-19-001469-index.htm
management discussion and analysis placement: EX-99.3




In [94]:
# notes on 'N/A' observation for BBU
'''

BROOKFIELD BUSINESS PRTNR LP is classed as a 'foreign private issuer'; therefore, its annual filings are form 20-F filings and form 10-K filings
Data_Date: 2018-12-31 - report is of type '20-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '20-F' parameter

'''
# run functions again for 'N/A' observation for BBU
# note: 'N/A' observation for BBU is located in index 1412 of filing_page_df
# retrieve the filing date for BBU observation
filing_page_df.loc[1412, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[1412, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[1412, 'CIK'], '20-F'),
                                                           ['20-F'])
# set the filing type for BBU observation to '20-F'
filing_page_df.loc[1412, 'Filing_Type'] = '20-F'
# retrieve the filing url for BBU observation
filing_page_df.loc[1412, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[1412, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[1412, 'CIK'], '20-F'),
                                                          ['20-F'])
# set the filing mda placement for BBU observations to Filing_Type value
filing_page_df.loc[1412, 'MDA_Placement'] = filing_page_df.loc[1412, 'Filing_Type']
# provide user feedback
print('-------------------------------------------------------------------------------------')
print(filing_page_df.loc[1412, 'Ticker_Symbol'])
print('period of report:', filing_page_df.loc[1412, 'Data_Date'].strftime('%Y-%m-%d'))
print('filing date:', filing_page_df.loc[1412, 'Filing_Date'])
print('filing type:', filing_page_df.loc[1412, 'Filing_Type'])
print('filing url:', filing_page_df.loc[1412, 'Filing_Url'])
print('management discussion and analysis placement:', filing_page_df.loc[1412, 'MDA_Placement'])

-------------------------------------------------------------------------------------
BBU
period of report: 2018-12-31
filing date: 2019-03-18
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1654795/000162828019003025/0001628280-19-003025-index.htm
management discussion and analysis placement: 20-F


In [95]:
# notes on 'N/A' observations for FTS
'''

FORTIS INC is domiciled in Canada; therefore, its annual filings are form 40-F filings and not form 10-K filings
Data_Date: 2012-12-31 - report does not exist
Data_Date: 2013-12-31 - report does not exist
Data_Date: 2014-12-31 - report does not exist
Data_Date: 2015-12-31 - report does not exist
Data_Date: 2016-12-31 - report is of type '40-F'
Data_Date: 2017-12-31 - report is of type '40-F'
Data_Date: 2018-12-31 - report is of type '40-F'
solution - run edgar_filing_date function and edgar_filing_page function again with '40-F' parameter (for obtainable filings)

'''
# run functions again for 'N/A' observations for FTS
# note: 'N/A' observations for FTS are located in indexes 1418 through to 1420 of filing_page_df
for idx in range(1418, 1421):
    # retrieve the filing date for FTS observations
    if idx==1418:
        filing_page_df.loc[idx, 'Filing_Date'] = '2017-02-16'
    else:
        filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '40-F'),
                                                           ['40-F'])
    # set the filing type for FTS observations to '40-F'
    filing_page_df.loc[idx, 'Filing_Type'] = '40-F'
    # retrieve the filing url for FTS observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '40-F'),
                                                          ['40-F'])
    # set the filing mda placement for FTS observations to 'EX-99.3' (as the mda section is contained in a separate exhibit)
    filing_page_df.loc[idx, 'MDA_Placement'] = 'EX-99.3'
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

-------------------------------------------------------------------------------------
FTS
period of report: 2016-12-31
filing date: 2017-02-16
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/1666175/000166617517000006/0001666175-17-000006-index.htm
management discussion and analysis placement: EX-99.3


-------------------------------------------------------------------------------------
FTS
period of report: 2017-12-31
filing date: 2018-02-15
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/1666175/000166617518000005/0001666175-18-000005-index.htm
management discussion and analysis placement: EX-99.3


-------------------------------------------------------------------------------------
FTS
period of report: 2018-12-31
filing date: 2019-02-15
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/1666175/000166617519000011/0001666175-19-000011-index.htm
management discussion and analysis placement: EX-99.3




In [96]:
%%capture
# notes on 'N/A' observation for ASH
'''

ASHLAND GLOBAL HOLDINGS INC is domiciled in US; therefore, its annual filings form 10-K filings
Data_Date: 2011-09-30 - report does not exist
solution - UNSOLVED

'''

In [97]:
%%capture
# notes on 'N/A' observation for DXC
'''

DXC TECHNOLOGY CO is domiciled in US; therefore, its annual filings form 10-K filings
Data_Date: 2011-03-31 - report does not exist
solution - UNSOLVED

'''

In [98]:
# notes on 'N/A' observations for DK
'''

financial information for DELEK US HOLDINGS INC correponds to its subsidiary, DELEK US ENERGY INC
DELEK US ENERGY INC is domiciled in US; therefore, its annual filings form 10-K filings
Data_Date: 2011-12-31 - report is of type '10-K'
Data_Date: 2015-12-31 - report is of type '10-K'
solution - change necessary information to correspond to DELEK US ENERGY INC and run edgar_filing_date function and edgar_filing_page function again with '10-K' parameter

'''
# change CIK for indexes 1427 and 1428 to 1351541 in filing_page_df
# change Company_Name for indexes 1427 and 1428 to DELEK US ENERGY INC
# run functions again for 'N/A' observations for DK
for idx in range(1427, 1429):
    filing_page_df.loc[idx, 'CIK'] = 1351541
    filing_page_df.loc[idx, 'Company_Name'] = 'DELEK US ENERGY INC'
    # retrieve the filing date for DK observations
    filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '10-K'),
                                                           ['10-K'])
    # set the filing type for DK observations to '10-K'
    filing_page_df.loc[idx, 'Filing_Type'] = '10-K'
    # retrieve the filing url for DK observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '10-K'),
                                                          ['10-K'])
    # set the filing mda placement for DK observations to Filing_Type value
    filing_page_df.loc[idx, 'MDA_Placement'] = filing_page_df.loc[idx, 'Filing_Type']
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

-------------------------------------------------------------------------------------
DK
period of report: 2011-12-31
filing date: 2012-03-14
filing type: 10-K
filing url: https://www.sec.gov/Archives/edgar/data/1351541/000144530512000666/0001445305-12-000666-index.htm
management discussion and analysis placement: 10-K


-------------------------------------------------------------------------------------
DK
period of report: 2015-12-31
filing date: 2016-02-29
filing type: 10-K
filing url: https://www.sec.gov/Archives/edgar/data/1351541/000135154116000179/0001351541-16-000179-index.htm
management discussion and analysis placement: 10-K




In [99]:
%%capture
# notes on 'N/A' observations for KL
'''

KIRKLAND LAKE GOLD LTD is domiciled in Canada; therefore, its annual filings are form 40-F filings and not form 10-K filings
Data_Date: 2013-04-30 - report does not exist
Data_Date: 2014-04-30 - report does not exist
soultion - UNSOLVED

'''

In [100]:
# notes on 'N/A' observations for HPR
'''

HIGHPOINT RESOURCES CORP is domiciled in US; therefore, its annual filings form 10-K filings
HIGHPOINT RESOURCES CORP was formerly known as BILL BARRETT CORP (pre March 2018)
Data_Date: 2012-12-31 - report is of type '10-K'; corresponds to BILL BARRETT CORP
Data_Date: 2014-12-31 - report is of type '10-K'; corresponds to BILL BARRETT CORP
Data_Date: 2015-12-31 - report is of type '10-K'; corresponds to BILL BARRETT CORP
Data_Date: 2016-12-31 - report is of type '10-K'; corresponds to BILL BARRETT CORP
Data_Date: 2017-12-31 - report is of type '10-K'; corresponds to BILL BARRETT CORP
Data_Date: 2018-12-31 - report is of type '10-K'; corresponds to HIGHPOINT RESOURCES CORP
solution - change necessary information to correspond to each company name and run edgar_filing_date function and edgar_filing_page function again with '10-K' parameter

'''
# change CIK for indexes 1431 through to 1435 to correspond to BILL BARRETT CORP cik number in filing_page_df
# change Company_Name for indexes 1431 through to 1435 to BILL BARRETT CORP in filing_page_df
# run functions again for 'N/A' observations for indexes 1431 through to 1435
for idx in range(1431, 1436):
    filing_page_df.loc[idx, 'CIK'] = 1172139
    filing_page_df.loc[idx, 'Company_Name'] = 'BILL BARRETT CORP'
    # retrieve the filing date for HPR observations
    filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '10-K'),
                                                           ['10-K'])
    # set the filing type for HPR observations to '10-K'
    filing_page_df.loc[idx, 'Filing_Type'] = '10-K'
    # retrieve the filing url for HPR observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '10-K'),
                                                          ['10-K'])
    # Set the filing mda placement for HPR observations to Filing_Type value
    filing_page_df.loc[idx, 'MDA_Placement'] = filing_page_df.loc[idx, 'Filing_Type']
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

# run function again for 'N/A' observation for index 1436
# retrieve the filing date for HPR observation
filing_page_df.loc[1436, 'Filing_Date'] = '2019-02-26'
# set the filing type for HPR observation to '10-K'
filing_page_df.loc[1436, 'Filing_Type'] = '10-K'
# retrieve the filing url for HPR observation
filing_page_df.loc[1436, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[1436, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[1436, 'CIK'], '10-K'),
                                                          ['10-K'])
# set the filing mda placement for HPR observations to Filing_Type value
filing_page_df.loc[1436, 'MDA_Placement'] = filing_page_df.loc[1436, 'Filing_Type']
# provide user feedback
print('-------------------------------------------------------------------------------------')
print(filing_page_df.loc[1436, 'Ticker_Symbol'])
print('period of report:', filing_page_df.loc[1436, 'Data_Date'].strftime('%Y-%m-%d'))
print('filing date:', filing_page_df.loc[1436, 'Filing_Date'])
print('filing type:', filing_page_df.loc[1436, 'Filing_Type'])
print('filing url:', filing_page_df.loc[1436, 'Filing_Url'])
print('management discussion and analysis placement:', filing_page_df.loc[1436, 'MDA_Placement'])

-------------------------------------------------------------------------------------
HPR
period of report: 2012-12-31
filing date: 2013-02-22
filing type: 10-K
filing url: https://www.sec.gov/Archives/edgar/data/1172139/000119312513070731/0001193125-13-070731-index.htm
management discussion and analysis placement: 10-K


-------------------------------------------------------------------------------------
HPR
period of report: 2014-12-31
filing date: 2015-02-27
filing type: 10-K
filing url: https://www.sec.gov/Archives/edgar/data/1172139/000117213915000046/0001172139-15-000046-index.htm
management discussion and analysis placement: 10-K


-------------------------------------------------------------------------------------
HPR
period of report: 2015-12-31
filing date: 2016-03-02
filing type: 10-K
filing url: https://www.sec.gov/Archives/edgar/data/1172139/000117213916000132/0001172139-16-000132-index.htm
management discussion and analysis placement: 10-K


----------------------------

In [101]:
# notes on 'N/A' observation for NTR
'''

NUTRIEN LTD is domiciled in Canada; therefore, its annual filings are form 40-F filings and not form 10-K filings
Data_Date: 2018-12-31 - report is of type '40-F'
solution - add filing date to filing_page_df manually for NTR observation, and run edgar_filing_page function again with '40-F' parameter

'''
# run functions again for 'N/A' observation for NTR
# note: 'N/A' observation for NTR is located in index 1437 of filing_page_df
# retrieve the filing date for NTR observation
filing_page_df.loc[1437, 'Filing_Date'] = '2019-02-27'
# set the filing type for NTR observation to '40-F'
filing_page_df.loc[1437, 'Filing_Type'] = '40-F'
# retrieve the filing url for NTR observation
filing_page_df.loc[1437, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[1437, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[1437, 'CIK'], '40-F'),
                                                          ['40-F'])
# set the filing mda placement for NTR observations to 'EX-99.2' (as the mda section is contained in a separate exhibit)
filing_page_df.loc[1437, 'MDA_Placement'] = 'EX-99.2'
# provide user feedback
print('-------------------------------------------------------------------------------------')
print(filing_page_df.loc[1437, 'Ticker_Symbol'])
print('period of report:', filing_page_df.loc[1437, 'Data_Date'].strftime('%Y-%m-%d'))
print('filing date:', filing_page_df.loc[1437, 'Filing_Date'])
print('filing type:', filing_page_df.loc[1437, 'Filing_Type'])
print('filing url:', filing_page_df.loc[1437, 'Filing_Url'])
print('management discussion and analysis placement:', filing_page_df.loc[1437, 'MDA_Placement'])

-------------------------------------------------------------------------------------
NTR
period of report: 2018-12-31
filing date: 2019-02-27
filing type: 40-F
filing url: https://www.sec.gov/Archives/edgar/data/1725964/000119312519053855/0001193125-19-053855-index.htm
management discussion and analysis placement: EX-99.2


In [102]:
# notes on 'N/A' observations for BH
'''

BIGLARI HOLDINGS INC is domiciled in US; therefore, its annual filings form 10-K filings
BIGLARI HOLDINGS INC was formerly known as OBH INC (pre April 2018)
Data_Date: 2011-09-30 - report is of type '10-K'; corresponds to OBH INC
Data_Date: 2012-09-30 - report is of type '10-K'; corresponds to OBH INC
Data_Date: 2013-09-30 - report is of type '10-K'; corresponds to OBH INC
Data_Date: 2014-09-30 - report is of type '10-K'; corresponds to OBH INC
Data_Date: 2015-12-31 - report is of type '10-K'; corresponds to OBH INC (change in issue month)
Data_Date: 2016-12-31 - report is of type '10-K'; corresponds to OBH INC
Data_Date: 2017-12-31 - report is of type '10-K'; corresponds to OBH INC
Data_Date: 2018-12-31 - report is of type '10-K'; corresponds to BIGLARI HOLDINGS INC
solution - change necessary information to correspond to each company name and run edgar_filing_date function and edgar_filing_page function again with '10-K' parameter

'''
# change CIK for indexes 1438 through to 1444 to correspond to OBH INC cik number in filing_page_df
# change Company_Name for indexes 1438 through to 1444 to OBH INC in filing_page_df
# run functions again for 'N/A' observations for indexes 1438 through to 1444
for idx in range(1438, 1445):
    filing_page_df.loc[idx, 'CIK'] = 93859
    filing_page_df.loc[idx, 'Company_Name'] = 'OBH INC'
    # retrieve the filing date for BH observations
    filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '10-K'),
                                                           ['10-K', '10-KT'])
    # set the filing type for BH observations to '10-K'
    filing_page_df.loc[idx, 'Filing_Type'] = '10-K'
    # retrieve the filing url for BH observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '10-K'),
                                                          ['10-K', '10-KT'])
    # Set the filing mda placement for BH observations to Filing_Type value
    filing_page_df.loc[idx, 'MDA_Placement'] = filing_page_df.loc[idx, 'Filing_Type']
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

# run function again for 'N/A' observation for index 1445
# retrieve the filing date for BH observation
filing_page_df.loc[1445, 'Filing_Date'] = '2019-02-25'
# set the filing type for BH observation to '10-K'
filing_page_df.loc[1445, 'Filing_Type'] = '10-K'
# retrieve the filing url for BH observation
filing_page_df.loc[1445, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[1445, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[1445, 'CIK'], '10-K'),
                                                          ['10-K'])
# set the filing mda placement for BH observations to Filing_Type value
filing_page_df.loc[1445, 'MDA_Placement'] = filing_page_df.loc[1445, 'Filing_Type']
# provide user feedback
print('-------------------------------------------------------------------------------------')
print(filing_page_df.loc[1445, 'Ticker_Symbol'])
print('period of report:', filing_page_df.loc[1445, 'Data_Date'].strftime('%Y-%m-%d'))
print('filing date:', filing_page_df.loc[1445, 'Filing_Date'])
print('filing type:', filing_page_df.loc[1445, 'Filing_Type'])
print('filing url:', filing_page_df.loc[1445, 'Filing_Url'])
print('management discussion and analysis placement:', filing_page_df.loc[1445, 'MDA_Placement'])

-------------------------------------------------------------------------------------
BH
period of report: 2011-09-30
filing date: 2011-12-12
filing type: 10-K
filing url: https://www.sec.gov/Archives/edgar/data/93859/000119380511002350/0001193805-11-002350-index.htm
management discussion and analysis placement: 10-K


-------------------------------------------------------------------------------------
BH
period of report: 2012-09-30
filing date: 2012-12-10
filing type: 10-K
filing url: https://www.sec.gov/Archives/edgar/data/93859/000119380512001930/0001193805-12-001930-index.htm
management discussion and analysis placement: 10-K


-------------------------------------------------------------------------------------
BH
period of report: 2013-09-30
filing date: 2013-12-09
filing type: 10-K
filing url: https://www.sec.gov/Archives/edgar/data/93859/000152153613001025/0001521536-13-001025-index.htm
management discussion and analysis placement: 10-K


-------------------------------------

In [103]:
# notes on 'N/A' observations for WRK
'''

WESTROCK CO is domiciled in US; therefore, its annual filings form 10-K filings
WESTROCK CO was formerly known as WRKCO INC (pre 2018)
ROCK-TENN CO merged with MEADWESTVACO CORP to form WRKCO INC (pre June 2015)
Data_Date: 2011-09-30 - report is of type '10-K'; corresponds to ROCK-TENN CO
Data_Date: 2015-09-30 - report is of type '10-K'; corresponds to WRKCO INC
solution - change necessary information to correspond to each company name and run edgar_filing_date function and edgar_filing_page function again with '10-K' parameter

'''
# change CIK for index 1446 to correspond to ROCK-TENN CO cik number in filing_page_df
# change Company_Name for index 1446 to ROCK-TENN CO in filing_page_df
# run function again for 'N/A' observation for index 1446
# retrieve the filing date for WRK observation
filing_page_df.loc[1446, 'CIK'] = 230498
filing_page_df.loc[1446, 'Company_Name'] = 'ROCK-TENN CO'
# set Filing_Date in index 1446 manually
filing_page_df.loc[1446, 'Filing_Date'] = '2011-11-23'
# set the filing type for WRK observation to '10-K'
filing_page_df.loc[1446, 'Filing_Type'] = '10-K'
# retrieve the filing url for WRK observation
filing_page_df.loc[1446, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[1446, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[1446, 'CIK'], '10-K'),
                                                          ['10-K'])
# set the filing mda placement for WRK observations to Filing_Type value
filing_page_df.loc[1446, 'MDA_Placement'] = filing_page_df.loc[1446, 'Filing_Type']
# provide user feedback
print('-------------------------------------------------------------------------------------')
print(filing_page_df.loc[1446, 'Ticker_Symbol'])
print('period of report:', filing_page_df.loc[1446, 'Data_Date'].strftime('%Y-%m-%d'))
print('filing date:', filing_page_df.loc[1446, 'Filing_Date'])
print('filing type:', filing_page_df.loc[1446, 'Filing_Type'])
print('filing url:', filing_page_df.loc[1446, 'Filing_Url'])
print('management discussion and analysis placement:', filing_page_df.loc[1446, 'MDA_Placement'])
print('\n')

# change CIK for index 1447 to correspond to WRKCO INC cik number in filing_page_df
# change Company_Name for index 1447 to WRKCO INC in filing_page_df
# run function again for 'N/A' observation for index 1447
# retrieve the filing date for WRK observation
filing_page_df.loc[1447, 'CIK'] = 1636023
filing_page_df.loc[1447, 'Company_Name'] = 'WRKCO INC'
# set Filing_Date in index 1446 manually
filing_page_df.loc[1447, 'Filing_Date'] = '2015-11-27'
# set the filing type for WRK observation to '10-K'
filing_page_df.loc[1447, 'Filing_Type'] = '10-K'
# retrieve the filing url for WRK observation
filing_page_df.loc[1447, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[1447, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[1447, 'CIK'], '10-K'),
                                                          ['10-K'])
# set the filing mda placement for WRK observations to Filing_Type value
filing_page_df.loc[1447, 'MDA_Placement'] = filing_page_df.loc[1447, 'Filing_Type']
# provide user feedback
print('-------------------------------------------------------------------------------------')
print(filing_page_df.loc[1447, 'Ticker_Symbol'])
print('period of report:', filing_page_df.loc[1447, 'Data_Date'].strftime('%Y-%m-%d'))
print('filing date:', filing_page_df.loc[1447, 'Filing_Date'])
print('filing type:', filing_page_df.loc[1447, 'Filing_Type'])
print('filing url:', filing_page_df.loc[1447, 'Filing_Url'])
print('management discussion and analysis placement:', filing_page_df.loc[1447, 'MDA_Placement'])

-------------------------------------------------------------------------------------
WRK
period of report: 2011-09-30
filing date: 2011-11-23
filing type: 10-K
filing url: https://www.sec.gov/Archives/edgar/data/230498/000023049811000097/0000230498-11-000097-index.htm
management discussion and analysis placement: 10-K


-------------------------------------------------------------------------------------
WRK
period of report: 2015-09-30
filing date: 2015-11-27
filing type: 10-K
filing url: https://www.sec.gov/Archives/edgar/data/1636023/000163602315000208/0001636023-15-000208-index.htm
management discussion and analysis placement: 10-K


In [104]:
# notes on 'N/A' observations for SDRL
'''

SEADRILL LTD is classed as a 'foreign private issuer'; therefore, its annual filings are form 20-F filings and form 10-K filings
SEADRILL LTD cik number was changed pre May 2018
Data_Date: 2014-12-31 - report is of type '20-F'
Data_Date: 2015-12-31 - report is of type '20-F'
Data_Date: 2016-12-31 - report is of type '20-F'
Data_Date: 2017-12-31 - report is of type '20-F'
Data_Date: 2018-12-31 - report is of type '20-F'
solution - change necessary information to correspond to correspond to correct cik number and run edgar_filing_date function and edgar_filing_page function again with '20-F' parameter

'''
# change CIK for indexes 1449 through to 1452 to correspond to old SEADRILL LTD cik number in filing_page_df
# run functions again for 'N/A' observations for SDRL
# note: 'N/A' observations for SDRL are located in indexes 1449 and 1452 of filing_page_df
for idx in range(1449, 1453):
    filing_page_df.loc[idx, 'CIK'] = 1351413
    # retrieve the filing date for SDRL observations
    filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                           ['20-F'])
    # set the filing type for SDRL observations to '20-F'
    filing_page_df.loc[idx, 'Filing_Type'] = '20-F'
    # retrieve the filing url for SDRL observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '20-F'),
                                                          ['20-F'])
    # set the filing mda placement for SDRL observations to Filing_Type value
    filing_page_df.loc[idx, 'MDA_Placement'] = filing_page_df.loc[idx, 'Filing_Type']
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

# run functions again for 'N/A' observation for SDRL
# note: 'N/A' observation for SDRL is located in index 1453 of filing_page_df
# retrieve the filing date for SDRL observation
# set Filing_Date manually
filing_page_df.loc[1453, 'Filing_Date'] = '2019-03-28'
# set the filing type for SDRL observation to '20-F'
filing_page_df.loc[1453, 'Filing_Type'] = '20-F'
# retrieve the filing url for SDRL observation
filing_page_df.loc[1453, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[1453, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[1453, 'CIK'], '20-F'),
                                                          ['20-F'])
# set the filing mda placement for SDRL observations to Filing_Type value
filing_page_df.loc[1453, 'MDA_Placement'] = filing_page_df.loc[1453, 'Filing_Type']
# provide user feedback
print('-------------------------------------------------------------------------------------')
print(filing_page_df.loc[1453, 'Ticker_Symbol'])
print('period of report:', filing_page_df.loc[1453, 'Data_Date'].strftime('%Y-%m-%d'))
print('filing date:', filing_page_df.loc[1453, 'Filing_Date'])
print('filing type:', filing_page_df.loc[1453, 'Filing_Type'])
print('filing url:', filing_page_df.loc[1453, 'Filing_Url'])
print('management discussion and analysis placement:', filing_page_df.loc[1453, 'MDA_Placement'])

-------------------------------------------------------------------------------------
SDRL
period of report: 2014-12-31
filing date: 2015-04-21
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1351413/000135141315000005/0001351413-15-000005-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
SDRL
period of report: 2015-12-31
filing date: 2016-04-28
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1351413/000135141316000008/0001351413-16-000008-index.htm
management discussion and analysis placement: 20-F


-------------------------------------------------------------------------------------
SDRL
period of report: 2016-12-31
filing date: 2017-04-27
filing type: 20-F
filing url: https://www.sec.gov/Archives/edgar/data/1351413/000135141317000004/0001351413-17-000004-index.htm
management discussion and analysis placement: 20-F


-------------------------

In [105]:
# notes on 'N/A' observations for XRX
'''

XEROX HOLDINGS CORP is domiciled in US; therefore, its annual filings form 10-K filings
financial data matches that of XEROX CORP
Data_Date: 2011-12-31 - report is of type '10-K'; corresponds to XEROX CORP
Data_Date: 2012-12-31 - report is of type '10-K'; corresponds to XEROX CORP
Data_Date: 2013-12-31 - report is of type '10-K'; corresponds to XEROX CORP
Data_Date: 2015-12-31 - report is of type '10-K'; corresponds to XEROX CORP
Data_Date: 2018-12-31 - report is of type '10-K'; corresponds to XEROX CORP
solution - change necessary information to correspond to correspond to XEROX CORP and run edgar_filing_date function and edgar_filing_page function again with '20-F' parameter

'''
# change CIK for indexes 1454 through to 1458 to correspond to XEROX CORP cik number in filing_page_df
# change Company_Name for indexes 1454 through to 1458 to XEROX CORP in filing_page_df
# run functions again for 'N/A' observations for indexes 1454 through to 1458
for idx in range(1454, 1459):
    filing_page_df.loc[idx, 'CIK'] = 108772
    filing_page_df.loc[idx, 'Company_Name'] = 'XEROX CORP'
    # retrieve the filing date for XRX observations
    filing_page_df.loc[idx, 'Filing_Date'] = edgar_filing_date(filing_page_df.loc[idx, 'Data_Date'], 
                                                           edgar_search(filing_page_df.loc[idx, 'CIK'], '10-K'),
                                                           ['10-K'])
    # set the filing type for XRX observations to '10-K'
    filing_page_df.loc[idx, 'Filing_Type'] = '10-K'
    # retrieve the filing url for XRX observations
    filing_page_df.loc[idx, 'Filing_Url'] = edgar_filing_page(filing_page_df.loc[idx, 'Filing_Date'],
                                                          edgar_search(filing_page_df.loc[idx, 'CIK'], '10-K'),
                                                          ['10-K'])
    # Set the filing mda placement for XRX observations to Filing_Type value
    filing_page_df.loc[idx, 'MDA_Placement'] = filing_page_df.loc[idx, 'Filing_Type']
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('filing url:', filing_page_df.loc[idx, 'Filing_Url'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

-------------------------------------------------------------------------------------
XRX
period of report: 2011-12-31
filing date: 2012-02-23
filing type: 10-K
filing url: https://www.sec.gov/Archives/edgar/data/108772/000144530512000425/0001445305-12-000425-index.htm
management discussion and analysis placement: 10-K


-------------------------------------------------------------------------------------
XRX
period of report: 2012-12-31
filing date: 2013-02-21
filing type: 10-K
filing url: https://www.sec.gov/Archives/edgar/data/108772/000010877213000006/0000108772-13-000006-index.htm
management discussion and analysis placement: 10-K


-------------------------------------------------------------------------------------
XRX
period of report: 2013-12-31
filing date: 2014-02-21
filing type: 10-K
filing url: https://www.sec.gov/Archives/edgar/data/108772/000010877214000005/0000108772-14-000005-index.htm
management discussion and analysis placement: 10-K


-------------------------------

## Check Filing Page URLs Correspond to Correct Filings

In [106]:
# time
start = time.time()

# note: check if the filing urls that have been retrieved correspond the correct filing
# note: each filing page contains a 'Period of Report' date; this can be cross-referenced with Data_Date in filing_page_df to check if the page corresponds to the correct filing

# create new column (Filing_Check) in filing_page_df for checking if Data_Date is equal to 'Period of Report' date on filing page
filing_page_df['Filing_Check'] = ''
# if Data_Date is equal to 'Period of Report' date then set Filing_Check to 'True', else set to 'False'
for idx, row in filing_page_df.iterrows():
    # set Filing_Check to 'N/A' for observations that have not been retrieved (i.e. observations with 'N/A' Filing_Url) 
    if row['Filing_Url']=='N/A':
        filing_page_df.loc[idx, 'Filing_Check'] = 'N/A'
    else:
        # retrieve 'Period of Report' date from filing page
        filing_url = row['Filing_Url']
        filing_page_request = requests.get(url=filing_url)
        filing_page_soup = BeautifulSoup(filing_page_request.content, 'html.parser')
        period_of_report = filing_page_soup.find_all('div', class_='formGrouping')[1].find_all('div', class_='info')[0].text.strip()
        if row['Data_Date'].strftime('%Y-%m-%d')==period_of_report:
            filing_page_df.loc[idx, 'Filing_Check'] = 'True'
        else:
            filing_page_df.loc[idx, 'Filing_Check'] = 'False'
    # provide user feedback 
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing page period of report:', period_of_report)
    print(filing_page_df.loc[idx, 'Filing_Check'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

end = time.time()
print('time taken:', end - start)

-------------------------------------------------------------------------------------
AIR
period of report: 2012-05-31
filing page period of report: 2012-05-31
True


-------------------------------------------------------------------------------------
AIR
period of report: 2013-05-31
filing page period of report: 2013-05-31
True


-------------------------------------------------------------------------------------
AIR
period of report: 2014-05-31
filing page period of report: 2014-05-31
True


-------------------------------------------------------------------------------------
AIR
period of report: 2016-05-31
filing page period of report: 2016-05-31
True


-------------------------------------------------------------------------------------
AIR
period of report: 2019-05-31
filing page period of report: 2019-05-31
True


-------------------------------------------------------------------------------------
HES
period of report: 2011-12-31
filing page period of report: 2011-12-31
True


-------------------------------------------------------------------------------------
CACI
period of report: 2014-06-30
filing page period of report: 2014-06-30
True


-------------------------------------------------------------------------------------
LUB
period of report: 2011-08-31
filing page period of report: 2011-08-31
True


-------------------------------------------------------------------------------------
LUB
period of report: 2012-08-31
filing page period of report: 2012-08-29
False


-------------------------------------------------------------------------------------
LUB
period of report: 2013-08-31
filing page period of report: 2013-08-28
False


-------------------------------------------------------------------------------------
LUB
period of report: 2014-08-31
filing page period of report: 2014-08-27
False


-------------------------------------------------------------------------------------
LUB
period of report: 2015-08-31
filing page period of report: 2015-08-26
F

-------------------------------------------------------------------------------------
DCO
period of report: 2014-12-31
filing page period of report: 2014-12-31
True


-------------------------------------------------------------------------------------
DCO
period of report: 2015-12-31
filing page period of report: 2015-12-31
True


-------------------------------------------------------------------------------------
KODK
period of report: 2017-12-31
filing page period of report: 2017-12-31
True


-------------------------------------------------------------------------------------
KODK
period of report: 2018-12-31
filing page period of report: 2018-12-31
True


-------------------------------------------------------------------------------------
EBF
period of report: 2011-02-28
filing page period of report: 2011-02-28
True


-------------------------------------------------------------------------------------
EBF
period of report: 2012-02-29
filing page period of report: 2012-02-29
Tru

-------------------------------------------------------------------------------------
THC
period of report: 2016-12-31
filing page period of report: 2016-12-31
True


-------------------------------------------------------------------------------------
THC
period of report: 2017-12-31
filing page period of report: 2017-12-31
True


-------------------------------------------------------------------------------------
THC
period of report: 2018-12-31
filing page period of report: 2018-12-31
True


-------------------------------------------------------------------------------------
GPX
period of report: 2018-12-31
filing page period of report: 2018-12-31
True


-------------------------------------------------------------------------------------
NR
period of report: 2015-12-31
filing page period of report: 2015-12-31
True


-------------------------------------------------------------------------------------
NR
period of report: 2018-12-31
filing page period of report: 2018-12-31
True




-------------------------------------------------------------------------------------
SUP
period of report: 2011-12-31
filing page period of report: 2011-12-25
False


-------------------------------------------------------------------------------------
SUP
period of report: 2012-12-31
filing page period of report: 2012-12-30
False


-------------------------------------------------------------------------------------
SUP
period of report: 2013-12-31
filing page period of report: 2013-12-29
False


-------------------------------------------------------------------------------------
SUP
period of report: 2014-12-31
filing page period of report: 2014-12-28
False


-------------------------------------------------------------------------------------
SUP
period of report: 2015-12-31
filing page period of report: 2015-12-27
False


-------------------------------------------------------------------------------------
SUP
period of report: 2017-12-31
filing page period of report: 2017-12-31


...

-------------------------------------------------------------------------------------
POST
period of report: 2014-09-30
filing page period of report: 2014-09-30
True


-------------------------------------------------------------------------------------
EROS
period of report: 2016-03-31
filing page period of report: 2016-03-31
True


-------------------------------------------------------------------------------------
EROS
period of report: 2017-03-31
filing page period of report: 2017-03-31
True


-------------------------------------------------------------------------------------
EROS
period of report: 2018-03-31
filing page period of report: 2018-03-31
True


-------------------------------------------------------------------------------------
EROS
period of report: 2019-03-31
filing page period of report: 2019-03-31
True


-------------------------------------------------------------------------------------
BEP
period of report: 2013-12-31
filing page period of report: 2013-12-31


-------------------------------------------------------------------------------------
CWEN
period of report: 2015-12-31
filing page period of report: 2015-12-31
True


-------------------------------------------------------------------------------------
CWEN
period of report: 2016-12-31
filing page period of report: 2016-12-31
True


-------------------------------------------------------------------------------------
CWEN
period of report: 2017-12-31
filing page period of report: 2017-12-31
True


-------------------------------------------------------------------------------------
CWEN
period of report: 2018-12-31
filing page period of report: 2018-12-31
True


-------------------------------------------------------------------------------------
MNK
period of report: 2017-12-31
filing page period of report: 2017-12-29
False


-------------------------------------------------------------------------------------
MNK
period of report: 2018-12-31
filing page period of report: 2018-12-28


-------------------------------------------------------------------------------------
RUBI
period of report: 2016-12-31
filing page period of report: 2016-12-31
True


-------------------------------------------------------------------------------------
RUBI
period of report: 2017-12-31
filing page period of report: 2017-12-31
True


-------------------------------------------------------------------------------------
LPG
period of report: 2017-03-31
filing page period of report: 2017-03-31
True


-------------------------------------------------------------------------------------
LPG
period of report: 2018-03-31
filing page period of report: 2018-03-31
True


-------------------------------------------------------------------------------------
LPG
period of report: 2019-03-31
filing page period of report: 2019-03-31
True


-------------------------------------------------------------------------------------
PSV
period of report: 2016-12-31
filing page period of report: 2016-12-31
Tru

-------------------------------------------------------------------------------------
JAX
period of report: 2011-12-31
filing page period of report: 2019-03-31
N/A


-------------------------------------------------------------------------------------
JAX
period of report: 2018-12-31
filing page period of report: 2018-12-30
False


-------------------------------------------------------------------------------------
QSR
period of report: 2014-12-31
filing page period of report: 2014-12-31
True


-------------------------------------------------------------------------------------
IGT
period of report: 2018-12-31
filing page period of report: 2018-12-31
True


-------------------------------------------------------------------------------------
SUM
period of report: 2018-12-31
filing page period of report: 2018-12-29
False


-------------------------------------------------------------------------------------
OR
period of report: 2013-12-31
filing page period of report: 2018-12-29
N/A



-------------------------------------------------------------------------------------
BH
period of report: 2016-12-31
filing page period of report: 2016-12-31
True


-------------------------------------------------------------------------------------
BH
period of report: 2017-12-31
filing page period of report: 2017-12-31
True


-------------------------------------------------------------------------------------
BH
period of report: 2018-12-31
filing page period of report: 2018-12-31
True


-------------------------------------------------------------------------------------
WRK
period of report: 2011-09-30
filing page period of report: 2011-09-30
True


-------------------------------------------------------------------------------------
WRK
period of report: 2015-09-30
filing page period of report: 2015-09-30
True


-------------------------------------------------------------------------------------
WRK
period of report: 2018-09-30
filing page period of report: 2018-09-30
True


-

## Investigate Observations with False Filing_Check

In [107]:
# note: to be certain the filing urls obtained correspond to the correct filings, an investigation is necessary
# note: each 'False' observation in the Filing_Check column of filing_page_df will be examined in turn

# row count of 'N/A' observations
print('filing_page_df - \'False\' row count:', 
      filing_page_df.loc[filing_page_df['Filing_Check']=='False'].shape[0])

filing_page_df - 'False' row count: 104


In [108]:
# display all 'False' observation in the Filing_Check column of filing_page_df
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(filing_page_df.loc[filing_page_df['Filing_Check']=='False'])

,CIK,Ticker_Symbol,Company_Name,Data_Date,Filing_Date,Filing_Type,Filing_Url,MDA_Placement,Filing_Check
24,6885,SSI,STAGE STORES INC,2011-01-31,2011-03-30,10-K,https://www.sec.gov/Archives/edgar/data/6885/0...,10-K,False
25,6885,SSI,STAGE STORES INC,2012-01-31,2012-03-28,10-K,https://www.sec.gov/Archives/edgar/data/6885/0...,10-K,False
26,6885,SSI,STAGE STORES INC,2016-01-31,2016-03-30,10-K,https://www.sec.gov/Archives/edgar/data/6885/0...,10-K,False
27,6885,SSI,STAGE STORES INC,2017-01-31,2017-04-04,10-K,https://www.sec.gov/Archives/edgar/data/6885/0...,10-K,False
28,6885,SSI,STAGE STORES INC,2018-01-31,2018-04-10,10-K,https://www.sec.gov/Archives/edgar/data/6885/0...,10-K,False
29,6885,SSI,STAGE STORES INC,2019-01-31,2019-04-05,10-K,https://www.sec.gov/Archives/edgar/data/6885/0...,10-K,False
45,14707,CAL,CALERES INC,2012-01-31,2012-03-27,10-K,https://www.sec.gov/Archives/edgar/data/14707/...,10-K,False
52,16099,LUB,LUBYS INC,2012-08-31,2012-11-13,10-K,https://www.sec.gov/Archives/edgar/data/16099/...,10-K,False
53,16099,LUB,LUBYS INC,2013-08-31,2013-11-12,10-K,https://www.sec.gov/Archives/edgar/data/16099/...,10-K,False
54,16099,LUB,LUBYS INC,2014-08-31,2014-11-12,10-K,https://www.sec.gov/Archives/edgar/data/16099/...,10-K,False


In [109]:
# notes on 'False' observations for SSI
# observe financial statement data in funda_df and compare to financial statement data on relevant filing page of SEC EDGAR database
'''

STAGE STORES INC
Data_Date: 2011-01-31 - filing url corresponds to correct filing (financial statement data matches)
Data_Date: 2012-01-31 - filing url corresponds to correct filing (financial statement data matches)
Data_Date: 2016-01-31 - filing url corresponds to correct filing (financial statement data matches)
Data_Date: 2017-01-31 - filing url corresponds to correct filing (financial statement data matches)
Data_Date: 2018-01-31 - filing url corresponds to correct filing (financial statement data matches)
Data_Date: 2019-01-31 - filing url corresponds to correct filing (financial statement data matches)
As the observations correspond to the correct filings, the Filing_Check column of filing_page_df can be set to 'True'

'''
# set Filing_Check to 'True' for indexes 24 through to 29
for idx in range(24, 30):
    filing_page_df.loc[idx, 'Filing_Check'] = 'True'

In [110]:
# notes on 'False' observation for CAL
# observe financial statement data in funda_df and compare to financial statement data on relevant filing page of SEC EDGAR database
'''

CALERES INC
Data_Date: 2012-01-31 - filing url corresponds to correct filing (financial statement data matches)
As the observations correspond to the correct filings, the Filing_Check column of filing_page_df can be set to 'True'

'''
# set Filing_Check to 'True' for index 45
filing_page_df.loc[45, 'Filing_Check'] = 'True'

In [111]:
# notes on 'False' observations for LUB
# observe financial statement data in funda_df and compare to financial statement data on relevant filing page of SEC EDGAR database
'''

LUBYS INC
Data_Date: 2012-08-31 - filing url corresponds to correct filing (financial statement data matches)
Data_Date: 2013-08-31 - filing url corresponds to correct filing (financial statement data matches)
Data_Date: 2014-08-31 - filing url corresponds to correct filing (financial statement data matches)
Data_Date: 2015-08-31 - filing url corresponds to correct filing (financial statement data matches)
Data_Date: 2017-08-31 - filing url corresponds to correct filing (financial statement data matches)
Data_Date: 2018-08-31 - filing url corresponds to correct filing (financial statement data matches)
As the observations correspond to the correct filings, the Filing_Check column of filing_page_df can be set to 'True'

'''
# set Filing_Check to 'True' for indexes 52 through to 58 (excluding 56)
for idx in range(52, 59):
    if idx==56:
        continue
    else:
        filing_page_df.loc[idx, 'Filing_Check'] = 'True'

In [112]:
# notes on 'False' observations for CATO
# observe financial statement data in funda_df and compare to financial statement data on relevant filing page of SEC EDGAR database
'''

CATO CORP -CL A
Data_Date: 2018-01-31 - filing url corresponds to correct filing (financial statement data matches)
Data_Date: 2019-01-31 - filing url corresponds to correct filing (financial statement data matches)
As the observations correspond to the correct filings, the Filing_Check column of filing_page_df can be set to 'True'

'''
# set Filing_Check to 'True' for indexes 59 and 60
for idx in range(59, 61):
    filing_page_df.loc[idx, 'Filing_Check'] = 'True'

In [113]:
# notes on 'False' observations for GCO
# observe financial statement data in funda_df and compare to financial statement data on relevant filing page of SEC EDGAR database
'''

GENESCO INC
Data_Date: 2018-01-31 - filing url corresponds to correct filing (financial statement data matches)
Data_Date: 2019-01-31 - filing url corresponds to correct filing (financial statement data matches)
As the observations correspond to the correct filings, the Filing_Check column of filing_page_df can be set to 'True'

'''
# set Filing_Check to 'True' for indexes 61 and 62
for idx in range(61, 63):
    filing_page_df.loc[idx, 'Filing_Check'] = 'True'

In [114]:
# notes on 'False' observation for TAP
# observe financial statement data in funda_df and compare to financial statement data on relevant filing page of SEC EDGAR database
'''

MOLSON COORS BREWING CO
Data_Date: 2012-12-31 - filing url corresponds to correct filing (financial statement data matches)
As the observations correspond to the correct filings, the Filing_Check column of filing_page_df can be set to 'True'

'''
# set Filing_Check to 'True' for index 81
filing_page_df.loc[81, 'Filing_Check'] = 'True'

In [115]:
# notes on 'False' observations for DDS
# observe financial statement data in funda_df and compare to financial statement data on relevant filing page of SEC EDGAR database
'''

DILLARDS INC -CL A
Data_Date: 2011-01-31 - filing url corresponds to correct filing (financial statement data matches)
Data_Date: 2012-01-31 - filing url corresponds to correct filing (financial statement data matches)
Data_Date: 2017-01-31 - filing url corresponds to correct filing (financial statement data matches)
Data_Date: 2018-01-31 - filing url corresponds to correct filing (financial statement data matches)
Data_Date: 2019-01-31 - filing url corresponds to correct filing (financial statement data matches)
As the observations correspond to the correct filings, the Filing_Check column of filing_page_df can be set to 'True'

'''
# set Filing_Check to 'True' for indexes 92 through to 96
for idx in range(92, 97):
    filing_page_df.loc[idx, 'Filing_Check'] = 'True'

In [116]:
# notes on 'False' observation for AXE
# observe financial statement data in funda_df and compare to financial statement data on relevant filing page of SEC EDGAR database
'''

ANIXTER INTL INC
Data_Date: 2018-12-31 - filing url corresponds to correct filing (financial statement data matches)
As the observations correspond to the correct filings, the Filing_Check column of filing_page_df can be set to 'True'

'''
# set Filing_Check to 'True' for index 127
filing_page_df.loc[127, 'Filing_Check'] = 'True'

In [117]:
# notes on 'False' observation for JEC
# observe financial statement data in funda_df and compare to financial statement data on relevant filing page of SEC EDGAR database
'''

JACOBS ENGINEERING GROUP INC
Data_Date: 2015-09-30 - filing url corresponds to correct filing (financial statement data matches)
As the observations correspond to the correct filings, the Filing_Check column of filing_page_df can be set to 'True'

'''
# set Filing_Check to 'True' for index 128
filing_page_df.loc[128, 'Filing_Check'] = 'True'

In [118]:
# notes on 'False' observations for MCS
# observe financial statement data in funda_df and compare to financial statement data on relevant filing page of SEC EDGAR database
'''

MARCUS CORP
Data_Date: 2011-08-09 - filing url corresponds to correct filing (financial statement data matches)
Data_Date: 2013-05-31 - filing url corresponds to correct filing (financial statement data matches)
As the observations correspond to the correct filings, the Filing_Check column of filing_page_df can be set to 'True'

'''
# set Filing_Check to 'True' for indexes 143 and 145
for idx in range(143, 146):
    if idx==144:
        continue
    else:
        filing_page_df.loc[idx, 'Filing_Check'] = 'True'

In [119]:
# notes on 'False' observation for RAD
# observe financial statement data in funda_df and compare to financial statement data on relevant filing page of SEC EDGAR database
'''

RITE AID CORP
Data_Date: 2019-02-28 - filing url corresponds to correct filing (financial statement data matches)
As the observations correspond to the correct filings, the Filing_Check column of filing_page_df can be set to 'True'

'''
# set Filing_Check to 'True' for index 185
filing_page_df.loc[185, 'Filing_Check'] = 'True'

In [120]:
# notes on 'False' observations for SUP
# observe financial statement data in funda_df and compare to financial statement data on relevant filing page of SEC EDGAR database
'''

SUPERIOR INDUSTRIES INTL
Data_Date: 2011-12-31 - filing url corresponds to correct filing (financial statement data matches)
Data_Date: 2012-12-31 - filing url corresponds to correct filing (financial statement data matches)
Data_Date: 2013-12-31 - filing url corresponds to correct filing (financial statement data matches)
Data_Date: 2014-12-31 - filing url corresponds to correct filing (financial statement data matches)
Data_Date: 2015-12-31 - filing url corresponds to correct filing (financial statement data matches)
As the observations correspond to the correct filings, the Filing_Check column of filing_page_df can be set to 'True'

'''
# set Filing_Check to 'True' for indexes 199 through to 203
for idx in range(199, 204):
    filing_page_df.loc[idx, 'Filing_Check'] = 'True'

In [121]:
# notes on 'False' observations for TSN
# observe financial statement data in funda_df and compare to financial statement data on relevant filing page of SEC EDGAR database
'''

TYSON FOODS INC -CL A
Data_Date: 2011-09-30 - filing url corresponds to correct filing (financial statement data matches)
Data_Date: 2012-09-30 - filing url corresponds to correct filing (financial statement data matches)
As the observations correspond to the correct filings, the Filing_Check column of filing_page_df can be set to 'True'

'''
# set Filing_Check to 'True' for indexes 217 and 218
for idx in range(217, 219):
    filing_page_df.loc[idx, 'Filing_Check'] = 'True'

In [122]:
# notes on 'False' observations for UFI
# observe financial statement data in funda_df and compare to financial statement data on relevant filing page of SEC EDGAR database
'''

UNIFI INC
Data_Date: 2011-06-30 - filing url corresponds to correct filing (financial statement data matches)
Data_Date: 2012-06-30 - filing url corresponds to correct filing (financial statement data matches)
As the observations correspond to the correct filings, the Filing_Check column of filing_page_df can be set to 'True'

'''
# set Filing_Check to 'True' for indexes 219 and 220
for idx in range(219, 221):
    filing_page_df.loc[idx, 'Filing_Check'] = 'True'

In [123]:
# notes on 'False' observation for WMK
# observe financial statement data in funda_df and compare to financial statement data on relevant filing page of SEC EDGAR database
'''

WEIS MARKETS INC
Data_Date: 2017-12-31 - filing url corresponds to correct filing (financial statement data matches)
As the observations correspond to the correct filings, the Filing_Check column of filing_page_df can be set to 'True'

'''
# set Filing_Check to 'True' for index 254
filing_page_df.loc[254, 'Filing_Check'] = 'True'

In [124]:
# notes on 'False' observation for PIR
# observe financial statement data in funda_df and compare to financial statement data on relevant filing page of SEC EDGAR database
'''

PIER 1 IMPORTS INC/DE
Data_Date: 2018-02-28 - filing url corresponds to correct filing (financial statement data matches)
As the observations correspond to the correct filings, the Filing_Check column of filing_page_df can be set to 'True'

'''
# set Filing_Check to 'True' for index 264
filing_page_df.loc[264, 'Filing_Check'] = 'True'

In [125]:
# notes on 'False' observations for SIG
# observe financial statement data in funda_df and compare to financial statement data on relevant filing page of SEC EDGAR database
'''

SIGNET JEWELERS LTD
Data_Date: 2018-01-31 - filing url corresponds to correct filing (financial statement data matches)
Data_Date: 2019-01-31 - filing url corresponds to correct filing (financial statement data matches)
As the observations correspond to the correct filings, the Filing_Check column of filing_page_df can be set to 'True'

'''
# set Filing_Check to 'True' for indexes 386 and 387
for idx in range(386, 388):
    filing_page_df.loc[idx, 'Filing_Check'] = 'True'

In [126]:
# notes on 'False' observations for ACM
# observe financial statement data in funda_df and compare to financial statement data on relevant filing page of SEC EDGAR database
'''

AECOM
Data_Date: 2015-09-30 - filing url corresponds to correct filing (financial statement data matches)
Data_Date: 2018-09-30 - filing url corresponds to correct filing (financial statement data matches)
As the observations correspond to the correct filings, the Filing_Check column of filing_page_df can be set to 'True'

'''
# set Filing_Check to 'True' for indexes 437 and 438
for idx in range(437, 439):
    filing_page_df.loc[idx, 'Filing_Check'] = 'True'

In [127]:
# notes on 'False' observations for CNR
# observe financial statement data in funda_df and compare to financial statement data on relevant filing page of SEC EDGAR database
'''

CORNERSTNE BULDNG BRNDS INC
Data_Date: 2011-10-31 - filing url corresponds to correct filing (financial statement data matches)
Data_Date: 2012-10-31 - filing url corresponds to correct filing (financial statement data matches)
As the observations correspond to the correct filings, the Filing_Check column of filing_page_df can be set to 'True'

'''
# set Filing_Check to 'True' for indexes 445 and 446
for idx in range(445, 447):
    filing_page_df.loc[idx, 'Filing_Check'] = 'True'

In [128]:
# notes on 'False' observation for COT
# observe financial statement data in funda_df and compare to financial statement data on relevant filing page of SEC EDGAR database
'''

COTT CORP QUE
Data_Date: 2014-12-31 - filing url corresponds to correct filing (financial statement data matches)
As the observations correspond to the correct filings, the Filing_Check column of filing_page_df can be set to 'True'

'''
# set Filing_Check to 'True' for index 450
filing_page_df.loc[450, 'Filing_Check'] = 'True'

In [129]:
# notes on 'False' observation for KAI
# observe financial statement data in funda_df and compare to financial statement data on relevant filing page of SEC EDGAR database
'''

KADANT INC
Data_Date: 2012-12-31 - filing url corresponds to correct filing (financial statement data matches)
As the observations correspond to the correct filings, the Filing_Check column of filing_page_df can be set to 'True'

'''
# set Filing_Check to 'True' for index 458
filing_page_df.loc[458, 'Filing_Check'] = 'True'

In [130]:
# notes on 'False' observation for CHS
# observe financial statement data in funda_df and compare to financial statement data on relevant filing page of SEC EDGAR database
'''

CHICOS FAS INC
Data_Date: 2019-01-31 - filing url corresponds to correct filing (financial statement data matches)
As the observations correspond to the correct filings, the Filing_Check column of filing_page_df can be set to 'True'

'''
# set Filing_Check to 'True' for index 489
filing_page_df.loc[489, 'Filing_Check'] = 'True'

In [131]:
# notes on 'False' observation for GES
# observe financial statement data in funda_df and compare to financial statement data on relevant filing page of SEC EDGAR database
'''

GUESS INC
Data_Date: 2017-01-31 - filing url corresponds to correct filing (financial statement data matches)
As the observations correspond to the correct filings, the Filing_Check column of filing_page_df can be set to 'True'

'''
# set Filing_Check to 'True' for index 503
filing_page_df.loc[503, 'Filing_Check'] = 'True'

In [132]:
# notes on 'False' observation for DAR
# observe financial statement data in funda_df and compare to financial statement data on relevant filing page of SEC EDGAR database
'''

DARLING INGREDIENTS INC
Data_Date: 2015-12-31 - filing url corresponds to correct filing (financial statement data matches)
As the observations correspond to the correct filings, the Filing_Check column of filing_page_df can be set to 'True'

'''
# set Filing_Check to 'True' for index 507
filing_page_df.loc[507, 'Filing_Check'] = 'True'

In [133]:
# notes on 'False' observation for GEO
# observe financial statement data in funda_df and compare to financial statement data on relevant filing page of SEC EDGAR database
'''

GEO GROUP INC
Data_Date: 2011-12-31 - filing url corresponds to correct filing (financial statement data matches)
As the observations correspond to the correct filings, the Filing_Check column of filing_page_df can be set to 'True'

'''
# set Filing_Check to 'True' for index 515
filing_page_df.loc[515, 'Filing_Check'] = 'True'

In [134]:
# notes on 'False' observation for AEE
# observe financial statement data in funda_df and compare to financial statement data on relevant filing page of SEC EDGAR database
'''

AMEREN CORP
Data_Date: 2011-12-31 - filing url corresponds to correct filing (financial statement data matches)
As the observations correspond to the correct filings, the Filing_Check column of filing_page_df can be set to 'True'

'''
# set Filing_Check to 'True' for index 584
filing_page_df.loc[584, 'Filing_Check'] = 'True'

In [135]:
# notes on 'False' observations for ANF
# observe financial statement data in funda_df and compare to financial statement data on relevant filing page of SEC EDGAR database
'''

ABERCROMBIE & FITCH -CL A
Data_Date: 2017-01-31 - filing url corresponds to correct filing (financial statement data matches)
Data_Date: 2018-01-31 - filing url corresponds to correct filing (financial statement data matches)
Data_Date: 2019-01-31 - filing url corresponds to correct filing (financial statement data matches)
As the observations correspond to the correct filings, the Filing_Check column of filing_page_df can be set to 'True'

'''
# set Filing_Check to 'True' for indexes 617 through to 619
for idx in range(617, 620):
    filing_page_df.loc[idx, 'Filing_Check'] = 'True'

In [136]:
# notes on 'False' observations for UNFI
# observe financial statement data in funda_df and compare to financial statement data on relevant filing page of SEC EDGAR database
'''

UNITED NATURAL FOODS INC
Data_Date: 2017-07-31 - filing url corresponds to correct filing (financial statement data matches)
Data_Date: 2018-07-31 - filing url corresponds to correct filing (financial statement data matches)
As the observations correspond to the correct filings, the Filing_Check column of filing_page_df can be set to 'True'

'''
# set Filing_Check to 'True' for indexes 624 and 625
for idx in range(624, 626):
    filing_page_df.loc[idx, 'Filing_Check'] = 'True'

In [137]:
# notes on 'False' observations for FDP
# observe financial statement data in funda_df and compare to financial statement data on relevant filing page of SEC EDGAR database
'''

FRESH DEL MONTE PRODUCE INC
Data_Date: 2011-12-31 - filing url corresponds to correct filing (financial statement data matches)
Data_Date: 2012-12-31 - filing url corresponds to correct filing (financial statement data matches)
Data_Date: 2013-12-31 - filing url corresponds to correct filing (financial statement data matches)
Data_Date: 2014-12-31 - filing url corresponds to correct filing (financial statement data matches)
Data_Date: 2015-12-31 - filing url corresponds to correct filing (financial statement data matches)
Data_Date: 2018-12-31 - filing url corresponds to correct filing (financial statement data matches)
As the observations correspond to the correct filings, the Filing_Check column of filing_page_df can be set to 'True'

'''
# set Filing_Check to 'True' for indexes 658 through to 663
for idx in range(658, 664):
    filing_page_df.loc[idx, 'Filing_Check'] = 'True'

In [138]:
# notes on 'False' observations for BB
# observe financial statement data in funda_df and compare to financial statement data on relevant filing page of SEC EDGAR database
'''

BLACKBERRY LTD
Data_Date: 2012-02-29 - filing url corresponds to correct filing (financial statement data matches)
Data_Date: 2013-02-28 - filing url corresponds to correct filing (financial statement data matches)
As the observations correspond to the correct filings, the Filing_Check column of filing_page_df can be set to 'True'

'''
# set Filing_Check to 'True' for indexes 698 and 699
for idx in range(698, 700):
    filing_page_df.loc[idx, 'Filing_Check'] = 'True'

In [139]:
# notes on 'False' observations for AGI
# observe financial statement data in funda_df and compare to financial statement data on relevant filing page of SEC EDGAR database
'''

ALAMOS GOLD INC
Data_Date: 2016-12-31 - filing url corresponds to correct filing (financial statement data matches)
Data_Date: 2018-12-31 - filing url corresponds to correct filing (financial statement data matches)
As the observations correspond to the correct filings, the Filing_Check column of filing_page_df can be set to 'True'

'''
# set Filing_Check to 'True' for indexes 711 and 713
for idx in range(711, 714):
    if idx==712:
        continue
    else:
        filing_page_df.loc[idx, 'Filing_Check'] = 'True'

In [140]:
# notes on 'False' observations for BBW
# observe financial statement data in funda_df and compare to financial statement data on relevant filing page of SEC EDGAR database
'''

BUILD-A-BEAR WORKSHOP INC
Data_Date: 2012-12-31 - filing url corresponds to correct filing (financial statement data matches)
Data_Date: 2019-01-31 - filing url corresponds to correct filing (financial statement data matches)
As the observations correspond to the correct filings, the Filing_Check column of filing_page_df can be set to 'True'

'''
# set Filing_Check to 'True' for indexes 748 and 749
for idx in range(748, 750):
    filing_page_df.loc[idx, 'Filing_Check'] = 'True'

In [141]:
# notes on 'False' observations for ITGR
# observe financial statement data in funda_df and compare to financial statement data on relevant filing page of SEC EDGAR database
'''

INTEGER HOLDINGS CORP
Data_Date: 2011-12-31 - filing url corresponds to correct filing (financial statement data matches)
Data_Date: 2012-12-31 - filing url corresponds to correct filing (financial statement data matches)
As the observations correspond to the correct filings, the Filing_Check column of filing_page_df can be set to 'True'

'''
# set Filing_Check to 'True' for indexes 750 and 751
for idx in range(750, 752):
    filing_page_df.loc[idx, 'Filing_Check'] = 'True'

In [142]:
# notes on 'False' observations for JCP
# observe financial statement data in funda_df and compare to financial statement data on relevant filing page of SEC EDGAR database
'''

PENNEY (J C) CO
Data_Date: 2014-01-31 - filing url corresponds to correct filing (financial statement data matches)
Data_Date: 2018-01-31 - filing url corresponds to correct filing (financial statement data matches)
Data_Date: 2019-01-31 - filing url corresponds to correct filing (financial statement data matches)
As the observations correspond to the correct filings, the Filing_Check column of filing_page_df can be set to 'True'

'''
# set Filing_Check to 'True' for indexes 818 through to 821 (excluding 819)
for idx in range(818, 822):
    if idx==819:
        continue
    else:
        filing_page_df.loc[idx, 'Filing_Check'] = 'True'

In [143]:
# notes on 'False' observation for AUY
# observe financial statement data in funda_df and compare to financial statement data on relevant filing page of SEC EDGAR database
'''

YAMANA GOLD INC
Data_Date: 2016-12-31 - filing url corresponds to correct filing (financial statement data matches)
As the observations correspond to the correct filings, the Filing_Check column of filing_page_df can be set to 'True'

'''
# set Filing_Check to 'True' for index 859
filing_page_df.loc[859, 'Filing_Check'] = 'True'

In [144]:
# notes on 'False' observations for GME
# observe financial statement data in funda_df and compare to financial statement data on relevant filing page of SEC EDGAR database
'''

GAMESTOP CORP
Data_Date: 2011-01-31 - filing url corresponds to correct filing (financial statement data matches)
Data_Date: 2012-01-31 - filing url corresponds to correct filing (financial statement data matches)
Data_Date: 2013-01-31 - filing url corresponds to correct filing (financial statement data matches)
Data_Date: 2017-01-31 - filing url corresponds to correct filing (financial statement data matches)
Data_Date: 2018-01-31 - filing url corresponds to correct filing (financial statement data matches)
Data_Date: 2019-01-31 - filing url corresponds to correct filing (financial statement data matches)
As the observations correspond to the correct filings, the Filing_Check column of filing_page_df can be set to 'True'

'''
# set Filing_Check to 'True' for indexes 931 through to 936
for idx in range(931, 937):
    filing_page_df.loc[idx, 'Filing_Check'] = 'True'

In [145]:
# notes on 'False' observations for VSI
# observe financial statement data in funda_df and compare to financial statement data on relevant filing page of SEC EDGAR database
'''

VITAMIN SHOPPE INC
Data_Date: 2017-12-31 - filing url corresponds to correct filing (financial statement data matches)
Data_Date: 2018-12-31 - filing url corresponds to correct filing (financial statement data matches)
As the observations correspond to the correct filings, the Filing_Check column of filing_page_df can be set to 'True'

'''
# set Filing_Check to 'True' for indexes 988 and 989
for idx in range(988, 990):
    filing_page_df.loc[idx, 'Filing_Check'] = 'True'

In [146]:
# notes on 'False' observation for TCS
# observe financial statement data in funda_df and compare to financial statement data on relevant filing page of SEC EDGAR database
'''

CONTAINER STORE GROUP
Data_Date: 2017-03-31 - filing url corresponds to correct filing (financial statement data matches)
As the observations correspond to the correct filings, the Filing_Check column of filing_page_df can be set to 'True'

'''
# set Filing_Check to 'True' for index 1070
filing_page_df.loc[1070, 'Filing_Check'] = 'True'

In [147]:
# notes on 'False' observations for EXPR
# observe financial statement data in funda_df and compare to financial statement data on relevant filing page of SEC EDGAR database
'''

EXPRESS INC
Data_Date: 2018-01-31 - filing url corresponds to correct filing (financial statement data matches)
Data_Date: 2019-01-31 - filing url corresponds to correct filing (financial statement data matches)
As the observations correspond to the correct filings, the Filing_Check column of filing_page_df can be set to 'True'

'''
# set Filing_Check to 'True' for indexes 1174 and 1175
for idx in range(1174, 1176):
    filing_page_df.loc[idx, 'Filing_Check'] = 'True'

In [148]:
# notes on 'False' observation for STNG
# observe financial statement data in funda_df and compare to financial statement data on relevant filing page of SEC EDGAR database
'''

SCORPIO TANKERS INC
Data_Date: 2015-12-31 - filing url corresponds to correct filing (financial statement data matches)
As the observations correspond to the correct filings, the Filing_Check column of filing_page_df can be set to 'True'

'''
# set Filing_Check to 'True' for index 1177
filing_page_df.loc[1177, 'Filing_Check'] = 'True'

In [149]:
# notes on 'False' observation for TLYS
# observe financial statement data in funda_df and compare to financial statement data on relevant filing page of SEC EDGAR database
'''

TILLY'S INC
Data_Date: 2016-01-31 - filing url corresponds to correct filing (financial statement data matches)
As the observations correspond to the correct filings, the Filing_Check column of filing_page_df can be set to 'True'

'''
# set Filing_Check to 'True' for index 1232
filing_page_df.loc[1232, 'Filing_Check'] = 'True'

In [150]:
# notes on 'False' observation for RH
# observe financial statement data in funda_df and compare to financial statement data on relevant filing page of SEC EDGAR database
'''

RH
Data_Date: 2017-01-31 - filing url corresponds to correct filing (financial statement data matches)
As the observations correspond to the correct filings, the Filing_Check column of filing_page_df can be set to 'True'

'''
# set Filing_Check to 'True' for index 1241
filing_page_df.loc[1241, 'Filing_Check'] = 'True'

In [151]:
# notes on 'False' observation for BEP
# observe financial statement data in funda_df and compare to financial statement data on relevant filing page of SEC EDGAR database
'''

BROOKFIELD RENEWABLE PRTS LP
Data_Date: 2015-12-31 - filing url corresponds to correct filing (financial statement data matches)
As the observations correspond to the correct filings, the Filing_Check column of filing_page_df can be set to 'True'

'''
# set Filing_Check to 'True' for index 1253
filing_page_df.loc[1253, 'Filing_Check'] = 'True'

In [152]:
# notes on 'False' observation for CPG
# observe financial statement data in funda_df and compare to financial statement data on relevant filing page of SEC EDGAR database
'''

CRESCENT POINT ENERGY CORP
Data_Date: 2015-12-31 - filing url corresponds to correct filing (financial statement data matches)
As the observations correspond to the correct filings, the Filing_Check column of filing_page_df can be set to 'True'

'''
# set Filing_Check to 'True' for index 1276
filing_page_df.loc[1276, 'Filing_Check'] = 'True'

In [153]:
# notes on 'False' observations for MNK
# observe financial statement data in funda_df and compare to financial statement data on relevant filing page of SEC EDGAR database
'''

MALLINCKRODT PLC
Data_Date: 2017-12-31 - filing url corresponds to correct filing (financial statement data matches)
Data_Date: 2018-12-31 - filing url corresponds to correct filing (financial statement data matches)
As the observations correspond to the correct filings, the Filing_Check column of filing_page_df can be set to 'True'

'''
# set Filing_Check to 'True' for indexes 1299 and 1300
for idx in range(1299, 1301):
    filing_page_df.loc[idx, 'Filing_Check'] = 'True'

In [154]:
# notes on 'False' observation for VNCE
# observe financial statement data in funda_df and compare to financial statement data on relevant filing page of SEC EDGAR database
'''

VINCE HOLDING CORP
Data_Date: 2018-01-31 - filing url corresponds to correct filing (financial statement data matches)
As the observations correspond to the correct filings, the Filing_Check column of filing_page_df can be set to 'True'

'''
# set Filing_Check to 'True' for index 1313
filing_page_df.loc[1313, 'Filing_Check'] = 'True'

In [155]:
# notes on 'False' observations for NEWM
# observe financial statement data in funda_df and compare to financial statement data on relevant filing page of SEC EDGAR database
'''

NEW MEDIA INVESTMENT GROUP
Data_Date: 2016-12-31 - filing url corresponds to correct filing (financial statement data matches)
Data_Date: 2018-12-31 - filing url corresponds to correct filing (financial statement data matches)
As the observations correspond to the correct filings, the Filing_Check column of filing_page_df can be set to 'True'

'''
# set Filing_Check to 'True' for indexes 1314 and 1315
for idx in range(1314, 1316):
    filing_page_df.loc[idx, 'Filing_Check'] = 'True'

In [156]:
# notes on 'False' observation for JAX
# observe financial statement data in funda_df and compare to financial statement data on relevant filing page of SEC EDGAR database
'''

J. ALEXANDER'S HOLDINGS INC
Data_Date: 2018-12-31 - filing url corresponds to correct filing (financial statement data matches)
As the observations correspond to the correct filings, the Filing_Check column of filing_page_df can be set to 'True'

'''
# set Filing_Check to 'True' for index 1394
filing_page_df.loc[1394, 'Filing_Check'] = 'True'

In [157]:
# notes on 'False' observation for SUM
# observe financial statement data in funda_df and compare to financial statement data on relevant filing page of SEC EDGAR database
'''

SUMMIT MATERIALS INC
Data_Date: 2018-12-31 - filing url corresponds to correct filing (financial statement data matches)
As the observations correspond to the correct filings, the Filing_Check column of filing_page_df can be set to 'True'

'''
# set Filing_Check to 'True' for index 1397
filing_page_df.loc[1397, 'Filing_Check'] = 'True'

In [158]:
# notes on 'False' observations for BNED
# observe financial statement data in funda_df and compare to financial statement data on relevant filing page of SEC EDGAR database
'''

BARNES & NOBLE EDUCATION INC
Data_Date: 2018-04-30 - filing url corresponds to correct filing (financial statement data matches)
Data_Date: 2018-04-30 - filing url corresponds to correct filing (financial statement data matches)
As the observations correspond to the correct filings, the Filing_Check column of filing_page_df can be set to 'True'

'''
# set Filing_Check to 'True' for indexes 1404 and 1405
for idx in range(1404, 1406):
    filing_page_df.loc[idx, 'Filing_Check'] = 'True'

In [159]:
# notes on 'False' observations for BH
# observe financial statement data in funda_df and compare to financial statement data on relevant filing page of SEC EDGAR database
'''

OBH INC
Data_Date: 2011-09-30 - filing url corresponds to correct filing (financial statement data matches)
Data_Date: 2012-09-30 - filing url corresponds to correct filing (financial statement data matches)
Data_Date: 2013-09-30 - filing url corresponds to correct filing (financial statement data matches)
Data_Date: 2014-09-30 - filing url corresponds to correct filing (financial statement data matches)
As the observations correspond to the correct filings, the Filing_Check column of filing_page_df can be set to 'True'

'''
# set Filing_Check to 'True' for indexes 1438 through to 1441
for idx in range(1438, 1442):
    filing_page_df.loc[idx, 'Filing_Check'] = 'True'

## Retrieve Filing Report Urls

In [160]:
# note: obtain the filing report url for each observation in filing_page_df

# create a new column to store filing report urls in filing_page_df
filing_page_df['Filing_Report_Url'] = ''
# set Filing_Report_Url of 'N/A' observations in filing_page_df to 'N/A'
filing_page_df.loc[filing_page_df['Filing_Url']=='N/A', 'Filing_Report_Url'] = 'N/A'

In [161]:
# SEC EDGAR filing report function
def edgar_filing_report(filing_url, mda_placement):
    if filing_url=='N/A':
        filing_report_url = 'N/A'
    else:
        # define base url for filing report url
        edgar_filing_page_base_url = r'https://www.sec.gov'
        # request filing page content
        filing_page_request = requests.get(url=filing_url)
        # parse filing_page_request response html
        filing_page_soup = BeautifulSoup(filing_page_request.content, 'html.parser')
        # locate tables on filing page
        filing_page_tables = filing_page_soup.find_all('table', class_='tableFile')
        # set return variable to 'N/A'
        filing_report_url = 'N/A'
        # loop over files in the first table on filing page and identify relevant filing report url
        for idx, row in enumerate(filing_page_tables[0].find_all('tr')):
            # find all columns on row
            row_column = row.find_all('td')
            # retrieve filing report url
            if len(row_column)!=0 and 'iXBRL' not in row_column[2].text.strip() and (row_column[3].text.strip()==mda_placement or
                                                                                     row_column[1].text.strip()==mda_placement):
                filing_report_url = edgar_filing_page_base_url + row_column[2].find('a', {'href':True})['href']
                break
            elif len(row_column)!=0 and 'iXBRL' in row_column[2].text.strip() and (row_column[3].text.strip()==mda_placement or
                                                                                   row_column[1].text.strip()==mda_placement):
                filing_report_url = edgar_filing_page_base_url + row_column[2].find('a', {'href':True})['href']
                filing_report_url = filing_report_url.replace('ix?doc=/', '')
                break
            elif len(row_column)!=0:
                continue
            elif idx==0:
                continue
            else:
                break
    return filing_report_url

In [162]:
# time 
start = time.time()

# retrieve filing_report_url for each observation in filing_page_df
for idx, row in filing_page_df.iterrows():
    filing_page_df.loc[idx, 'Filing_Report_Url'] = edgar_filing_report(row['Filing_Url'], row['MDA_Placement'])
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(filing_page_df.loc[idx, 'Ticker_Symbol'])
    print('period of report:', filing_page_df.loc[idx, 'Data_Date'].strftime('%Y-%m-%d'))
    print('filing date:', filing_page_df.loc[idx, 'Filing_Date'])
    print('filing type:', filing_page_df.loc[idx, 'Filing_Type'])
    print('management discussion and analysis placement:', filing_page_df.loc[idx, 'MDA_Placement'])
    print('filing report url:', filing_page_df.loc[idx, 'Filing_Report_Url'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 0.5 seconds 
    time.sleep(0.5)

end = time.time()
print('time taken:', end - start)

-------------------------------------------------------------------------------------
AIR
period of report: 2012-05-31
filing date: 2012-07-19
filing type: 10-K
management discussion and analysis placement: 10-K
filing report url: https://www.sec.gov/Archives/edgar/data/1750/000104746912007300/a2210166z10-k.htm


-------------------------------------------------------------------------------------
AIR
period of report: 2013-05-31
filing date: 2013-07-26
filing type: 10-K
management discussion and analysis placement: 10-K
filing report url: https://www.sec.gov/Archives/edgar/data/1750/000104746913007797/a2216052z10-k.htm


-------------------------------------------------------------------------------------
AIR
period of report: 2014-05-31
filing date: 2014-07-17
filing type: 10-K
management discussion and analysis placement: 10-K
filing report url: https://www.sec.gov/Archives/edgar/data/1750/000104746914006243/a2220733z10-k.htm


-------------------------------------------------------

-------------------------------------------------------------------------------------
SSI
period of report: 2017-01-31
filing date: 2017-04-04
filing type: 10-K
management discussion and analysis placement: 10-K
filing report url: https://www.sec.gov/Archives/edgar/data/6885/000000688517000079/ssi-2017_10k.htm


-------------------------------------------------------------------------------------
SSI
period of report: 2018-01-31
filing date: 2018-04-10
filing type: 10-K
management discussion and analysis placement: 10-K
filing report url: https://www.sec.gov/Archives/edgar/data/6885/000000688518000135/ssi-20180203_10k.htm


-------------------------------------------------------------------------------------
SSI
period of report: 2019-01-31
filing date: 2019-04-05
filing type: 10-K
management discussion and analysis placement: 10-K
filing report url: https://www.sec.gov/Archives/edgar/data/6885/000000688519000109/ssi-20190202_10k.htm


--------------------------------------------------

-------------------------------------------------------------------------------------
LUB
period of report: 2013-08-31
filing date: 2013-11-12
filing type: 10-K
management discussion and analysis placement: 10-K
filing report url: https://www.sec.gov/Archives/edgar/data/16099/000143774913014511/lub20130827_10k.htm


-------------------------------------------------------------------------------------
LUB
period of report: 2014-08-31
filing date: 2014-11-12
filing type: 10-K
management discussion and analysis placement: 10-K
filing report url: https://www.sec.gov/Archives/edgar/data/16099/000143774914020335/lub20140827_10k.htm


-------------------------------------------------------------------------------------
LUB
period of report: 2015-08-31
filing date: 2015-11-09
filing type: 10-K
management discussion and analysis placement: 10-K
filing report url: https://www.sec.gov/Archives/edgar/data/16099/000143774915020265/lub20150826_10k.htm


----------------------------------------------

-------------------------------------------------------------------------------------
CMC
period of report: 2012-08-31
filing date: 2012-10-30
filing type: 10-K
management discussion and analysis placement: 10-K
filing report url: https://www.sec.gov/Archives/edgar/data/22444/000002244412000006/cmc-8312012x10k.htm


-------------------------------------------------------------------------------------
TAP
period of report: 2011-12-31
filing date: 2012-02-27
filing type: 10-K
management discussion and analysis placement: 10-K
filing report url: https://www.sec.gov/Archives/edgar/data/24545/000144530512000454/tap2011123110-k.htm


-------------------------------------------------------------------------------------
TAP
period of report: 2012-12-31
filing date: 2013-02-22
filing type: 10-K
management discussion and analysis placement: 10-K
filing report url: https://www.sec.gov/Archives/edgar/data/24545/000002454513000005/tap2012122910-k.htm


----------------------------------------------

-------------------------------------------------------------------------------------
EBF
period of report: 2013-02-28
filing date: 2013-05-10
filing type: 10-K
management discussion and analysis placement: 10-K
filing report url: https://www.sec.gov/Archives/edgar/data/33002/000119312513214067/d445052d10k.htm


-------------------------------------------------------------------------------------
EBF
period of report: 2014-02-28
filing date: 2014-05-09
filing type: 10-K
management discussion and analysis placement: 10-K
filing report url: https://www.sec.gov/Archives/edgar/data/33002/000119312514192331/d671747d10k.htm


-------------------------------------------------------------------------------------
EQT
period of report: 2015-12-31
filing date: 2016-02-11
filing type: 10-K
management discussion and analysis placement: 10-K
filing report url: https://www.sec.gov/Archives/edgar/data/33213/000003321316000018/eqt-12312015x10k.htm


-----------------------------------------------------

...

-------------------------------------------------------------------------------------
KN
period of report: 2017-12-31
filing date: 2018-02-20
filing type: 10-K
management discussion and analysis placement: 10-K
filing report url: https://www.sec.gov/Archives/edgar/data/1587523/000158752318000006/knowles20171231-10xk.htm


-------------------------------------------------------------------------------------
KN
period of report: 2018-12-31
filing date: 2019-02-19
filing type: 10-K
management discussion and analysis placement: 10-K
filing report url: https://www.sec.gov/Archives/edgar/data/1587523/000158752319000004/knowles20181231-10xk.htm


-------------------------------------------------------------------------------------
CVEO
period of report: 2016-12-31
filing date: 2017-02-23
filing type: 10-K
management discussion and analysis placement: 10-K
filing report url: https://www.sec.gov/Archives/edgar/data/1590584/000143774917003062/oisasc20161231_10k.htm


----------------------------

-------------------------------------------------------------------------------------
TMST
period of report: 2018-12-31
filing date: 2019-02-20
filing type: 10-K
management discussion and analysis placement: 10-K
filing report url: https://www.sec.gov/Archives/edgar/data/1598428/000159842819000018/tmst10-k12312018.htm


-------------------------------------------------------------------------------------
GLOP
period of report: 2016-12-31
filing date: 2017-02-13
filing type: 20-F
management discussion and analysis placement: 20-F
filing report url: https://www.sec.gov/Archives/edgar/data/1598655/000093041317000414/c87353_20f.htm


-------------------------------------------------------------------------------------
GLOP
period of report: 2017-12-31
filing date: 2018-02-12
filing type: 20-F
management discussion and analysis placement: 20-F
filing report url: https://www.sec.gov/Archives/edgar/data/1598655/000104746918000752/a2234434z20-f.htm


-------------------------------------------

-------------------------------------------------------------------------------------
ATTO
period of report: 2018-12-31
filing date: 2019-03-20
filing type: 20-F
management discussion and analysis placement: 20-F
filing report url: https://www.sec.gov/Archives/edgar/data/1606457/000129281419000802/attoform20f_2018.htm


-------------------------------------------------------------------------------------
VSLR
period of report: 2016-12-31
filing date: 2017-03-16
filing type: 10-K
management discussion and analysis placement: 10-K
filing report url: https://www.sec.gov/Archives/edgar/data/1607716/000156459017004608/vslr-10k_20161231.htm


-------------------------------------------------------------------------------------
VSLR
period of report: 2017-12-31
filing date: 2018-03-07
filing type: 10-K
management discussion and analysis placement: 10-K
filing report url: https://www.sec.gov/Archives/edgar/data/1607716/000156459018004717/vslr-10k_20171231.htm


--------------------------------

-------------------------------------------------------------------------------------
GCI
period of report: 2018-12-31
filing date: 2019-02-27
filing type: 10-K
management discussion and analysis placement: 10-K
filing report url: https://www.sec.gov/Archives/edgar/data/1635718/000163571819000008/gci-20181231x10k.htm


-------------------------------------------------------------------------------------
EXTN
period of report: 2018-12-31
filing date: 2019-02-27
filing type: 10-K
management discussion and analysis placement: 10-K
filing report url: https://www.sec.gov/Archives/edgar/data/1635881/000163588119000006/extn-20181231x10k.htm


-------------------------------------------------------------------------------------
HPE
period of report: 2018-10-31
filing date: 2018-12-12
filing type: 10-K
management discussion and analysis placement: 10-K
filing report url: https://www.sec.gov/Archives/edgar/data/1645590/000162828018015054/hpe-10312018x10k.htm


-----------------------------------

-------------------------------------------------------------------------------------
HPR
period of report: 2018-12-31
filing date: 2019-02-26
filing type: 10-K
management discussion and analysis placement: 10-K
filing report url: https://www.sec.gov/Archives/edgar/data/1725526/000172552619000062/hpr-12312018x10xk.htm


-------------------------------------------------------------------------------------
NTR
period of report: 2018-12-31
filing date: 2019-02-27
filing type: 40-F
management discussion and analysis placement: EX-99.2
filing report url: https://www.sec.gov/Archives/edgar/data/1725964/000119312519053855/d636808dex992.htm


-------------------------------------------------------------------------------------
BH
period of report: 2011-09-30
filing date: 2011-12-12
filing type: 10-K
management discussion and analysis placement: 10-K
filing report url: https://www.sec.gov/Archives/edgar/data/93859/000119380511002350/form10k07428_09282011.htm


----------------------------------

## Save filing_page_df to CSV File

In [163]:
# save filing_page_df to csv file 
filing_page_df.to_csv('filing_page.csv', index=False)